<a href="https://colab.research.google.com/github/JosePu1201/proyectoViolencia/blob/master/PracticaViolencia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## instalacion de dependencias
import numpy as np
import matplotlib as mpl
import pandas as pd

In [2]:
## instalacion de dependeica de web scraping
from bs4 import BeautifulSoup
## para tener peticiones
import requests
!pip install selenium
import time
from selenium import webdriver


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 26.4 MB/s eta 0:00:00


In [3]:
# PARA NORMALIZAR
!pip install unidecode

from unidecode import unidecode

def normalizar_texto(texto):
    if isinstance(texto, str):
        return unidecode(texto).strip().title()
    return texto

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 7.1 MB/s eta 0:00:00


In [158]:
import re
from datetime import datetime
# Función para limpiar valores
def limpiar_texto(texto):
    if pd.isna(texto):
        return None

    # Convertir a string si no lo es
    if not isinstance(texto, str):
        texto = str(texto)

    # Eliminar espacios extra y caracteres no deseados
    texto = re.sub(r'\s+', ' ', texto).strip()
    return None if texto == '' else texto


# 1) Diccionario mes → número
mes_map = {
    'ENERO': 1, 'FEBRERO': 2, 'MARZO': 3, 'ABRIL': 4,
    'MAYO': 5, 'JUNIO': 6, 'JULIO': 7, 'AGOSTO': 8,
    'SEPTIEMBRE': 9, 'OCTUBRE': 10, 'NOVIEMBRE': 11, 'DICIEMBRE': 12
}

def convertir_fecha(mes_texto, anio):
    try:
        # 2) Chequeo de nulos
        if pd.isna(mes_texto) or pd.isna(anio):
            return None

        # 3) Normalizar el texto del mes
        mes_str = str(mes_texto).strip().upper()
        mes_num = mes_map.get(mes_str)
        if mes_num is None:
            return None

        # 4) Validar año
        anio = int(anio)
        if anio < 1900 or anio > 2100:
            return None

        # 5) Fijar día en 1 y construir la fecha
        dia = 1
        return datetime(anio, mes_num, dia).strftime('%Y-%m-%d')

    except Exception:
        return None

#normalizar fecha con dia
def convertir_fecha_dia(mes_texto, anio,dia):
    try:
        # 2) Chequeo de nulos
        if pd.isna(mes_texto) or pd.isna(anio) or pd.isna(dia):
            return None

        # 3) Normalizar el texto del mes
        mes_str = str(mes_texto).strip().upper()
        mes_num = mes_map.get(mes_str)
        if mes_num is None:
            return None

        # 4) Validar año
        anio = int(anio)
        dia = int(dia)
        if anio < 1900 or anio > 2100:
            return None
        if dia < 1 or dia > 31:
            return None
        # 5) Fijar día en 1 y construir la fecha
        return datetime(anio, mes_num, dia).strftime('%Y-%m-%d')

    except Exception:
        return None
# Función para normalizar nombres de departamentos y municipios
def normalizar_ubicacion(valor):
    if pd.isna(valor):
        return None

    # Convertir a string y limpiar
    valor = limpiar_texto(valor)
    if valor is None:
        return None

    # Convertir todo a mayúsculas sin acentos para normalizar
    valor = valor.upper()
    valor = re.sub(r'Á', 'A', valor)
    valor = re.sub(r'É', 'E', valor)
    valor = re.sub(r'Í', 'I', valor)
    valor = re.sub(r'Ó', 'O', valor)
    valor = re.sub(r'Ú', 'U', valor)

    return valor

# Función para normalizar edades
def normalizar_edad(edad):
    try:
        if pd.isna(edad):
            return None

        edad = float(edad)
        if edad < 0 or edad > 120:  # Validar rango razonable
            return None

        return int(edad)
    except:
        return None

# Función para normalizar sexo
def normalizar_sexo(sexo):
    if pd.isna(sexo):
        return None

    sexo = str(sexo).upper().strip()

    if sexo in ['M', 'MASCULINO', 'HOMBRE', 'H','HOMBRES']:
        return 'Hombre'
    elif sexo in ['F', 'FEMENINO', 'MUJER','MUJERES']:
        return 'Mujer'
    else:
        return 'Ignorado'
#normalizar tipo falta
def normalizar_falta(falta):
    if pd.isna(falta):
        return None

    falta_normalizada = normalizar_texto(falta)
    falta_normalizada = falta_normalizada.upper().strip()
    if falta_normalizada in ['CONTRA LAS PERSONAS', 'LAS PERSONAS']:
        return 'Las Personas'
    elif falta_normalizada in ['CONTRA LA PROPIEDAD', 'LA PROPIEDAD']:
        return 'La Propiedad'
    elif falta_normalizada in ['CONTRA LAS BUENAS COSTUMBRES', 'LAS BUENAS COSTUMBRES']:
        return 'Las buenas costumbres'
    elif falta_normalizada in ['CONTRA EL ORDEN PUBLICO', 'ORDEN PUBLICO']:
        return 'Orden publico'
    elif falta_normalizada in ['OTRAS']:
        return 'Otras'
    else:
        return 'Ignorado'


In [5]:
#Diccionarios
tipo_falta_map = {
    1: "Las Personas",
    2: "La Propiedad",
    3: "Las buenas costumbres",
    4: "Orden publico",
    5: "Otras",
    6: "Ignorado"
}

sexo_map = {
    1: "Hombre",
    2: "Mujer",
    3: "Ignorado"
}

grupo_etnico_map = {
    1: "Indigena",
    2: "No Indigena",
    3: "Ignorado"
}

estado_conyugal_map = {
    1: "Soltero(a)",
    2: "Casado(a)",
    3: "Unido(a)",
    4: "Viudo(a)",
    5: "Divorciado(a)",
    6: "Ignorado"
}

escolaridad_map = {
    1: "Preprimaria",
    2: "Primaria",
    3: "Basico",
    4: "Diversificado",
    5: "Superior",
    6: "Ninguno",
    7: "Ignorado"
}

area_geografica_map = {
    1: "Urbana",
    2: "Rural",
    3: "Ignorada"
}

condicion_infractor_map = {
    1: "Ebrio",
    2: "No Ebrio",
    9: "Ignorado"
}
alfabetismo_map = {
    1: "Alfabeta",
    2: "Analfabeta",
    9: "Ignorado"
}


In [7]:
urlExcel1 = '/content/faltasJudiciales.xlsx'
cargaExcel1 = pd.read_excel(urlExcel1, sheet_name='Sheet1', header=0)
fJudiciales = pd.DataFrame(cargaExcel1)
fJudiciales.head()
#dfExcelLocal[dfExcelLocal["League"] == "English Premier League"].head(100)
#dfExcelLocal.head(100)

,Núm_corre,Depto_boleta,Muni_boleta,Mes_boleta,Año_boleta,Falta_inf,Sexo_inf,Edad_inf,Grupo_étnico_inf,Est_conyugal_inf,...,G_primarios,Área_geo_inf,Est_ebriedad_inf,g_edad_80ymás,g_edad_60ymás,edad_quinquenales,Depto_nacimiento_inf,Depto_nacimiento_inf.1,Subg_principales,Gran_grupos
0,1,Guatemala,Santa Catarina Pinula,Enero,2023,Contra las buenas costumbres,Hombres,32,No indígena,Casado(a),...,Agricultores y trabajadores calificados de cul...,Urbana,Ebrio,30-34,30-34,30 - 34,Guatemala,Guatemalteco (a),Agricultores y trabajadores calificados de exp...,Agricultores y trabajadores calificados agrope...
1,2,Guatemala,Santa Catarina Pinula,Enero,2023,Contra el orden público,Hombres,32,No indígena,Casado(a),...,Agricultores y trabajadores calificados de cul...,Urbana,Ebrio,30-34,30-34,30 - 34,Guatemala,Guatemalteco (a),Agricultores y trabajadores calificados de exp...,Agricultores y trabajadores calificados agrope...
2,3,Guatemala,Santa Catarina Pinula,Enero,2023,Contra las buenas costumbres,Hombres,24,No indígena,Soltero(a),...,Estudiantes,Urbana,Ebrio,20-24,20-24,20 - 24,Guatemala,Guatemalteco (a),Estudiantes,Categorias funcionales de la No Pea
3,4,Guatemala,Santa Catarina Pinula,Enero,2023,Contra el orden público,Hombres,24,No indígena,Soltero(a),...,Estudiantes,Urbana,Ebrio,20-24,20-24,20 - 24,Guatemala,Guatemalteco (a),Estudiantes,Categorias funcionales de la No Pea
4,5,Guatemala,Santa Catarina Pinula,Enero,2023,Otras,Hombres,35,No indígena,Soltero(a),...,Vendedores no clasificados bajo otros epígrafes,Rural,Ebrio,35-39,35-39,35 - 39,Guatemala,Guatemalteco (a),Vendedores,Trabajadores de los servicios y vendedores de ...


In [8]:
#convertir fechas aun no funciona
fJudiciales['fecha'] = fJudiciales.apply(
    lambda r: convertir_fecha(r['Mes_boleta'], r['Año_boleta']),
    axis=1
)
fJudiciales = fJudiciales.drop(columns=['Año_boleta', 'Mes_boleta','Depto_nacimiento_inf.1'])
fJudiciales.head()

,Núm_corre,Depto_boleta,Muni_boleta,Falta_inf,Sexo_inf,Edad_inf,Grupo_étnico_inf,Est_conyugal_inf,Nacimiento_inf,Cond_alfabetismo_inf,...,G_primarios,Área_geo_inf,Est_ebriedad_inf,g_edad_80ymás,g_edad_60ymás,edad_quinquenales,Depto_nacimiento_inf,Subg_principales,Gran_grupos,fecha
0,1,Guatemala,Santa Catarina Pinula,Contra las buenas costumbres,Hombres,32,No indígena,Casado(a),Guatemala,Alfabeta,...,Agricultores y trabajadores calificados de cul...,Urbana,Ebrio,30-34,30-34,30 - 34,Guatemala,Agricultores y trabajadores calificados de exp...,Agricultores y trabajadores calificados agrope...,2023-01-01
1,2,Guatemala,Santa Catarina Pinula,Contra el orden público,Hombres,32,No indígena,Casado(a),Guatemala,Alfabeta,...,Agricultores y trabajadores calificados de cul...,Urbana,Ebrio,30-34,30-34,30 - 34,Guatemala,Agricultores y trabajadores calificados de exp...,Agricultores y trabajadores calificados agrope...,2023-01-01
2,3,Guatemala,Santa Catarina Pinula,Contra las buenas costumbres,Hombres,24,No indígena,Soltero(a),Guatemala,Alfabeta,...,Estudiantes,Urbana,Ebrio,20-24,20-24,20 - 24,Guatemala,Estudiantes,Categorias funcionales de la No Pea,2023-01-01
3,4,Guatemala,Santa Catarina Pinula,Contra el orden público,Hombres,24,No indígena,Soltero(a),Guatemala,Alfabeta,...,Estudiantes,Urbana,Ebrio,20-24,20-24,20 - 24,Guatemala,Estudiantes,Categorias funcionales de la No Pea,2023-01-01
4,5,Guatemala,Santa Catarina Pinula,Otras,Hombres,35,No indígena,Soltero(a),Guatemala,Alfabeta,...,Vendedores no clasificados bajo otros epígrafes,Rural,Ebrio,35-39,35-39,35 - 39,Guatemala,Vendedores,Trabajadores de los servicios y vendedores de ...,2023-01-01


In [9]:
fJudiciales['sexo'] = fJudiciales['Sexo_inf'].apply(normalizar_sexo)
fJudiciales.head()


,Núm_corre,Depto_boleta,Muni_boleta,Falta_inf,Sexo_inf,Edad_inf,Grupo_étnico_inf,Est_conyugal_inf,Nacimiento_inf,Cond_alfabetismo_inf,...,Área_geo_inf,Est_ebriedad_inf,g_edad_80ymás,g_edad_60ymás,edad_quinquenales,Depto_nacimiento_inf,Subg_principales,Gran_grupos,fecha,sexo
0,1,Guatemala,Santa Catarina Pinula,Contra las buenas costumbres,Hombres,32,No indígena,Casado(a),Guatemala,Alfabeta,...,Urbana,Ebrio,30-34,30-34,30 - 34,Guatemala,Agricultores y trabajadores calificados de exp...,Agricultores y trabajadores calificados agrope...,2023-01-01,Hombre
1,2,Guatemala,Santa Catarina Pinula,Contra el orden público,Hombres,32,No indígena,Casado(a),Guatemala,Alfabeta,...,Urbana,Ebrio,30-34,30-34,30 - 34,Guatemala,Agricultores y trabajadores calificados de exp...,Agricultores y trabajadores calificados agrope...,2023-01-01,Hombre
2,3,Guatemala,Santa Catarina Pinula,Contra las buenas costumbres,Hombres,24,No indígena,Soltero(a),Guatemala,Alfabeta,...,Urbana,Ebrio,20-24,20-24,20 - 24,Guatemala,Estudiantes,Categorias funcionales de la No Pea,2023-01-01,Hombre
3,4,Guatemala,Santa Catarina Pinula,Contra el orden público,Hombres,24,No indígena,Soltero(a),Guatemala,Alfabeta,...,Urbana,Ebrio,20-24,20-24,20 - 24,Guatemala,Estudiantes,Categorias funcionales de la No Pea,2023-01-01,Hombre
4,5,Guatemala,Santa Catarina Pinula,Otras,Hombres,35,No indígena,Soltero(a),Guatemala,Alfabeta,...,Rural,Ebrio,35-39,35-39,35 - 39,Guatemala,Vendedores,Trabajadores de los servicios y vendedores de ...,2023-01-01,Hombre


In [10]:
fJudiciales['Depto_nacimiento_inf'] = fJudiciales['Depto_nacimiento_inf'].apply(normalizar_texto)
departamentos = fJudiciales['Depto_nacimiento_inf'].dropna().unique()
departamento_map = {posicion: idx+1 for idx, posicion in enumerate(departamentos)}
#departamento_map = departamento_map.apply(normalizar_texto)
fJudiciales['Depto_boleta'] = fJudiciales['Depto_boleta'].apply(normalizar_texto)
fJudiciales['departamento_boleta'] = fJudiciales['Depto_boleta'].map(departamento_map)
fJudiciales['departamento_nacimiento'] = fJudiciales['Depto_nacimiento_inf'].map(departamento_map)
fJudiciales.head()

,Núm_corre,Depto_boleta,Muni_boleta,Falta_inf,Sexo_inf,Edad_inf,Grupo_étnico_inf,Est_conyugal_inf,Nacimiento_inf,Cond_alfabetismo_inf,...,g_edad_80ymás,g_edad_60ymás,edad_quinquenales,Depto_nacimiento_inf,Subg_principales,Gran_grupos,fecha,sexo,departamento_boleta,departamento_nacimiento
0,1,Guatemala,Santa Catarina Pinula,Contra las buenas costumbres,Hombres,32,No indígena,Casado(a),Guatemala,Alfabeta,...,30-34,30-34,30 - 34,Guatemala,Agricultores y trabajadores calificados de exp...,Agricultores y trabajadores calificados agrope...,2023-01-01,Hombre,1,1
1,2,Guatemala,Santa Catarina Pinula,Contra el orden público,Hombres,32,No indígena,Casado(a),Guatemala,Alfabeta,...,30-34,30-34,30 - 34,Guatemala,Agricultores y trabajadores calificados de exp...,Agricultores y trabajadores calificados agrope...,2023-01-01,Hombre,1,1
2,3,Guatemala,Santa Catarina Pinula,Contra las buenas costumbres,Hombres,24,No indígena,Soltero(a),Guatemala,Alfabeta,...,20-24,20-24,20 - 24,Guatemala,Estudiantes,Categorias funcionales de la No Pea,2023-01-01,Hombre,1,1
3,4,Guatemala,Santa Catarina Pinula,Contra el orden público,Hombres,24,No indígena,Soltero(a),Guatemala,Alfabeta,...,20-24,20-24,20 - 24,Guatemala,Estudiantes,Categorias funcionales de la No Pea,2023-01-01,Hombre,1,1
4,5,Guatemala,Santa Catarina Pinula,Otras,Hombres,35,No indígena,Soltero(a),Guatemala,Alfabeta,...,35-39,35-39,35 - 39,Guatemala,Vendedores,Trabajadores de los servicios y vendedores de ...,2023-01-01,Hombre,1,1


In [11]:
fJudiciales['falta'] = fJudiciales['Falta_inf'].apply(normalizar_falta)
fJudiciales.head()

,Núm_corre,Depto_boleta,Muni_boleta,Falta_inf,Sexo_inf,Edad_inf,Grupo_étnico_inf,Est_conyugal_inf,Nacimiento_inf,Cond_alfabetismo_inf,...,g_edad_60ymás,edad_quinquenales,Depto_nacimiento_inf,Subg_principales,Gran_grupos,fecha,sexo,departamento_boleta,departamento_nacimiento,falta
0,1,Guatemala,Santa Catarina Pinula,Contra las buenas costumbres,Hombres,32,No indígena,Casado(a),Guatemala,Alfabeta,...,30-34,30 - 34,Guatemala,Agricultores y trabajadores calificados de exp...,Agricultores y trabajadores calificados agrope...,2023-01-01,Hombre,1,1,Las buenas costumbres
1,2,Guatemala,Santa Catarina Pinula,Contra el orden público,Hombres,32,No indígena,Casado(a),Guatemala,Alfabeta,...,30-34,30 - 34,Guatemala,Agricultores y trabajadores calificados de exp...,Agricultores y trabajadores calificados agrope...,2023-01-01,Hombre,1,1,Orden publico
2,3,Guatemala,Santa Catarina Pinula,Contra las buenas costumbres,Hombres,24,No indígena,Soltero(a),Guatemala,Alfabeta,...,20-24,20 - 24,Guatemala,Estudiantes,Categorias funcionales de la No Pea,2023-01-01,Hombre,1,1,Las buenas costumbres
3,4,Guatemala,Santa Catarina Pinula,Contra el orden público,Hombres,24,No indígena,Soltero(a),Guatemala,Alfabeta,...,20-24,20 - 24,Guatemala,Estudiantes,Categorias funcionales de la No Pea,2023-01-01,Hombre,1,1,Orden publico
4,5,Guatemala,Santa Catarina Pinula,Otras,Hombres,35,No indígena,Soltero(a),Guatemala,Alfabeta,...,35-39,35 - 39,Guatemala,Vendedores,Trabajadores de los servicios y vendedores de ...,2023-01-01,Hombre,1,1,Otras


In [12]:
fJudiciales['id_falta'] = fJudiciales['falta'].map({v: k for k, v in tipo_falta_map.items()})
fJudiciales['id_sexo'] = fJudiciales['sexo'].map({v: k for k, v in sexo_map.items()  })
fJudiciales = fJudiciales.drop(columns=['Sexo_inf','Falta_inf','sexo','falta'])
fJudiciales.head()

,Núm_corre,Depto_boleta,Muni_boleta,Edad_inf,Grupo_étnico_inf,Est_conyugal_inf,Nacimiento_inf,Cond_alfabetismo_inf,Niv_escolaridad_inf,G_primarios,...,g_edad_60ymás,edad_quinquenales,Depto_nacimiento_inf,Subg_principales,Gran_grupos,fecha,departamento_boleta,departamento_nacimiento,id_falta,id_sexo
0,1,Guatemala,Santa Catarina Pinula,32,No indígena,Casado(a),Guatemala,Alfabeta,Diversificado,Agricultores y trabajadores calificados de cul...,...,30-34,30 - 34,Guatemala,Agricultores y trabajadores calificados de exp...,Agricultores y trabajadores calificados agrope...,2023-01-01,1,1,3,1
1,2,Guatemala,Santa Catarina Pinula,32,No indígena,Casado(a),Guatemala,Alfabeta,Diversificado,Agricultores y trabajadores calificados de cul...,...,30-34,30 - 34,Guatemala,Agricultores y trabajadores calificados de exp...,Agricultores y trabajadores calificados agrope...,2023-01-01,1,1,4,1
2,3,Guatemala,Santa Catarina Pinula,24,No indígena,Soltero(a),Guatemala,Alfabeta,Diversificado,Estudiantes,...,20-24,20 - 24,Guatemala,Estudiantes,Categorias funcionales de la No Pea,2023-01-01,1,1,3,1
3,4,Guatemala,Santa Catarina Pinula,24,No indígena,Soltero(a),Guatemala,Alfabeta,Diversificado,Estudiantes,...,20-24,20 - 24,Guatemala,Estudiantes,Categorias funcionales de la No Pea,2023-01-01,1,1,4,1
4,5,Guatemala,Santa Catarina Pinula,35,No indígena,Soltero(a),Guatemala,Alfabeta,Primaria,Vendedores no clasificados bajo otros epígrafes,...,35-39,35 - 39,Guatemala,Vendedores,Trabajadores de los servicios y vendedores de ...,2023-01-01,1,1,5,1


In [13]:
fJudiciales ['id_etnia'] = fJudiciales['Grupo_étnico_inf'].map(normalizar_texto).map({v: k for k, v in grupo_etnico_map.items()})
fJudiciales = fJudiciales.drop(columns=['Grupo_étnico_inf'])
fJudiciales.head()

,Núm_corre,Depto_boleta,Muni_boleta,Edad_inf,Est_conyugal_inf,Nacimiento_inf,Cond_alfabetismo_inf,Niv_escolaridad_inf,G_primarios,Área_geo_inf,...,edad_quinquenales,Depto_nacimiento_inf,Subg_principales,Gran_grupos,fecha,departamento_boleta,departamento_nacimiento,id_falta,id_sexo,id_etnia
0,1,Guatemala,Santa Catarina Pinula,32,Casado(a),Guatemala,Alfabeta,Diversificado,Agricultores y trabajadores calificados de cul...,Urbana,...,30 - 34,Guatemala,Agricultores y trabajadores calificados de exp...,Agricultores y trabajadores calificados agrope...,2023-01-01,1,1,3,1,2
1,2,Guatemala,Santa Catarina Pinula,32,Casado(a),Guatemala,Alfabeta,Diversificado,Agricultores y trabajadores calificados de cul...,Urbana,...,30 - 34,Guatemala,Agricultores y trabajadores calificados de exp...,Agricultores y trabajadores calificados agrope...,2023-01-01,1,1,4,1,2
2,3,Guatemala,Santa Catarina Pinula,24,Soltero(a),Guatemala,Alfabeta,Diversificado,Estudiantes,Urbana,...,20 - 24,Guatemala,Estudiantes,Categorias funcionales de la No Pea,2023-01-01,1,1,3,1,2
3,4,Guatemala,Santa Catarina Pinula,24,Soltero(a),Guatemala,Alfabeta,Diversificado,Estudiantes,Urbana,...,20 - 24,Guatemala,Estudiantes,Categorias funcionales de la No Pea,2023-01-01,1,1,4,1,2
4,5,Guatemala,Santa Catarina Pinula,35,Soltero(a),Guatemala,Alfabeta,Primaria,Vendedores no clasificados bajo otros epígrafes,Rural,...,35 - 39,Guatemala,Vendedores,Trabajadores de los servicios y vendedores de ...,2023-01-01,1,1,5,1,2


In [14]:
fJudiciales = fJudiciales.drop(columns=['Depto_boleta'])
fJudiciales['id_conyugal'] = fJudiciales['Est_conyugal_inf'].map({v: k for k, v in estado_conyugal_map.items()})
fJudiciales['id_alfabetismo'] = fJudiciales['Cond_alfabetismo_inf'].map(normalizar_texto).map({v: k for k, v in alfabetismo_map.items()})
fJudiciales['id_escolaridad'] = fJudiciales['Niv_escolaridad_inf'].map(normalizar_texto).map({v: k for k, v in escolaridad_map.items()})
fJudiciales['id_area'] = fJudiciales['Área_geo_inf'].map(normalizar_texto).map({v: k for k, v in area_geografica_map.items()}).astype('Int64')
fJudiciales['id_ebriedad'] = fJudiciales['Est_ebriedad_inf'].map(normalizar_texto).map({v: k for k, v in condicion_infractor_map.items()}).astype('Int64')
fJudiciales.head()

,Núm_corre,Muni_boleta,Edad_inf,Est_conyugal_inf,Nacimiento_inf,Cond_alfabetismo_inf,Niv_escolaridad_inf,G_primarios,Área_geo_inf,Est_ebriedad_inf,...,departamento_boleta,departamento_nacimiento,id_falta,id_sexo,id_etnia,id_conyugal,id_alfabetismo,id_escolaridad,id_area,id_ebriedad
0,1,Santa Catarina Pinula,32,Casado(a),Guatemala,Alfabeta,Diversificado,Agricultores y trabajadores calificados de cul...,Urbana,Ebrio,...,1,1,3,1,2,2,1,4,1,1
1,2,Santa Catarina Pinula,32,Casado(a),Guatemala,Alfabeta,Diversificado,Agricultores y trabajadores calificados de cul...,Urbana,Ebrio,...,1,1,4,1,2,2,1,4,1,1
2,3,Santa Catarina Pinula,24,Soltero(a),Guatemala,Alfabeta,Diversificado,Estudiantes,Urbana,Ebrio,...,1,1,3,1,2,1,1,4,1,1
3,4,Santa Catarina Pinula,24,Soltero(a),Guatemala,Alfabeta,Diversificado,Estudiantes,Urbana,Ebrio,...,1,1,4,1,2,1,1,4,1,1
4,5,Santa Catarina Pinula,35,Soltero(a),Guatemala,Alfabeta,Primaria,Vendedores no clasificados bajo otros epígrafes,Rural,Ebrio,...,1,1,5,1,2,1,1,2,2,1


In [15]:
fJudiciales = fJudiciales.drop(columns=['Est_conyugal_inf','Cond_alfabetismo_inf','Niv_escolaridad_inf','Área_geo_inf','Est_ebriedad_inf'])
fJudiciales.head()

,Núm_corre,Muni_boleta,Edad_inf,Nacimiento_inf,G_primarios,g_edad_80ymás,g_edad_60ymás,edad_quinquenales,Depto_nacimiento_inf,Subg_principales,...,departamento_boleta,departamento_nacimiento,id_falta,id_sexo,id_etnia,id_conyugal,id_alfabetismo,id_escolaridad,id_area,id_ebriedad
0,1,Santa Catarina Pinula,32,Guatemala,Agricultores y trabajadores calificados de cul...,30-34,30-34,30 - 34,Guatemala,Agricultores y trabajadores calificados de exp...,...,1,1,3,1,2,2,1,4,1,1
1,2,Santa Catarina Pinula,32,Guatemala,Agricultores y trabajadores calificados de cul...,30-34,30-34,30 - 34,Guatemala,Agricultores y trabajadores calificados de exp...,...,1,1,4,1,2,2,1,4,1,1
2,3,Santa Catarina Pinula,24,Guatemala,Estudiantes,20-24,20-24,20 - 24,Guatemala,Estudiantes,...,1,1,3,1,2,1,1,4,1,1
3,4,Santa Catarina Pinula,24,Guatemala,Estudiantes,20-24,20-24,20 - 24,Guatemala,Estudiantes,...,1,1,4,1,2,1,1,4,1,1
4,5,Santa Catarina Pinula,35,Guatemala,Vendedores no clasificados bajo otros epígrafes,35-39,35-39,35 - 39,Guatemala,Vendedores,...,1,1,5,1,2,1,1,2,2,1


In [16]:
fJudiciales  = fJudiciales.drop(columns=['Depto_nacimiento_inf'])
fJudiciales.head()

,Núm_corre,Muni_boleta,Edad_inf,Nacimiento_inf,G_primarios,g_edad_80ymás,g_edad_60ymás,edad_quinquenales,Subg_principales,Gran_grupos,...,departamento_boleta,departamento_nacimiento,id_falta,id_sexo,id_etnia,id_conyugal,id_alfabetismo,id_escolaridad,id_area,id_ebriedad
0,1,Santa Catarina Pinula,32,Guatemala,Agricultores y trabajadores calificados de cul...,30-34,30-34,30 - 34,Agricultores y trabajadores calificados de exp...,Agricultores y trabajadores calificados agrope...,...,1,1,3,1,2,2,1,4,1,1
1,2,Santa Catarina Pinula,32,Guatemala,Agricultores y trabajadores calificados de cul...,30-34,30-34,30 - 34,Agricultores y trabajadores calificados de exp...,Agricultores y trabajadores calificados agrope...,...,1,1,4,1,2,2,1,4,1,1
2,3,Santa Catarina Pinula,24,Guatemala,Estudiantes,20-24,20-24,20 - 24,Estudiantes,Categorias funcionales de la No Pea,...,1,1,3,1,2,1,1,4,1,1
3,4,Santa Catarina Pinula,24,Guatemala,Estudiantes,20-24,20-24,20 - 24,Estudiantes,Categorias funcionales de la No Pea,...,1,1,4,1,2,1,1,4,1,1
4,5,Santa Catarina Pinula,35,Guatemala,Vendedores no clasificados bajo otros epígrafes,35-39,35-39,35 - 39,Vendedores,Trabajadores de los servicios y vendedores de ...,...,1,1,5,1,2,1,1,2,2,1


In [17]:
fJudiciales = fJudiciales.drop(columns=['Nacimiento_inf'])
fJudiciales = fJudiciales.drop(columns=['g_edad_80ymás','g_edad_60ymás','edad_quinquenales'])
fJudiciales.head()

,Núm_corre,Muni_boleta,Edad_inf,G_primarios,Subg_principales,Gran_grupos,fecha,departamento_boleta,departamento_nacimiento,id_falta,id_sexo,id_etnia,id_conyugal,id_alfabetismo,id_escolaridad,id_area,id_ebriedad
0,1,Santa Catarina Pinula,32,Agricultores y trabajadores calificados de cul...,Agricultores y trabajadores calificados de exp...,Agricultores y trabajadores calificados agrope...,2023-01-01,1,1,3,1,2,2,1,4,1,1
1,2,Santa Catarina Pinula,32,Agricultores y trabajadores calificados de cul...,Agricultores y trabajadores calificados de exp...,Agricultores y trabajadores calificados agrope...,2023-01-01,1,1,4,1,2,2,1,4,1,1
2,3,Santa Catarina Pinula,24,Estudiantes,Estudiantes,Categorias funcionales de la No Pea,2023-01-01,1,1,3,1,2,1,1,4,1,1
3,4,Santa Catarina Pinula,24,Estudiantes,Estudiantes,Categorias funcionales de la No Pea,2023-01-01,1,1,4,1,2,1,1,4,1,1
4,5,Santa Catarina Pinula,35,Vendedores no clasificados bajo otros epígrafes,Vendedores,Trabajadores de los servicios y vendedores de ...,2023-01-01,1,1,5,1,2,1,1,2,2,1


In [18]:
fJudiciales['municipios'] = fJudiciales['Muni_boleta'].apply(normalizar_texto)
fJudiciales['ocupaciones'] = fJudiciales['G_primarios'].apply(normalizar_texto)
fJudiciales = fJudiciales.drop(columns=['Muni_boleta','G_primarios','Subg_principales','Gran_grupos'])
fJudiciales.head()

,Núm_corre,Edad_inf,fecha,departamento_boleta,departamento_nacimiento,id_falta,id_sexo,id_etnia,id_conyugal,id_alfabetismo,id_escolaridad,id_area,id_ebriedad,municipios,ocupaciones
0,1,32,2023-01-01,1,1,3,1,2,2,1,4,1,1,Santa Catarina Pinula,Agricultores Y Trabajadores Calificados De Cul...
1,2,32,2023-01-01,1,1,4,1,2,2,1,4,1,1,Santa Catarina Pinula,Agricultores Y Trabajadores Calificados De Cul...
2,3,24,2023-01-01,1,1,3,1,2,1,1,4,1,1,Santa Catarina Pinula,Estudiantes
3,4,24,2023-01-01,1,1,4,1,2,1,1,4,1,1,Santa Catarina Pinula,Estudiantes
4,5,35,2023-01-01,1,1,5,1,2,1,1,2,2,1,Santa Catarina Pinula,Vendedores No Clasificados Bajo Otros Epigrafes


In [19]:
urlDiccionarioFaltas = '/content/diccionario.xlsx'
cargaExcelFaltas = pd.read_excel(urlDiccionarioFaltas, sheet_name='ocupaciones', header=0)
diccionarioOcupaciones = pd.DataFrame(cargaExcelFaltas)
cargaExcelMunicipios = pd.read_excel(urlDiccionarioFaltas, sheet_name='municipios', header=0)
diccionarioMunicipios = pd.DataFrame(cargaExcelMunicipios)
cargaExcelPaises = pd.read_excel(urlDiccionarioFaltas, sheet_name='Paises', header=0)
diccionarioPaises = pd.DataFrame(cargaExcelPaises)
diccionarioOcupaciones.head()

,id,nombre
0,1111,Miembros del poder legislativo
1,1112,Personal directivo de la administración pública
2,1113,Jefes de pequeñas poblaciones
3,1114,Dirigentes de organizaciones que presentan un ...
4,1120,Directores generales y gerentes generales


In [20]:
diccionarioOcupaciones['ocupacion'] = diccionarioOcupaciones['nombre'].apply(normalizar_texto)
diccionarioOcupaciones = diccionarioOcupaciones.drop(columns=['nombre'])
diccionarioOcupaciones.head()

,id,ocupacion
0,1111,Miembros Del Poder Legislativo
1,1112,Personal Directivo De La Administracion Publica
2,1113,Jefes De Pequenas Poblaciones
3,1114,Dirigentes De Organizaciones Que Presentan Un ...
4,1120,Directores Generales Y Gerentes Generales


In [21]:
diccionarioMunicipios['municipio'] = diccionarioMunicipios['nombre'].apply(normalizar_texto)
diccionarioMunicipios = diccionarioMunicipios.drop(columns=['nombre'])
diccionarioMunicipios.head()

,id,municipio
0,1,Guatemala
1,2,El Progreso
2,3,Sacatepequez
3,4,Chimaltenango
4,5,Escuintla


In [22]:
diccionarioMunicipios = diccionarioMunicipios.drop_duplicates(subset=['municipio'], keep='first')
diccionarioMunicipios.head()

,id,municipio
0,1,Guatemala
1,2,El Progreso
2,3,Sacatepequez
3,4,Chimaltenango
4,5,Escuintla


In [23]:
df_result = pd.merge(fJudiciales, diccionarioOcupaciones, left_on='ocupaciones', right_on='ocupacion', how='left')
df_result = df_result.drop(columns=['ocupaciones','ocupacion'])
df_result = df_result.rename(columns={'id': 'id_ocupacion'})
df_result.head()

,Núm_corre,Edad_inf,fecha,departamento_boleta,departamento_nacimiento,id_falta,id_sexo,id_etnia,id_conyugal,id_alfabetismo,id_escolaridad,id_area,id_ebriedad,municipios,id_ocupacion
0,1,32,2023-01-01,1,1,3,1,2,2,1,4,1,1,Santa Catarina Pinula,6111.0
1,2,32,2023-01-01,1,1,4,1,2,2,1,4,1,1,Santa Catarina Pinula,6111.0
2,3,24,2023-01-01,1,1,3,1,2,1,1,4,1,1,Santa Catarina Pinula,9712.0
3,4,24,2023-01-01,1,1,4,1,2,1,1,4,1,1,Santa Catarina Pinula,9712.0
4,5,35,2023-01-01,1,1,5,1,2,1,1,2,2,1,Santa Catarina Pinula,5249.0


In [24]:
df_reulst = pd.merge(df_result, diccionarioMunicipios, left_on='municipios', right_on='municipio', how='left')
df_reulst = df_reulst.drop(columns=['municipios','municipio'])
df_reulst = df_reulst.rename(columns={'id': 'id_municipio'})
df_reulst['id_municipio'] = df_reulst['id_municipio'].astype('Int64')  # Usa 'Int64' si hay valores nulos (NA)
df_reulst['id_ocupacion'] = df_reulst['id_ocupacion'].astype('Int64')
df_reulst.head()

,Núm_corre,Edad_inf,fecha,departamento_boleta,departamento_nacimiento,id_falta,id_sexo,id_etnia,id_conyugal,id_alfabetismo,id_escolaridad,id_area,id_ebriedad,id_ocupacion,id_municipio
0,1,32,2023-01-01,1,1,3,1,2,2,1,4,1,1,6111,102
1,2,32,2023-01-01,1,1,4,1,2,2,1,4,1,1,6111,102
2,3,24,2023-01-01,1,1,3,1,2,1,1,4,1,1,9712,102
3,4,24,2023-01-01,1,1,4,1,2,1,1,4,1,1,9712,102
4,5,35,2023-01-01,1,1,5,1,2,1,1,2,2,1,5249,102


In [25]:
df_reulst = df_reulst.rename(columns={'Núm_corre': 'id_falta_judicial','id_municipio':'id_municipio_boleta'})
df_reulst.head()

,id_falta_judicial,Edad_inf,fecha,departamento_boleta,departamento_nacimiento,id_falta,id_sexo,id_etnia,id_conyugal,id_alfabetismo,id_escolaridad,id_area,id_ebriedad,id_ocupacion,id_municipio_boleta
0,1,32,2023-01-01,1,1,3,1,2,2,1,4,1,1,6111,102
1,2,32,2023-01-01,1,1,4,1,2,2,1,4,1,1,6111,102
2,3,24,2023-01-01,1,1,3,1,2,1,1,4,1,1,9712,102
3,4,24,2023-01-01,1,1,4,1,2,1,1,4,1,1,9712,102
4,5,35,2023-01-01,1,1,5,1,2,1,1,2,2,1,5249,102


In [26]:
df_reulst = df_reulst.drop_duplicates(subset=['id_falta_judicial'], keep='first')
if df_reulst['id_falta_judicial'].duplicated().any():
  print("La columna 'id_falta_judicial' tiene valores repetidos.")
else:
  print("La columna 'id_falta_judicial' no tiene valores repetidos.")

La columna 'id_falta_judicial' no tiene valores repetidos.


In [27]:
import numpy as np

# Reemplaza "Ignorada" por NaN
df_reulst['Edad_inf'] = df_reulst['Edad_inf'].replace('Ignorada', np.nan)

# Convierte explícitamente el tipo de dato si es necesario
df_reulst['Edad_inf'] = pd.to_numeric(df_reulst['Edad_inf'], errors='coerce')

# Opcional: eliminar el warning
df_reulst = df_reulst.infer_objects(copy=False)


<ipython-input-27-fa8a74d02d74>:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_reulst['Edad_inf'] = df_reulst['Edad_inf'].replace('Ignorada', np.nan)


In [28]:
#1
df_reulst.to_csv('faltasJudiciales.csv', index=False)
#2
diccionarioOcupaciones.to_csv('diccionarioOcupaciones.csv', index=False)
#2
diccionarioMunicipios.to_csv('diccionarioMunicipios.csv', index=False)
#4
df_tipoFalta = pd.DataFrame(tipo_falta_map.items(), columns=['id', 'tipo_falta'])
df_tipoFalta.to_csv('df_tipoFalta.csv', index=False)
#5
df_sexo = pd.DataFrame(sexo_map.items(), columns=['id', 'sexo'])
df_sexo.to_csv('df_sexo.csv', index=False)
#6
df_etnia = pd.DataFrame(grupo_etnico_map.items(), columns=['id', 'etnia'])
df_etnia.to_csv('df_etnia.csv', index=False)
#7
df_conyugal = pd.DataFrame(estado_conyugal_map.items(), columns=['id', 'conyugal'])
df_conyugal.to_csv('df_conyugal.csv', index=False)
#8
df_escolaridad = pd.DataFrame(escolaridad_map.items(), columns=['id', 'escolaridad'])
df_escolaridad.to_csv('df_escolaridad.csv', index=False)
#9
df_area = pd.DataFrame(area_geografica_map.items(), columns=['id', 'area'])
df_area.to_csv('df_area.csv', index=False)
#10
df_condicion = pd.DataFrame(condicion_infractor_map.items(), columns=['id', 'condicion'])
df_condicion.to_csv('df_condicion.csv', index=False)
#11
df_alfabetismo = pd.DataFrame(alfabetismo_map.items(), columns=['id', 'alfabetismo'])
df_alfabetismo.to_csv('df_alfabetismo.csv', index=False)
#12
df_departamento = pd.DataFrame(departamento_map.items(), columns=['id', 'departamento'])
df_departamento.to_csv('df_departamento.csv', index=False)

In [29]:
urlAgraviado = '/content/agraviados.xlsx'
cargaExcelAgraviado = pd.read_excel(urlAgraviado, sheet_name='Sheet1', header=0)
agraviados = pd.DataFrame(cargaExcelAgraviado)
agraviados.head()

,núm_corre,año_denuncia,Reg_mes,día_denuncia,día_sem_denuncia,año_hecho,mes_hecho,día_hecho,dia_sem_hecho,depto_ocu_hecho,...,sexo_agraviados,g_edad_60ymás,g_edad_80ymás,g_edad_de0_80ymás,est_conyugal,delito_com,principales_delitos,hora_denuncia,g_hora_denuncia,g_hora_denuncia_mañ.tar.noch
0,1,2023,Marzo,29,3,2019,Julio,19,Viernes,Guatemala,...,Ignorado,Ignorado,Ignorado,Ignorado,Ignorado,Falsedad material,Otros,22,18:00 a 23:59,18:00 a 23:59
1,2,2023,Marzo,29,3,2019,Julio,19,Viernes,Guatemala,...,Mujer,Ignorado,Ignorado,Ignorado,Ignorado,Falsedad material,Otros,22,18:00 a 23:59,18:00 a 23:59
2,3,2023,Marzo,29,3,2019,Julio,19,Viernes,Guatemala,...,Ignorado,Ignorado,Ignorado,Ignorado,Ignorado,Falsedad ideológica,Otros,22,18:00 a 23:59,18:00 a 23:59
3,4,2023,Marzo,29,3,2019,Julio,19,Viernes,Guatemala,...,Mujer,Ignorado,Ignorado,Ignorado,Ignorado,Falsedad ideológica,Otros,22,18:00 a 23:59,18:00 a 23:59
4,5,2023,Marzo,29,3,2023,Marzo,29,Miercoles,Guatemala,...,Mujer,30-34,30-34,30 - 34,Casado,Robo,Robo,22,18:00 a 23:59,18:00 a 23:59


In [30]:
agraviados['fechaDenuncia'] = agraviados.apply(
    lambda r: convertir_fecha_dia(r['Reg_mes'], r['año_denuncia'],r['día_denuncia']),
    axis=1
)
agraviados['fechaHecho'] = agraviados.apply(
    lambda r: convertir_fecha_dia(r['mes_hecho'], r['año_hecho'],r['día_hecho']),
    axis=1
)
agraviados.head()

,núm_corre,año_denuncia,Reg_mes,día_denuncia,día_sem_denuncia,año_hecho,mes_hecho,día_hecho,dia_sem_hecho,depto_ocu_hecho,...,g_edad_80ymás,g_edad_de0_80ymás,est_conyugal,delito_com,principales_delitos,hora_denuncia,g_hora_denuncia,g_hora_denuncia_mañ.tar.noch,fechaDenuncia,fechaHecho
0,1,2023,Marzo,29,3,2019,Julio,19,Viernes,Guatemala,...,Ignorado,Ignorado,Ignorado,Falsedad material,Otros,22,18:00 a 23:59,18:00 a 23:59,2023-03-29,2019-07-19
1,2,2023,Marzo,29,3,2019,Julio,19,Viernes,Guatemala,...,Ignorado,Ignorado,Ignorado,Falsedad material,Otros,22,18:00 a 23:59,18:00 a 23:59,2023-03-29,2019-07-19
2,3,2023,Marzo,29,3,2019,Julio,19,Viernes,Guatemala,...,Ignorado,Ignorado,Ignorado,Falsedad ideológica,Otros,22,18:00 a 23:59,18:00 a 23:59,2023-03-29,2019-07-19
3,4,2023,Marzo,29,3,2019,Julio,19,Viernes,Guatemala,...,Ignorado,Ignorado,Ignorado,Falsedad ideológica,Otros,22,18:00 a 23:59,18:00 a 23:59,2023-03-29,2019-07-19
4,5,2023,Marzo,29,3,2023,Marzo,29,Miercoles,Guatemala,...,30-34,30 - 34,Casado,Robo,Robo,22,18:00 a 23:59,18:00 a 23:59,2023-03-29,2023-03-29


In [31]:
agraviados = agraviados.drop(columns=['año_denuncia','Reg_mes','día_denuncia','día_sem_denuncia','año_hecho','mes_hecho','día_hecho','dia_sem_hecho'])
agraviados.head()

,núm_corre,depto_ocu_hecho,mupio_ocu_hecho,zona_ocu_hecho,edad_agrav,sexo_agraviados,g_edad_60ymás,g_edad_80ymás,g_edad_de0_80ymás,est_conyugal,delito_com,principales_delitos,hora_denuncia,g_hora_denuncia,g_hora_denuncia_mañ.tar.noch,fechaDenuncia,fechaHecho
0,1,Guatemala,Guatemala,Ignorada,Ignorada,Ignorado,Ignorado,Ignorado,Ignorado,Ignorado,Falsedad material,Otros,22,18:00 a 23:59,18:00 a 23:59,2023-03-29,2019-07-19
1,2,Guatemala,Guatemala,Ignorada,Ignorada,Mujer,Ignorado,Ignorado,Ignorado,Ignorado,Falsedad material,Otros,22,18:00 a 23:59,18:00 a 23:59,2023-03-29,2019-07-19
2,3,Guatemala,Guatemala,Ignorada,Ignorada,Ignorado,Ignorado,Ignorado,Ignorado,Ignorado,Falsedad ideológica,Otros,22,18:00 a 23:59,18:00 a 23:59,2023-03-29,2019-07-19
3,4,Guatemala,Guatemala,Ignorada,Ignorada,Mujer,Ignorado,Ignorado,Ignorado,Ignorado,Falsedad ideológica,Otros,22,18:00 a 23:59,18:00 a 23:59,2023-03-29,2019-07-19
4,5,Guatemala,Guatemala,6,30,Mujer,30-34,30-34,30 - 34,Casado,Robo,Robo,22,18:00 a 23:59,18:00 a 23:59,2023-03-29,2023-03-29


In [32]:
agraviados = agraviados.rename(columns={'depto_ocu_hecho': 'departamento','mupio_ocu_hecho':'municipio','zona_ocu_hecho':'zona'})
agraviados = agraviados.drop(columns=['g_edad_60ymás','g_edad_80ymás','g_edad_de0_80ymás'])
agraviados.head()

,núm_corre,departamento,municipio,zona,edad_agrav,sexo_agraviados,est_conyugal,delito_com,principales_delitos,hora_denuncia,g_hora_denuncia,g_hora_denuncia_mañ.tar.noch,fechaDenuncia,fechaHecho
0,1,Guatemala,Guatemala,Ignorada,Ignorada,Ignorado,Ignorado,Falsedad material,Otros,22,18:00 a 23:59,18:00 a 23:59,2023-03-29,2019-07-19
1,2,Guatemala,Guatemala,Ignorada,Ignorada,Mujer,Ignorado,Falsedad material,Otros,22,18:00 a 23:59,18:00 a 23:59,2023-03-29,2019-07-19
2,3,Guatemala,Guatemala,Ignorada,Ignorada,Ignorado,Ignorado,Falsedad ideológica,Otros,22,18:00 a 23:59,18:00 a 23:59,2023-03-29,2019-07-19
3,4,Guatemala,Guatemala,Ignorada,Ignorada,Mujer,Ignorado,Falsedad ideológica,Otros,22,18:00 a 23:59,18:00 a 23:59,2023-03-29,2019-07-19
4,5,Guatemala,Guatemala,6,30,Mujer,Casado,Robo,Robo,22,18:00 a 23:59,18:00 a 23:59,2023-03-29,2023-03-29


In [33]:
agraviados['id_sexo'] = agraviados['sexo_agraviados'].map({v: k for k, v in sexo_map.items()  })
agraviados.head()

,núm_corre,departamento,municipio,zona,edad_agrav,sexo_agraviados,est_conyugal,delito_com,principales_delitos,hora_denuncia,g_hora_denuncia,g_hora_denuncia_mañ.tar.noch,fechaDenuncia,fechaHecho,id_sexo
0,1,Guatemala,Guatemala,Ignorada,Ignorada,Ignorado,Ignorado,Falsedad material,Otros,22,18:00 a 23:59,18:00 a 23:59,2023-03-29,2019-07-19,3
1,2,Guatemala,Guatemala,Ignorada,Ignorada,Mujer,Ignorado,Falsedad material,Otros,22,18:00 a 23:59,18:00 a 23:59,2023-03-29,2019-07-19,2
2,3,Guatemala,Guatemala,Ignorada,Ignorada,Ignorado,Ignorado,Falsedad ideológica,Otros,22,18:00 a 23:59,18:00 a 23:59,2023-03-29,2019-07-19,3
3,4,Guatemala,Guatemala,Ignorada,Ignorada,Mujer,Ignorado,Falsedad ideológica,Otros,22,18:00 a 23:59,18:00 a 23:59,2023-03-29,2019-07-19,2
4,5,Guatemala,Guatemala,6,30,Mujer,Casado,Robo,Robo,22,18:00 a 23:59,18:00 a 23:59,2023-03-29,2023-03-29,2


In [34]:
agraviados['departamentos'] = agraviados['departamento'].apply(normalizar_texto)
agraviados['municipios'] = agraviados['municipio'].apply(normalizar_texto)
agraviados = agraviados.drop(columns=['departamento','municipio'])
agraviados.head()

,núm_corre,zona,edad_agrav,sexo_agraviados,est_conyugal,delito_com,principales_delitos,hora_denuncia,g_hora_denuncia,g_hora_denuncia_mañ.tar.noch,fechaDenuncia,fechaHecho,id_sexo,departamentos,municipios
0,1,Ignorada,Ignorada,Ignorado,Ignorado,Falsedad material,Otros,22,18:00 a 23:59,18:00 a 23:59,2023-03-29,2019-07-19,3,Guatemala,Guatemala
1,2,Ignorada,Ignorada,Mujer,Ignorado,Falsedad material,Otros,22,18:00 a 23:59,18:00 a 23:59,2023-03-29,2019-07-19,2,Guatemala,Guatemala
2,3,Ignorada,Ignorada,Ignorado,Ignorado,Falsedad ideológica,Otros,22,18:00 a 23:59,18:00 a 23:59,2023-03-29,2019-07-19,3,Guatemala,Guatemala
3,4,Ignorada,Ignorada,Mujer,Ignorado,Falsedad ideológica,Otros,22,18:00 a 23:59,18:00 a 23:59,2023-03-29,2019-07-19,2,Guatemala,Guatemala
4,5,6,30,Mujer,Casado,Robo,Robo,22,18:00 a 23:59,18:00 a 23:59,2023-03-29,2023-03-29,2,Guatemala,Guatemala


In [35]:
agraviados['zona'] = agraviados['zona'].replace('Ignorada', np.nan)
agraviados['edad_agrav'] = agraviados['edad_agrav'].replace('Ignorada', np.nan)
agraviados = agraviados.drop(columns=['sexo_agraviados','g_hora_denuncia','g_hora_denuncia_mañ.tar.noch'])
agraviados.head()

<ipython-input-35-92ac9a2e5db2>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  agraviados['zona'] = agraviados['zona'].replace('Ignorada', np.nan)
<ipython-input-35-92ac9a2e5db2>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  agraviados['edad_agrav'] = agraviados['edad_agrav'].replace('Ignorada', np.nan)


,núm_corre,zona,edad_agrav,est_conyugal,delito_com,principales_delitos,hora_denuncia,fechaDenuncia,fechaHecho,id_sexo,departamentos,municipios
0,1,NaN,NaN,Ignorado,Falsedad material,Otros,22,2023-03-29,2019-07-19,3,Guatemala,Guatemala
1,2,NaN,NaN,Ignorado,Falsedad material,Otros,22,2023-03-29,2019-07-19,2,Guatemala,Guatemala
2,3,NaN,NaN,Ignorado,Falsedad ideológica,Otros,22,2023-03-29,2019-07-19,3,Guatemala,Guatemala
3,4,NaN,NaN,Ignorado,Falsedad ideológica,Otros,22,2023-03-29,2019-07-19,2,Guatemala,Guatemala
4,5,6.0,30.0,Casado,Robo,Robo,22,2023-03-29,2023-03-29,2,Guatemala,Guatemala


In [36]:
#Normalizar estado conyugal
def nomralizarEsado(estado):
  estadoC = normalizar_texto(estado)
  if estadoC in ['Soltero','soltera','Soltera','soltero']:
    return 'Soltero(a)'
  if estadoC in ['Casado','casada','Casada','casado']:
    return 'Casado(a)'
  if estadoC in ['Divorciado','divorciada','Divorciada','divorciado']:
    return 'Divorciado(a)'
  if estadoC in ['Viudo','viuda','Viuda','viudo']:
    return 'Viudo(a)'
  if estadoC in ['Unido','unida','Unida','unido']:
    return 'Unido(a)'
  return 'Ignorado'


In [37]:
agraviados ['estado_conyugal'] = agraviados['est_conyugal'].apply(nomralizarEsado)
agraviados = agraviados.drop(columns=['est_conyugal'])


In [38]:
agraviados ['id_conyugal'] = agraviados['estado_conyugal'].map({v: k for k, v in estado_conyugal_map.items()})
agraviados.head()

,núm_corre,zona,edad_agrav,delito_com,principales_delitos,hora_denuncia,fechaDenuncia,fechaHecho,id_sexo,departamentos,municipios,estado_conyugal,id_conyugal
0,1,NaN,NaN,Falsedad material,Otros,22,2023-03-29,2019-07-19,3,Guatemala,Guatemala,Ignorado,6
1,2,NaN,NaN,Falsedad material,Otros,22,2023-03-29,2019-07-19,2,Guatemala,Guatemala,Ignorado,6
2,3,NaN,NaN,Falsedad ideológica,Otros,22,2023-03-29,2019-07-19,3,Guatemala,Guatemala,Ignorado,6
3,4,NaN,NaN,Falsedad ideológica,Otros,22,2023-03-29,2019-07-19,2,Guatemala,Guatemala,Ignorado,6
4,5,6.0,30.0,Robo,Robo,22,2023-03-29,2023-03-29,2,Guatemala,Guatemala,Casado(a),2


In [39]:
agraviados = agraviados.drop(columns=['estado_conyugal'])
agraviados = pd.merge(agraviados, diccionarioMunicipios, left_on='municipios', right_on='municipio', how='left')
agraviados.head()

,núm_corre,zona,edad_agrav,delito_com,principales_delitos,hora_denuncia,fechaDenuncia,fechaHecho,id_sexo,departamentos,municipios,id_conyugal,id,municipio
0,1,NaN,NaN,Falsedad material,Otros,22,2023-03-29,2019-07-19,3,Guatemala,Guatemala,6,1.0,Guatemala
1,2,NaN,NaN,Falsedad material,Otros,22,2023-03-29,2019-07-19,2,Guatemala,Guatemala,6,1.0,Guatemala
2,3,NaN,NaN,Falsedad ideológica,Otros,22,2023-03-29,2019-07-19,3,Guatemala,Guatemala,6,1.0,Guatemala
3,4,NaN,NaN,Falsedad ideológica,Otros,22,2023-03-29,2019-07-19,2,Guatemala,Guatemala,6,1.0,Guatemala
4,5,6.0,30.0,Robo,Robo,22,2023-03-29,2023-03-29,2,Guatemala,Guatemala,2,1.0,Guatemala


In [40]:
agraviados = agraviados.drop(columns=['municipio','id'])
agraviados.head()

,núm_corre,zona,edad_agrav,delito_com,principales_delitos,hora_denuncia,fechaDenuncia,fechaHecho,id_sexo,departamentos,municipios,id_conyugal
0,1,NaN,NaN,Falsedad material,Otros,22,2023-03-29,2019-07-19,3,Guatemala,Guatemala,6
1,2,NaN,NaN,Falsedad material,Otros,22,2023-03-29,2019-07-19,2,Guatemala,Guatemala,6
2,3,NaN,NaN,Falsedad ideológica,Otros,22,2023-03-29,2019-07-19,3,Guatemala,Guatemala,6
3,4,NaN,NaN,Falsedad ideológica,Otros,22,2023-03-29,2019-07-19,2,Guatemala,Guatemala,6
4,5,6.0,30.0,Robo,Robo,22,2023-03-29,2023-03-29,2,Guatemala,Guatemala,2


In [41]:
agraviados = pd.merge(agraviados, diccionarioMunicipios, left_on='municipios', right_on='municipio', how='left')
#agraviados = agraviados.drop(columns=['municipio','municipios'])
agraviados = agraviados.rename(columns={'id': 'id_municipio'})
agraviados.head()

,núm_corre,zona,edad_agrav,delito_com,principales_delitos,hora_denuncia,fechaDenuncia,fechaHecho,id_sexo,departamentos,municipios,id_conyugal,id_municipio,municipio
0,1,NaN,NaN,Falsedad material,Otros,22,2023-03-29,2019-07-19,3,Guatemala,Guatemala,6,1.0,Guatemala
1,2,NaN,NaN,Falsedad material,Otros,22,2023-03-29,2019-07-19,2,Guatemala,Guatemala,6,1.0,Guatemala
2,3,NaN,NaN,Falsedad ideológica,Otros,22,2023-03-29,2019-07-19,3,Guatemala,Guatemala,6,1.0,Guatemala
3,4,NaN,NaN,Falsedad ideológica,Otros,22,2023-03-29,2019-07-19,2,Guatemala,Guatemala,6,1.0,Guatemala
4,5,6.0,30.0,Robo,Robo,22,2023-03-29,2023-03-29,2,Guatemala,Guatemala,2,1.0,Guatemala


In [42]:
agraviados = agraviados.drop(columns=['municipios','municipio'])
agraviados.head()

,núm_corre,zona,edad_agrav,delito_com,principales_delitos,hora_denuncia,fechaDenuncia,fechaHecho,id_sexo,departamentos,id_conyugal,id_municipio
0,1,NaN,NaN,Falsedad material,Otros,22,2023-03-29,2019-07-19,3,Guatemala,6,1.0
1,2,NaN,NaN,Falsedad material,Otros,22,2023-03-29,2019-07-19,2,Guatemala,6,1.0
2,3,NaN,NaN,Falsedad ideológica,Otros,22,2023-03-29,2019-07-19,3,Guatemala,6,1.0
3,4,NaN,NaN,Falsedad ideológica,Otros,22,2023-03-29,2019-07-19,2,Guatemala,6,1.0
4,5,6.0,30.0,Robo,Robo,22,2023-03-29,2023-03-29,2,Guatemala,2,1.0


In [43]:
agraviados = agraviados.drop_duplicates(subset=['núm_corre'], keep='first')
#agraviados = agraviados.drop(columns=['municipios'])
agraviados['id_departamento'] = agraviados['departamentos'].map(departamento_map)
agraviados.head()

,núm_corre,zona,edad_agrav,delito_com,principales_delitos,hora_denuncia,fechaDenuncia,fechaHecho,id_sexo,departamentos,id_conyugal,id_municipio,id_departamento
0,1,NaN,NaN,Falsedad material,Otros,22,2023-03-29,2019-07-19,3,Guatemala,6,1.0,1
1,2,NaN,NaN,Falsedad material,Otros,22,2023-03-29,2019-07-19,2,Guatemala,6,1.0,1
2,3,NaN,NaN,Falsedad ideológica,Otros,22,2023-03-29,2019-07-19,3,Guatemala,6,1.0,1
3,4,NaN,NaN,Falsedad ideológica,Otros,22,2023-03-29,2019-07-19,2,Guatemala,6,1.0,1
4,5,6.0,30.0,Robo,Robo,22,2023-03-29,2023-03-29,2,Guatemala,2,1.0,1


In [44]:
agraviados = agraviados.drop(columns=['departamentos'])
agraviados.head()

,núm_corre,zona,edad_agrav,delito_com,principales_delitos,hora_denuncia,fechaDenuncia,fechaHecho,id_sexo,id_conyugal,id_municipio,id_departamento
0,1,NaN,NaN,Falsedad material,Otros,22,2023-03-29,2019-07-19,3,6,1.0,1
1,2,NaN,NaN,Falsedad material,Otros,22,2023-03-29,2019-07-19,2,6,1.0,1
2,3,NaN,NaN,Falsedad ideológica,Otros,22,2023-03-29,2019-07-19,3,6,1.0,1
3,4,NaN,NaN,Falsedad ideológica,Otros,22,2023-03-29,2019-07-19,2,6,1.0,1
4,5,6.0,30.0,Robo,Robo,22,2023-03-29,2023-03-29,2,2,1.0,1


In [45]:
cargaExcelAgraviado = pd.read_excel(urlAgraviado, sheet_name='Sheet2', header=0)
delito = pd.DataFrame(cargaExcelAgraviado)
delito.head()

,id,nombre
0,1,Abandono por estado afectivo
1,2,"Abandono colectivo de funciones, cargos o empleos"
2,3,Abandono de cargo
3,4,Abandono de niños y de personas desvalidas
4,5,Aborto preterintencional


In [46]:
delito['delito'] = delito['nombre'].apply(normalizar_texto)
delito = delito.drop(columns=['nombre'])
agraviados['delito'] = agraviados['delito_com'].apply(normalizar_texto)
#agraviados = agraviados.drop(columns=['delito_com'])
delito.head()

,id,delito
0,1,Abandono Por Estado Afectivo
1,2,"Abandono Colectivo De Funciones, Cargos O Empleos"
2,3,Abandono De Cargo
3,4,Abandono De Ninos Y De Personas Desvalidas
4,5,Aborto Preterintencional


In [47]:
agraviados = agraviados.drop(columns=['delito_com'])
agraviados.head()

,núm_corre,zona,edad_agrav,principales_delitos,hora_denuncia,fechaDenuncia,fechaHecho,id_sexo,id_conyugal,id_municipio,id_departamento,delito
0,1,NaN,NaN,Otros,22,2023-03-29,2019-07-19,3,6,1.0,1,Falsedad Material
1,2,NaN,NaN,Otros,22,2023-03-29,2019-07-19,2,6,1.0,1,Falsedad Material
2,3,NaN,NaN,Otros,22,2023-03-29,2019-07-19,3,6,1.0,1,Falsedad Ideologica
3,4,NaN,NaN,Otros,22,2023-03-29,2019-07-19,2,6,1.0,1,Falsedad Ideologica
4,5,6.0,30.0,Robo,22,2023-03-29,2023-03-29,2,2,1.0,1,Robo


In [48]:
agraviados ['delito_principal'] = agraviados['principales_delitos'].apply(normalizar_texto)
agraviados.head()

,núm_corre,zona,edad_agrav,principales_delitos,hora_denuncia,fechaDenuncia,fechaHecho,id_sexo,id_conyugal,id_municipio,id_departamento,delito,delito_principal
0,1,NaN,NaN,Otros,22,2023-03-29,2019-07-19,3,6,1.0,1,Falsedad Material,Otros
1,2,NaN,NaN,Otros,22,2023-03-29,2019-07-19,2,6,1.0,1,Falsedad Material,Otros
2,3,NaN,NaN,Otros,22,2023-03-29,2019-07-19,3,6,1.0,1,Falsedad Ideologica,Otros
3,4,NaN,NaN,Otros,22,2023-03-29,2019-07-19,2,6,1.0,1,Falsedad Ideologica,Otros
4,5,6.0,30.0,Robo,22,2023-03-29,2023-03-29,2,2,1.0,1,Robo,Robo


In [49]:
agraviados = agraviados.drop(columns=['principales_delitos'])
agraviados  =  pd.merge(agraviados, delito, left_on='delito', right_on='delito', how='left')
agraviados.head()

,núm_corre,zona,edad_agrav,hora_denuncia,fechaDenuncia,fechaHecho,id_sexo,id_conyugal,id_municipio,id_departamento,delito,delito_principal,id
0,1,NaN,NaN,22,2023-03-29,2019-07-19,3,6,1.0,1,Falsedad Material,Otros,167.0
1,2,NaN,NaN,22,2023-03-29,2019-07-19,2,6,1.0,1,Falsedad Material,Otros,167.0
2,3,NaN,NaN,22,2023-03-29,2019-07-19,3,6,1.0,1,Falsedad Ideologica,Otros,166.0
3,4,NaN,NaN,22,2023-03-29,2019-07-19,2,6,1.0,1,Falsedad Ideologica,Otros,166.0
4,5,6.0,30.0,22,2023-03-29,2023-03-29,2,2,1.0,1,Robo,Robo,311.0


In [50]:
agraviados = agraviados.drop(columns=['delito'])
agraviados = agraviados.rename(columns={'id': 'id_delito'})
agraviados = pd.merge(agraviados, delito, left_on='delito_principal', right_on='delito', how='left')
agraviados.head()

,núm_corre,zona,edad_agrav,hora_denuncia,fechaDenuncia,fechaHecho,id_sexo,id_conyugal,id_municipio,id_departamento,delito_principal,id_delito,id,delito
0,1,NaN,NaN,22,2023-03-29,2019-07-19,3,6,1.0,1,Otros,167.0,901,Otros
1,2,NaN,NaN,22,2023-03-29,2019-07-19,2,6,1.0,1,Otros,167.0,901,Otros
2,3,NaN,NaN,22,2023-03-29,2019-07-19,3,6,1.0,1,Otros,166.0,901,Otros
3,4,NaN,NaN,22,2023-03-29,2019-07-19,2,6,1.0,1,Otros,166.0,901,Otros
4,5,6.0,30.0,22,2023-03-29,2023-03-29,2,2,1.0,1,Robo,311.0,311,Robo


In [51]:
agraviados = agraviados.drop(columns=['delito_principal','delito'])
agraviados = agraviados.rename(columns={'id': 'id_delito_principal'})
agraviados.head()

,núm_corre,zona,edad_agrav,hora_denuncia,fechaDenuncia,fechaHecho,id_sexo,id_conyugal,id_municipio,id_departamento,id_delito,id_delito_principal
0,1,NaN,NaN,22,2023-03-29,2019-07-19,3,6,1.0,1,167.0,901
1,2,NaN,NaN,22,2023-03-29,2019-07-19,2,6,1.0,1,167.0,901
2,3,NaN,NaN,22,2023-03-29,2019-07-19,3,6,1.0,1,166.0,901
3,4,NaN,NaN,22,2023-03-29,2019-07-19,2,6,1.0,1,166.0,901
4,5,6.0,30.0,22,2023-03-29,2023-03-29,2,2,1.0,1,311.0,311


In [52]:
# Remove rows with NaN values in 'id_delito_principal'
agraviados = agraviados.dropna(subset=['id_delito'])

# Reset the index if needed
agraviados = agraviados.reset_index(drop=True)
# Check if 'id_delito' has any NaN values
nan_count = agraviados['id_delito'].isna().sum()

# Print the result
if nan_count > 0:
  print(f"The 'id_delito' column has {nan_count} NaN values.")
else:
  print("The 'id_delito' column does not have any NaN values.")

The 'id_delito' column does not have any NaN values.


In [53]:
agraviados.to_csv('agraviados.csv', index=False)
delito.to_csv('delito.csv', index=False)

In [54]:
urlEvaMedInacif = '/content/EvaluacionMedicaINACIF.xlsx'
cargaExcelEvaMedInacif = pd.read_excel(urlEvaMedInacif, sheet_name='Sheet1', header=0)
evaMedInacif = pd.DataFrame(cargaExcelEvaMedInacif)
evaMedInacif.head()


,núm_corre,año_ocu,mes_ocu,día_ocu,dia_sem_ocu,depto_ocu,edad_per,g_edad_60ymás,g_edad_80ymás,edad_quinquenales,menor_mayor,sexo_per,clasif_eval
0,1,2023,Enero,23,Lunes,Guatemala,34,30-34,30-34,30 - 34,Mayor de edad,Mujer,Otros
1,2,2023,Enero,23,Lunes,Guatemala,41,40-44,40-44,40 - 44,Mayor de edad,Hombre,Lesiones
2,3,2023,Enero,23,Lunes,Guatemala,Ignorada,Ignorado,Ignorado,Ignorado,Ignorado,Mujer,Lesiones
3,4,2023,Enero,24,Martes,Guatemala,Ignorada,Ignorado,Ignorado,Ignorado,Ignorado,Hombre,Lesiones
4,5,2023,Enero,25,Miércoles,Guatemala,Ignorada,Ignorado,Ignorado,Ignorado,Ignorado,Hombre,Lesiones


In [55]:
evaMedInacif['fecha'] = evaMedInacif.apply(
    lambda r: convertir_fecha_dia(r['mes_ocu'], r['año_ocu'],r['día_ocu']),
    axis=1
)
evaMedInacif = evaMedInacif.drop(columns=['año_ocu','mes_ocu','día_ocu','dia_sem_ocu'])
evaMedInacif.head()

,núm_corre,depto_ocu,edad_per,g_edad_60ymás,g_edad_80ymás,edad_quinquenales,menor_mayor,sexo_per,clasif_eval,fecha
0,1,Guatemala,34,30-34,30-34,30 - 34,Mayor de edad,Mujer,Otros,2023-01-23
1,2,Guatemala,41,40-44,40-44,40 - 44,Mayor de edad,Hombre,Lesiones,2023-01-23
2,3,Guatemala,Ignorada,Ignorado,Ignorado,Ignorado,Ignorado,Mujer,Lesiones,2023-01-23
3,4,Guatemala,Ignorada,Ignorado,Ignorado,Ignorado,Ignorado,Hombre,Lesiones,2023-01-24
4,5,Guatemala,Ignorada,Ignorado,Ignorado,Ignorado,Ignorado,Hombre,Lesiones,2023-01-25


In [56]:
evaMedInacif ['depto_ocu'] = evaMedInacif['depto_ocu'].apply(normalizar_texto)
evaMedInacif['id_departamento'] = evaMedInacif['depto_ocu'].map(departamento_map)
evaMedInacif.head()

,núm_corre,depto_ocu,edad_per,g_edad_60ymás,g_edad_80ymás,edad_quinquenales,menor_mayor,sexo_per,clasif_eval,fecha,id_departamento
0,1,Guatemala,34,30-34,30-34,30 - 34,Mayor de edad,Mujer,Otros,2023-01-23,1
1,2,Guatemala,41,40-44,40-44,40 - 44,Mayor de edad,Hombre,Lesiones,2023-01-23,1
2,3,Guatemala,Ignorada,Ignorado,Ignorado,Ignorado,Ignorado,Mujer,Lesiones,2023-01-23,1
3,4,Guatemala,Ignorada,Ignorado,Ignorado,Ignorado,Ignorado,Hombre,Lesiones,2023-01-24,1
4,5,Guatemala,Ignorada,Ignorado,Ignorado,Ignorado,Ignorado,Hombre,Lesiones,2023-01-25,1


In [57]:
evaMedInacif = evaMedInacif.drop(columns=['depto_ocu','g_edad_60ymás','g_edad_80ymás','edad_quinquenales','menor_mayor'])
evaMedInacif.head()

,núm_corre,edad_per,sexo_per,clasif_eval,fecha,id_departamento
0,1,34,Mujer,Otros,2023-01-23,1
1,2,41,Hombre,Lesiones,2023-01-23,1
2,3,Ignorada,Mujer,Lesiones,2023-01-23,1
3,4,Ignorada,Hombre,Lesiones,2023-01-24,1
4,5,Ignorada,Hombre,Lesiones,2023-01-25,1


In [58]:
evaMedInacif['id_sexo'] = evaMedInacif['sexo_per'].map({v: k for k, v in sexo_map.items()  })
evaMedInacif['edad_per'] = evaMedInacif['edad_per'].apply(normalizar_edad).astype('Int64')
evaMedInacif = evaMedInacif.drop(columns=['sexo_per'])
evaMedInacif.head(15)

,núm_corre,edad_per,clasif_eval,fecha,id_departamento,id_sexo
0,1,34,Otros,2023-01-23,1,2
1,2,41,Lesiones,2023-01-23,1,1
2,3,<NA>,Lesiones,2023-01-23,1,2
3,4,<NA>,Lesiones,2023-01-24,1,1
4,5,<NA>,Lesiones,2023-01-25,1,1
5,6,<NA>,Lesiones,2023-01-25,1,1
6,7,27,Lesiones,2023-01-25,1,1
7,8,23,Lesiones,2023-01-25,1,2
8,9,37,Lesiones,2023-01-25,1,2
9,10,26,Lesiones,2023-01-25,1,1


In [59]:
evaluacion = pd.read_excel(urlEvaMedInacif,sheet_name='Sheet2', header=0)
evaluacion ['nombre'] = evaluacion['nombre'].apply(normalizar_texto)
evaMedInacif['clasif_eval'] = evaMedInacif['clasif_eval'].apply(normalizar_texto)

evaMedInacif = pd.merge(evaMedInacif, evaluacion, left_on='clasif_eval', right_on='nombre', how='left')
evaMedInacif.head()


,núm_corre,edad_per,clasif_eval,fecha,id_departamento,id_sexo,id,nombre
0,1,34,Otros,2023-01-23,1,2,32,Otros
1,2,41,Lesiones,2023-01-23,1,1,21,Lesiones
2,3,<NA>,Lesiones,2023-01-23,1,2,21,Lesiones
3,4,<NA>,Lesiones,2023-01-24,1,1,21,Lesiones
4,5,<NA>,Lesiones,2023-01-25,1,1,21,Lesiones


In [60]:
evaMedInacif = evaMedInacif.drop(columns=['clasif_eval','nombre'])
evaMedInacif = evaMedInacif.rename(columns={'id': 'id_clasif_eval','núm_corre':'id'})
evaMedInacif

,id,edad_per,fecha,id_departamento,id_sexo,id_clasif_eval
0,1,34,2023-01-23,1,2,32
1,2,41,2023-01-23,1,1,21
2,3,<NA>,2023-01-23,1,2,21
3,4,<NA>,2023-01-24,1,1,21
4,5,<NA>,2023-01-25,1,1,21
...,...,...,...,...,...,...
148532,148533,16,2023-12-31,1,2,14
148533,148534,12,2023-12-31,1,1,14
148534,148535,4,2023-12-31,1,1,14
148535,148536,19,2023-12-31,20,2,14


In [61]:
evaMedInacif = evaMedInacif.dropna(subset=['id_clasif_eval'])
evaMedInacif.to_csv('evaMedInacif.csv', index=False)
evaluacion.to_csv('evaluacion.csv', index=False)

In [64]:
urlExum = '/content/Exhumaciones.xlsx'
cargaExcelExum = pd.read_excel(urlExum, sheet_name='Sheet1', header=0)
exum = pd.DataFrame(cargaExcelExum)
exum.head()

,núm_corre,año_ocu,mes_ocu,día_ocu,dia_sem_ocu,depto_ocu
0,1,2023,Enero,26,Jueves,Huehuetenango
1,2,2023,Enero,26,Jueves,Huehuetenango
2,3,2023,Enero,26,Jueves,Alta Verapaz
3,4,2023,Enero,17,Martes,Huehuetenango
4,5,2023,Enero,10,Martes,Alta Verapaz


In [65]:
exum ['fecha'] = exum.apply(
    lambda r: convertir_fecha_dia(r['mes_ocu'], r['año_ocu'],r['día_ocu']),
    axis=1
)
exum['depto_ocu'] = exum['depto_ocu'].apply(normalizar_texto)
exum['id_departamento'] = exum['depto_ocu'].map(departamento_map)
exum = exum.drop(columns=['año_ocu','mes_ocu','día_ocu','dia_sem_ocu','depto_ocu'])
exum.head()

,núm_corre,fecha,id_departamento
0,1,2023-01-26,16
1,2,2023-01-26,16
2,3,2023-01-26,3
3,4,2023-01-17,16
4,5,2023-01-10,3


In [66]:
exum = exum.rename(columns={'núm_corre':'id'})
exum.to_csv('exhumaciones.csv', index=False)
exum.head()

,id,fecha,id_departamento
0,1,2023-01-26,16
1,2,2023-01-26,16
2,3,2023-01-26,3
3,4,2023-01-17,16
4,5,2023-01-10,3


In [67]:
urlNectropsis = '/content/necropsis.xlsx'
cargaExcelNectropsis = pd.read_excel(urlNectropsis, sheet_name='Sheet1', header=0)
necropsis = pd.DataFrame(cargaExcelNectropsis)
necropsis.head()


,núm_corre,año_ing,mes_ing,día_ing,día_sem_ing,depto_ocu,mupio_ocu,edad_per,g_edad_60ymás,g_edad_80ymás,edad_quinquenales,menor_mayor,sexo_per,causa_muerte
0,1,2023,Enero,1,Domingo,Sacatepéquez,Santa Lucía Milpas Altas,27,25-29,25-29,25 - 29,Mayor de edad,Hombre,Traumatismo
1,2,2023,Enero,1,Domingo,Sacatepéquez,Ciudad Vieja,30,30-34,30-34,30 - 34,Mayor de edad,Hombre,Traumatismo
2,3,2023,Enero,1,Domingo,Sacatepéquez,San Lucas Sacatepéquez,34,30-34,30-34,30 - 34,Mayor de edad,Hombre,Asfixia
3,4,2023,Enero,2,Lunes,Sacatepéquez,San Lucas Sacatepéquez,33,30-34,30-34,30 - 34,Mayor de edad,Mujer,Asfixia
4,5,2023,Enero,2,Lunes,Sacatepéquez,San Lucas Sacatepéquez,0,Menor de 15,Menor de 15,0 - 4,Menor de edad,Hombre,Indeterminada


In [68]:
necropsis['fecha'] = necropsis.apply(
    lambda r: convertir_fecha_dia(r['mes_ing'], r['año_ing'],r['día_ing']),
    axis=1)
necropsis['depto_ocu'] = necropsis['depto_ocu'].apply(normalizar_texto)
necropsis['id_departamento'] = necropsis['depto_ocu'].map(departamento_map)
necropsis['mupio_ocu'] = necropsis['mupio_ocu'].apply(normalizar_texto)
necropsis['edad_per'] = necropsis['edad_per'].apply(normalizar_edad).astype('Int64')
necropsis['id_sexo'] = necropsis['sexo_per'].map({v: k for k, v in sexo_map.items()})
necropsis['causa_muerte'] = necropsis['causa_muerte'].apply(normalizar_texto)
necropsis = necropsis.drop(columns=['año_ing','mes_ing','día_ing','día_sem_ing','depto_ocu','g_edad_60ymás','g_edad_80ymás','edad_quinquenales','menor_mayor'])
necropsis.head()


,núm_corre,mupio_ocu,edad_per,sexo_per,causa_muerte,fecha,id_departamento,id_sexo
0,1,Santa Lucia Milpas Altas,27,Hombre,Traumatismo,2023-01-01,14,1
1,2,Ciudad Vieja,30,Hombre,Traumatismo,2023-01-01,14,1
2,3,San Lucas Sacatepequez,34,Hombre,Asfixia,2023-01-01,14,1
3,4,San Lucas Sacatepequez,33,Mujer,Asfixia,2023-01-02,14,2
4,5,San Lucas Sacatepequez,0,Hombre,Indeterminada,2023-01-02,14,1


In [69]:
necropsis = necropsis.drop(columns=['sexo_per'])
necropsis = pd.merge(necropsis, diccionarioMunicipios, left_on='mupio_ocu', right_on='municipio', how='left')
necropsis.head()


,núm_corre,mupio_ocu,edad_per,causa_muerte,fecha,id_departamento,id_sexo,id,municipio
0,1,Santa Lucia Milpas Altas,27,Traumatismo,2023-01-01,14,1,309.0,Santa Lucia Milpas Altas
1,2,Ciudad Vieja,30,Traumatismo,2023-01-01,14,1,312.0,Ciudad Vieja
2,3,San Lucas Sacatepequez,34,Asfixia,2023-01-01,14,1,308.0,San Lucas Sacatepequez
3,4,San Lucas Sacatepequez,33,Asfixia,2023-01-02,14,2,308.0,San Lucas Sacatepequez
4,5,San Lucas Sacatepequez,0,Indeterminada,2023-01-02,14,1,308.0,San Lucas Sacatepequez


In [70]:
necropsis = necropsis.drop(columns=['mupio_ocu','municipio'])
necropsis = necropsis.rename(columns={'id': 'id_municipio'})
necropsis['id_municipio'] = necropsis['id_municipio'].astype('Int64')
necropsis.head()

,núm_corre,edad_per,causa_muerte,fecha,id_departamento,id_sexo,id_municipio
0,1,27,Traumatismo,2023-01-01,14,1,309
1,2,30,Traumatismo,2023-01-01,14,1,312
2,3,34,Asfixia,2023-01-01,14,1,308
3,4,33,Asfixia,2023-01-02,14,2,308
4,5,0,Indeterminada,2023-01-02,14,1,308


In [71]:
mapCausaMuerte = pd.read_excel(urlNectropsis,sheet_name='Sheet2', header=0)
causaMuerte = pd.DataFrame(mapCausaMuerte)
causaMuerte.head()

,id,nombre
0,1,Amputación
1,2,Asfixia
2,3,Cardiomiopatía
3,4,Decapitación
4,5,Depleción


In [72]:
causaMuerte ['nombre'] = causaMuerte['nombre'].apply(normalizar_texto)
necropsis = pd.merge(necropsis, causaMuerte, left_on='causa_muerte', right_on='nombre', how='left')
necropsis.head()


,núm_corre,edad_per,causa_muerte,fecha,id_departamento,id_sexo,id_municipio,id,nombre
0,1,27,Traumatismo,2023-01-01,14,1,309,37,Traumatismo
1,2,30,Traumatismo,2023-01-01,14,1,312,37,Traumatismo
2,3,34,Asfixia,2023-01-01,14,1,308,2,Asfixia
3,4,33,Asfixia,2023-01-02,14,2,308,2,Asfixia
4,5,0,Indeterminada,2023-01-02,14,1,308,21,Indeterminada


In [73]:
necropsis = necropsis.drop(columns=['causa_muerte','nombre'])
necropsis = necropsis.rename(columns={'id': 'id_causa_muerte','núm_corre':'id_necropsis'})
necropsis.head()

,id_necropsis,edad_per,fecha,id_departamento,id_sexo,id_municipio,id_causa_muerte
0,1,27,2023-01-01,14,1,309,37
1,2,30,2023-01-01,14,1,312,37
2,3,34,2023-01-01,14,1,308,2
3,4,33,2023-01-02,14,2,308,2
4,5,0,2023-01-02,14,1,308,21


In [74]:
necropsis.to_csv('necropsis.csv', index=False)
causaMuerte.to_csv('causaMuerte.csv', index=False)

In [75]:
urlSentenciadosOJ = '/content/OrganismoJudicial-Sentenciados.xlsx'
cargaExcelSentenciadosOJ = pd.read_excel(urlSentenciadosOJ, sheet_name='Sheet1', header=0)
sentenciadosOJ = pd.DataFrame(cargaExcelSentenciadosOJ)
sentenciadosOJ.head()


,núm_corre,año_reg,mes_reg,men_may,sexo,nacionalidad,Involucramiento,tip_fallo,depto_reg,delito_cod,tip_ley,título,capítulo
0,1,2023,Enero,Mayor,Hombre,guatemalteca,Acusado,Absolutoria,Baja Verapaz,Otros,Otras leyes,Otros títulos,Otros capítulos
1,2,2023,Enero,Mayor,Hombre,guatemalteca,Acusado,Absolutoria,Petén,Otros,Otras leyes,Otros títulos,Otros capítulos
2,3,2023,Enero,Mayor,Hombre,guatemalteca,Acusado,Condenatoria,Petén,Otros,Otras leyes,Otros títulos,Otros capítulos
3,4,2023,Enero,Mayor,Hombre,guatemalteca,Acusado,Absolutoria,Baja Verapaz,Otros,Otras leyes,Otros títulos,Otros capítulos
4,5,2023,Enero,Mayor,Hombre,guatemalteca,Acusado,Absolutoria,Petén,Otros,Otras leyes,Otros títulos,Otros capítulos


In [76]:
sentenciadosOJ['fecha'] = sentenciadosOJ.apply(
    lambda r: convertir_fecha(r['mes_reg'], r['año_reg']),
    axis=1
)
sentenciadosOJ = sentenciadosOJ.drop(columns=['año_reg','mes_reg'])
sentenciadosOJ['depto_reg'] = sentenciadosOJ['depto_reg'].apply(normalizar_texto)
sentenciadosOJ['id_departamento'] = sentenciadosOJ['depto_reg'].map(departamento_map)
sentenciadosOJ = sentenciadosOJ.drop(columns=['depto_reg'])
sentenciadosOJ

,núm_corre,men_may,sexo,nacionalidad,Involucramiento,tip_fallo,delito_cod,tip_ley,título,capítulo,fecha,id_departamento
0,1,Mayor,Hombre,guatemalteca,Acusado,Absolutoria,Otros,Otras leyes,Otros títulos,Otros capítulos,2023-01-01,12
1,2,Mayor,Hombre,guatemalteca,Acusado,Absolutoria,Otros,Otras leyes,Otros títulos,Otros capítulos,2023-01-01,23
2,3,Mayor,Hombre,guatemalteca,Acusado,Condenatoria,Otros,Otras leyes,Otros títulos,Otros capítulos,2023-01-01,23
3,4,Mayor,Hombre,guatemalteca,Acusado,Absolutoria,Otros,Otras leyes,Otros títulos,Otros capítulos,2023-01-01,12
4,5,Mayor,Hombre,guatemalteca,Acusado,Absolutoria,Otros,Otras leyes,Otros títulos,Otros capítulos,2023-01-01,23
...,...,...,...,...,...,...,...,...,...,...,...,...
59807,59808,Mayor,Hombre,guatemalteca,Sindicado,Condenatoria,Extorsión,Ley del Ramo Penal,De los delitos contra el patrimonio,De la extorsión y del chantaje (Código Penal),2023-12-01,1
59808,59809,Mayor,Hombre,guatemalteca,Sindicado,Condenatoria,Otros,Otras leyes,Otros títulos,Otros capítulos,2023-12-01,1
59809,59810,Mayor,Mujer,ignorada,Sindicado,Condenatoria,Extorsión,Ley del Ramo Penal,De los delitos contra el patrimonio,De la extorsión y del chantaje (Código Penal),2023-12-01,1
59810,59811,Mayor,Hombre,ignorada,Sindicado,Condenatoria,Extorsión,Ley del Ramo Penal,De los delitos contra el patrimonio,De la extorsión y del chantaje (Código Penal),2023-12-01,1


In [77]:
sentenciadosOJ ['sexo'] = sentenciadosOJ['sexo'].apply(normalizar_sexo)
sentenciadosOJ['id_sexo']= sentenciadosOJ['sexo'].map({v: k for k, v in sexo_map.items()})
sentenciadosOJ = sentenciadosOJ.drop(columns=['sexo'])
sentenciadosOJ

,núm_corre,men_may,nacionalidad,Involucramiento,tip_fallo,delito_cod,tip_ley,título,capítulo,fecha,id_departamento,id_sexo
0,1,Mayor,guatemalteca,Acusado,Absolutoria,Otros,Otras leyes,Otros títulos,Otros capítulos,2023-01-01,12,1
1,2,Mayor,guatemalteca,Acusado,Absolutoria,Otros,Otras leyes,Otros títulos,Otros capítulos,2023-01-01,23,1
2,3,Mayor,guatemalteca,Acusado,Condenatoria,Otros,Otras leyes,Otros títulos,Otros capítulos,2023-01-01,23,1
3,4,Mayor,guatemalteca,Acusado,Absolutoria,Otros,Otras leyes,Otros títulos,Otros capítulos,2023-01-01,12,1
4,5,Mayor,guatemalteca,Acusado,Absolutoria,Otros,Otras leyes,Otros títulos,Otros capítulos,2023-01-01,23,1
...,...,...,...,...,...,...,...,...,...,...,...,...
59807,59808,Mayor,guatemalteca,Sindicado,Condenatoria,Extorsión,Ley del Ramo Penal,De los delitos contra el patrimonio,De la extorsión y del chantaje (Código Penal),2023-12-01,1,1
59808,59809,Mayor,guatemalteca,Sindicado,Condenatoria,Otros,Otras leyes,Otros títulos,Otros capítulos,2023-12-01,1,1
59809,59810,Mayor,ignorada,Sindicado,Condenatoria,Extorsión,Ley del Ramo Penal,De los delitos contra el patrimonio,De la extorsión y del chantaje (Código Penal),2023-12-01,1,2
59810,59811,Mayor,ignorada,Sindicado,Condenatoria,Extorsión,Ley del Ramo Penal,De los delitos contra el patrimonio,De la extorsión y del chantaje (Código Penal),2023-12-01,1,1


In [78]:
sentenciadosOJ['capítulo'  ] = sentenciadosOJ['capítulo'].apply(normalizar_texto)
sentenciadosOJ['nacionalidad'  ] = sentenciadosOJ['nacionalidad'].apply(normalizar_texto)
sentenciadosOJ['Involucramiento'  ] = sentenciadosOJ['Involucramiento'].apply(normalizar_texto)
sentenciadosOJ['tip_fallo'  ] = sentenciadosOJ['tip_fallo'].apply(normalizar_texto)
sentenciadosOJ['delito_cod'  ] = sentenciadosOJ['delito_cod'].apply(normalizar_texto)
sentenciadosOJ['tip_ley'  ] = sentenciadosOJ['tip_ley'].apply(normalizar_texto)
sentenciadosOJ['título'  ] = sentenciadosOJ['título'].apply(normalizar_texto)
sentenciadosOJ['men_may'  ] = sentenciadosOJ['men_may'].apply(normalizar_texto)
sentenciadosOJ.head()

,núm_corre,men_may,nacionalidad,Involucramiento,tip_fallo,delito_cod,tip_ley,título,capítulo,fecha,id_departamento,id_sexo
0,1,Mayor,Guatemalteca,Acusado,Absolutoria,Otros,Otras Leyes,Otros Titulos,Otros Capitulos,2023-01-01,12,1
1,2,Mayor,Guatemalteca,Acusado,Absolutoria,Otros,Otras Leyes,Otros Titulos,Otros Capitulos,2023-01-01,23,1
2,3,Mayor,Guatemalteca,Acusado,Condenatoria,Otros,Otras Leyes,Otros Titulos,Otros Capitulos,2023-01-01,23,1
3,4,Mayor,Guatemalteca,Acusado,Absolutoria,Otros,Otras Leyes,Otros Titulos,Otros Capitulos,2023-01-01,12,1
4,5,Mayor,Guatemalteca,Acusado,Absolutoria,Otros,Otras Leyes,Otros Titulos,Otros Capitulos,2023-01-01,23,1


In [79]:
mapNacionalidad = pd.read_excel(urlSentenciadosOJ,sheet_name='nacionalidad', header=0)
mapNacionalidad = pd.DataFrame(mapNacionalidad)
mapNacionalidad.head()

,id,nombre
0,1,guatemalteca
1,2,salvadoreña
2,3,hondureña
3,4,nicaragüense
4,5,costaricense


In [80]:
mapNacionalidad['nombre'] = mapNacionalidad['nombre'].apply(normalizar_texto)
mapNacionalidad.to_csv('diccionarioNacionalidades.csv', index=False)
sentenciadosOJ = pd.merge(sentenciadosOJ, mapNacionalidad, left_on='nacionalidad', right_on='nombre', how='left')
sentenciadosOJ.head()

,núm_corre,men_may,nacionalidad,Involucramiento,tip_fallo,delito_cod,tip_ley,título,capítulo,fecha,id_departamento,id_sexo,id,nombre
0,1,Mayor,Guatemalteca,Acusado,Absolutoria,Otros,Otras Leyes,Otros Titulos,Otros Capitulos,2023-01-01,12,1,1.0,Guatemalteca
1,2,Mayor,Guatemalteca,Acusado,Absolutoria,Otros,Otras Leyes,Otros Titulos,Otros Capitulos,2023-01-01,23,1,1.0,Guatemalteca
2,3,Mayor,Guatemalteca,Acusado,Condenatoria,Otros,Otras Leyes,Otros Titulos,Otros Capitulos,2023-01-01,23,1,1.0,Guatemalteca
3,4,Mayor,Guatemalteca,Acusado,Absolutoria,Otros,Otras Leyes,Otros Titulos,Otros Capitulos,2023-01-01,12,1,1.0,Guatemalteca
4,5,Mayor,Guatemalteca,Acusado,Absolutoria,Otros,Otras Leyes,Otros Titulos,Otros Capitulos,2023-01-01,23,1,1.0,Guatemalteca


In [81]:
sentenciadosOJ = sentenciadosOJ.drop(columns=['nacionalidad','nombre'])
sentenciadosOJ = sentenciadosOJ.rename(columns={'id': 'id_nacionalidad'})
sentenciadosOJ.head()

,núm_corre,men_may,Involucramiento,tip_fallo,delito_cod,tip_ley,título,capítulo,fecha,id_departamento,id_sexo,id_nacionalidad
0,1,Mayor,Acusado,Absolutoria,Otros,Otras Leyes,Otros Titulos,Otros Capitulos,2023-01-01,12,1,1.0
1,2,Mayor,Acusado,Absolutoria,Otros,Otras Leyes,Otros Titulos,Otros Capitulos,2023-01-01,23,1,1.0
2,3,Mayor,Acusado,Condenatoria,Otros,Otras Leyes,Otros Titulos,Otros Capitulos,2023-01-01,23,1,1.0
3,4,Mayor,Acusado,Absolutoria,Otros,Otras Leyes,Otros Titulos,Otros Capitulos,2023-01-01,12,1,1.0
4,5,Mayor,Acusado,Absolutoria,Otros,Otras Leyes,Otros Titulos,Otros Capitulos,2023-01-01,23,1,1.0


In [82]:
diccInvolucramiento = pd.read_excel(urlSentenciadosOJ,sheet_name='invol', header=0)
diccInvolucramiento = pd.DataFrame(diccInvolucramiento)
diccInvolucramiento.head()

,id,nombre
0,1,Agresor
1,2,Denunciado
2,3,En contra
3,4,Procesado
4,5,Querellado


In [83]:
diccInvolucramiento['nombre'] = diccInvolucramiento['nombre'].apply(normalizar_texto)
diccInvolucramiento.to_csv('diccionarioInvolucramiento.csv', index=False)
sentenciadosOJ = pd.merge(sentenciadosOJ, diccInvolucramiento, left_on='Involucramiento', right_on='nombre', how='left')
sentenciadosOJ = sentenciadosOJ.drop(columns=['Involucramiento','nombre'])
sentenciadosOJ = sentenciadosOJ.rename(columns={'id': 'id_involucramiento'})
sentenciadosOJ.head()

,núm_corre,men_may,tip_fallo,delito_cod,tip_ley,título,capítulo,fecha,id_departamento,id_sexo,id_nacionalidad,id_involucramiento
0,1,Mayor,Absolutoria,Otros,Otras Leyes,Otros Titulos,Otros Capitulos,2023-01-01,12,1,1.0,8.0
1,2,Mayor,Absolutoria,Otros,Otras Leyes,Otros Titulos,Otros Capitulos,2023-01-01,23,1,1.0,8.0
2,3,Mayor,Condenatoria,Otros,Otras Leyes,Otros Titulos,Otros Capitulos,2023-01-01,23,1,1.0,8.0
3,4,Mayor,Absolutoria,Otros,Otras Leyes,Otros Titulos,Otros Capitulos,2023-01-01,12,1,1.0,8.0
4,5,Mayor,Absolutoria,Otros,Otras Leyes,Otros Titulos,Otros Capitulos,2023-01-01,23,1,1.0,8.0


In [84]:
diccionarioDelito = pd.read_excel(urlSentenciadosOJ,sheet_name='delitoCometido', header=0)
diccDelito = pd.DataFrame(diccionarioDelito)
diccDelito['nombre'] = diccDelito['nombre'].apply(normalizar_texto)
diccDelito.to_csv('diccionarioDelito.csv', index=False)
diccDelito.head()

,id,nombre
0,1,Homicidio Simple
1,2,Homicidio Estado De Emocion Violenta
2,3,Homicidio En Rina Tumultuaria
3,4,Homicidio Preter-Intencional
4,5,Homicidio Culposo


In [85]:
sentenciadosOJ = pd.merge(sentenciadosOJ, diccDelito, left_on='delito_cod', right_on='nombre', how='left')
sentenciadosOJ = sentenciadosOJ.drop(columns=['delito_cod','nombre'])
sentenciadosOJ = sentenciadosOJ.rename(columns={'id': 'id_delito'})
sentenciadosOJ.head()

,núm_corre,men_may,tip_fallo,tip_ley,título,capítulo,fecha,id_departamento,id_sexo,id_nacionalidad,id_involucramiento,id_delito
0,1,Mayor,Absolutoria,Otras Leyes,Otros Titulos,Otros Capitulos,2023-01-01,12,1,1.0,8.0,0.0
1,2,Mayor,Absolutoria,Otras Leyes,Otros Titulos,Otros Capitulos,2023-01-01,23,1,1.0,8.0,0.0
2,3,Mayor,Condenatoria,Otras Leyes,Otros Titulos,Otros Capitulos,2023-01-01,23,1,1.0,8.0,0.0
3,4,Mayor,Absolutoria,Otras Leyes,Otros Titulos,Otros Capitulos,2023-01-01,12,1,1.0,8.0,0.0
4,5,Mayor,Absolutoria,Otras Leyes,Otros Titulos,Otros Capitulos,2023-01-01,23,1,1.0,8.0,0.0


In [86]:
diccCapitulo = pd.read_excel(urlSentenciadosOJ,sheet_name='capitulo', header=0)
diccCapitulo = pd.DataFrame(diccCapitulo)
diccCapitulo['nombre'] = diccCapitulo['nombre'].apply(normalizar_texto)
diccCapitulo.head()

,id,nombre
0,1,Homicidios Simples (Codigo Penal)
1,2,Homicidios Calificados (Codigo Penal)
2,3,Lesiones (Codigo Penal)
3,4,De Los Delitos Contra La Libertad Individual (...
4,5,Hurto (Codigo Penal)


In [87]:
sentenciadosOJ = pd.merge(sentenciadosOJ, diccCapitulo, left_on='capítulo', right_on='nombre', how='left')
sentenciadosOJ = sentenciadosOJ.drop(columns=['capítulo','nombre'])
sentenciadosOJ = sentenciadosOJ.rename(columns={'id': 'id_capitulo'})
sentenciadosOJ.head()

,núm_corre,men_may,tip_fallo,tip_ley,título,fecha,id_departamento,id_sexo,id_nacionalidad,id_involucramiento,id_delito,id_capitulo
0,1,Mayor,Absolutoria,Otras Leyes,Otros Titulos,2023-01-01,12,1,1.0,8.0,0.0,NaN
1,2,Mayor,Absolutoria,Otras Leyes,Otros Titulos,2023-01-01,23,1,1.0,8.0,0.0,NaN
2,3,Mayor,Condenatoria,Otras Leyes,Otros Titulos,2023-01-01,23,1,1.0,8.0,0.0,NaN
3,4,Mayor,Absolutoria,Otras Leyes,Otros Titulos,2023-01-01,12,1,1.0,8.0,0.0,NaN
4,5,Mayor,Absolutoria,Otras Leyes,Otros Titulos,2023-01-01,23,1,1.0,8.0,0.0,NaN


In [88]:
# Find rows with NaN values in 'id_capitulo'
nan_rows = sentenciadosOJ[sentenciadosOJ['id_capitulo'].isnull()]

# Replace NaN values with 0
sentenciadosOJ['id_capitulo'] = sentenciadosOJ['id_capitulo'].fillna(0)

# Convert the column to integer type (optional)
sentenciadosOJ['id_capitulo'] = sentenciadosOJ['id_capitulo'].astype(int)
sentenciadosOJ

,núm_corre,men_may,tip_fallo,tip_ley,título,fecha,id_departamento,id_sexo,id_nacionalidad,id_involucramiento,id_delito,id_capitulo
0,1,Mayor,Absolutoria,Otras Leyes,Otros Titulos,2023-01-01,12,1,1.0,8.0,0.0,0
1,2,Mayor,Absolutoria,Otras Leyes,Otros Titulos,2023-01-01,23,1,1.0,8.0,0.0,0
2,3,Mayor,Condenatoria,Otras Leyes,Otros Titulos,2023-01-01,23,1,1.0,8.0,0.0,0
3,4,Mayor,Absolutoria,Otras Leyes,Otros Titulos,2023-01-01,12,1,1.0,8.0,0.0,0
4,5,Mayor,Absolutoria,Otras Leyes,Otros Titulos,2023-01-01,23,1,1.0,8.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
59807,59808,Mayor,Condenatoria,Ley Del Ramo Penal,De Los Delitos Contra El Patrimonio,2023-12-01,1,1,1.0,9.0,82.0,15
59808,59809,Mayor,Condenatoria,Otras Leyes,Otros Titulos,2023-12-01,1,1,1.0,9.0,0.0,0
59809,59810,Mayor,Condenatoria,Ley Del Ramo Penal,De Los Delitos Contra El Patrimonio,2023-12-01,1,2,99.0,9.0,82.0,15
59810,59811,Mayor,Condenatoria,Ley Del Ramo Penal,De Los Delitos Contra El Patrimonio,2023-12-01,1,1,99.0,9.0,82.0,15


In [89]:
diccCapitulo.to_csv('diccionarioCapitulo.csv', index=False)

In [90]:
diccTipoFayo = pd.read_excel(urlSentenciadosOJ,sheet_name='tipoFallo', header=0)
diccTipoFayo = pd.DataFrame(diccTipoFayo)
diccTipoFayo['nombre'] = diccTipoFayo['nombre'].apply(normalizar_texto)
diccTipoFayo.to_csv('diccionarioTipoFallo.csv', index=False)
sentenciadosOJ = pd.merge(sentenciadosOJ, diccTipoFayo, left_on='tip_fallo', right_on='nombre', how='left')
sentenciadosOJ = sentenciadosOJ.drop(columns=['tip_fallo','nombre'])
sentenciadosOJ = sentenciadosOJ.rename(columns={'id': 'id_tipo_fallo'})
sentenciadosOJ.head()

,núm_corre,men_may,tip_ley,título,fecha,id_departamento,id_sexo,id_nacionalidad,id_involucramiento,id_delito,id_capitulo,id_tipo_fallo
0,1,Mayor,Otras Leyes,Otros Titulos,2023-01-01,12,1,1.0,8.0,0.0,0,1
1,2,Mayor,Otras Leyes,Otros Titulos,2023-01-01,23,1,1.0,8.0,0.0,0,1
2,3,Mayor,Otras Leyes,Otros Titulos,2023-01-01,23,1,1.0,8.0,0.0,0,3
3,4,Mayor,Otras Leyes,Otros Titulos,2023-01-01,12,1,1.0,8.0,0.0,0,1
4,5,Mayor,Otras Leyes,Otros Titulos,2023-01-01,23,1,1.0,8.0,0.0,0,1


In [91]:
diccTitulo = pd.read_excel(urlSentenciadosOJ,sheet_name='titulo', header=0)
diccTitulo = pd.DataFrame(diccTitulo)
diccTitulo['nombre'] = diccTitulo['nombre'].apply(normalizar_texto)
diccTitulo.to_csv('diccionarioTitulo.csv', index=False)
sentenciadosOJ = pd.merge(sentenciadosOJ, diccTitulo, left_on='título', right_on='nombre', how='left')
sentenciadosOJ = sentenciadosOJ.drop(columns=['título','nombre'])
sentenciadosOJ = sentenciadosOJ.rename(columns={'id': 'id_titulo'})
sentenciadosOJ.head()

,núm_corre,men_may,tip_ley,fecha,id_departamento,id_sexo,id_nacionalidad,id_involucramiento,id_delito,id_capitulo,id_tipo_fallo,id_titulo
0,1,Mayor,Otras Leyes,2023-01-01,12,1,1.0,8.0,0.0,0,1,NaN
1,2,Mayor,Otras Leyes,2023-01-01,23,1,1.0,8.0,0.0,0,1,NaN
2,3,Mayor,Otras Leyes,2023-01-01,23,1,1.0,8.0,0.0,0,3,NaN
3,4,Mayor,Otras Leyes,2023-01-01,12,1,1.0,8.0,0.0,0,1,NaN
4,5,Mayor,Otras Leyes,2023-01-01,23,1,1.0,8.0,0.0,0,1,NaN


In [92]:
nan_rows = sentenciadosOJ[sentenciadosOJ['id_titulo'].isnull()]

# Replace NaN values with 0
sentenciadosOJ['id_titulo'] = sentenciadosOJ['id_titulo'].fillna(0)

# Convert the column to integer type (optional)
sentenciadosOJ['id_titulo'] = sentenciadosOJ['id_titulo'].astype(int)
sentenciadosOJ.head()

,núm_corre,men_may,tip_ley,fecha,id_departamento,id_sexo,id_nacionalidad,id_involucramiento,id_delito,id_capitulo,id_tipo_fallo,id_titulo
0,1,Mayor,Otras Leyes,2023-01-01,12,1,1.0,8.0,0.0,0,1,0
1,2,Mayor,Otras Leyes,2023-01-01,23,1,1.0,8.0,0.0,0,1,0
2,3,Mayor,Otras Leyes,2023-01-01,23,1,1.0,8.0,0.0,0,3,0
3,4,Mayor,Otras Leyes,2023-01-01,12,1,1.0,8.0,0.0,0,1,0
4,5,Mayor,Otras Leyes,2023-01-01,23,1,1.0,8.0,0.0,0,1,0


In [93]:
diccTipoLey = pd.read_excel(urlSentenciadosOJ,sheet_name='tipoLey', header=0)
diccTipoLey = pd.DataFrame(diccTipoLey)
diccTipoLey['nombre'] = diccTipoLey['nombre'].apply(normalizar_texto)
diccTipoLey.to_csv('diccionarioTipoLey.csv', index=False)
sentenciadosOJ = pd.merge(sentenciadosOJ, diccTipoLey, left_on='tip_ley', right_on='nombre', how='left')
sentenciadosOJ = sentenciadosOJ.drop(columns=['tip_ley','nombre'])
sentenciadosOJ = sentenciadosOJ.rename(columns={'id': 'id_tipo_ley'})
nan_rows = sentenciadosOJ[sentenciadosOJ['id_tipo_ley'].isnull()]

# Replace NaN values with 0
sentenciadosOJ['id_tipo_ley'] = sentenciadosOJ['id_tipo_ley'].fillna(0)

# Convert the column to integer type (optional)
sentenciadosOJ['id_tipo_ley'] = sentenciadosOJ['id_tipo_ley'].astype(int)
sentenciadosOJ.head()


,núm_corre,men_may,fecha,id_departamento,id_sexo,id_nacionalidad,id_involucramiento,id_delito,id_capitulo,id_tipo_fallo,id_titulo,id_tipo_ley
0,1,Mayor,2023-01-01,12,1,1.0,8.0,0.0,0,1,0,0
1,2,Mayor,2023-01-01,23,1,1.0,8.0,0.0,0,1,0,0
2,3,Mayor,2023-01-01,23,1,1.0,8.0,0.0,0,3,0,0
3,4,Mayor,2023-01-01,12,1,1.0,8.0,0.0,0,1,0,0
4,5,Mayor,2023-01-01,23,1,1.0,8.0,0.0,0,1,0,0


In [94]:
sentenciadosOJ['id_nacionalidad'] = sentenciadosOJ['id_nacionalidad'].astype('Int64')
sentenciadosOJ['id_involucramiento'] = sentenciadosOJ['id_involucramiento'].astype('Int64')
sentenciadosOJ['id_delito'] = sentenciadosOJ['id_delito'].astype('Int64')
# Assuming your DataFrame is named 'sentenciadosOJ'
sentenciadosOJ['men_may'] = sentenciadosOJ['men_may'].apply(lambda x: True if x == 'Mayor' else False)
sentenciadosOJ.head()

,núm_corre,men_may,fecha,id_departamento,id_sexo,id_nacionalidad,id_involucramiento,id_delito,id_capitulo,id_tipo_fallo,id_titulo,id_tipo_ley
0,1,True,2023-01-01,12,1,1,8,0,0,1,0,0
1,2,True,2023-01-01,23,1,1,8,0,0,1,0,0
2,3,True,2023-01-01,23,1,1,8,0,0,3,0,0
3,4,True,2023-01-01,12,1,1,8,0,0,1,0,0
4,5,True,2023-01-01,23,1,1,8,0,0,1,0,0


In [95]:
sentenciadosOJ = sentenciadosOJ.rename(columns={'núm_corre':'id'})
sentenciadosOJ.to_csv('sentenciadosOJ.csv', index=False)

In [96]:
urlSentenciadosPNC = '/content/PNC-Detenciados.xlsx'
cargaExcelSentenciadosPNC = pd.read_excel(urlSentenciadosPNC, sheet_name='Sheet1', header=0)
sentenciadosPNC = pd.DataFrame(cargaExcelSentenciadosPNC)
sentenciadosPNC.head()
#

,núm_corre,año_ocu,mes_ocu,día_ocu,día_sem_ocu,hora_ocu,g_hora,g_hora_mañ.tar.noch,área_geo_ocu,depto_ocu,mupio_ocu,zona_ocu,sexo_per,edad_per,g_edad_60ymás,g_edad_80ymás,edad_quinquenales,delito_com,g_delitos
0,1,2023,Enero,1,Domingo,7,06:00 a 11:59,00:00 a 11:59,Urbano,Guatemala,Guatemala,9,Hombre,35,35-39,35-39,35 - 39,Orden de captura,Otras causas
1,2,2023,Enero,1,Domingo,3,00:00 a 05:59,00:00 a 11:59,Urbano,Guatemala,Guatemala,1,Hombre,21,20-24,20-24,20 - 24,Otras causas,Otras causas
2,3,2023,Enero,1,Domingo,17,12:00 a 17:59,12:00 a 17:59,Urbano,Guatemala,Guatemala,18,Hombre,52,50-54,50-54,50 - 54,Otras causas,Otras causas
3,4,2023,Enero,1,Domingo,15,12:00 a 17:59,12:00 a 17:59,Rural,Guatemala,Palencia,Ignorada,Hombre,38,35-39,35-39,35 - 39,Otras causas,Otras causas
4,5,2023,Enero,1,Domingo,16,12:00 a 17:59,12:00 a 17:59,Rural,Guatemala,Chinautla,Ignorada,Hombre,24,20-24,20-24,20 - 24,Riña,Otras causas


In [97]:
sentenciadosPNC['fecha'] = sentenciadosPNC.apply(
    lambda r: convertir_fecha_dia(r['mes_ocu'], r['año_ocu'],r['día_ocu']),
    axis=1
)
sentenciadosPNC = sentenciadosPNC.drop(columns=['año_ocu','mes_ocu','día_ocu','día_sem_ocu','g_hora','g_hora_mañ.tar.noch','g_edad_60ymás','g_edad_80ymás','edad_quinquenales'])
sentenciadosPNC

,núm_corre,hora_ocu,área_geo_ocu,depto_ocu,mupio_ocu,zona_ocu,sexo_per,edad_per,delito_com,g_delitos,fecha
0,1,7,Urbano,Guatemala,Guatemala,9,Hombre,35,Orden de captura,Otras causas,2023-01-01
1,2,3,Urbano,Guatemala,Guatemala,1,Hombre,21,Otras causas,Otras causas,2023-01-01
2,3,17,Urbano,Guatemala,Guatemala,18,Hombre,52,Otras causas,Otras causas,2023-01-01
3,4,15,Rural,Guatemala,Palencia,Ignorada,Hombre,38,Otras causas,Otras causas,2023-01-01
4,5,16,Rural,Guatemala,Chinautla,Ignorada,Hombre,24,Riña,Otras causas,2023-01-01
...,...,...,...,...,...,...,...,...,...,...,...
45273,45274,1,Urbano,Sacatepéquez,Santo Domingo Xenacoj,Ignorada,Hombre,25,Otros robos y hurtos,Contra el patrimonio,2023-12-31
45274,45275,0,Rural,Sacatepéquez,Antigua Guatemala,Ignorada,Hombre,25,Otras causas,Otras causas,2023-12-31
45275,45276,0,Rural,Sacatepéquez,Antigua Guatemala,Ignorada,Mujer,24,Otras causas,Otras causas,2023-12-31
45276,45277,22,Rural,Sacatepéquez,Antigua Guatemala,Ignorada,Hombre,52,Otras causas,Otras causas,2023-12-31


In [98]:
sentenciadosPNC['depto_ocu'] = sentenciadosPNC['depto_ocu'].apply(normalizar_texto)
sentenciadosPNC['id_departamento'] = sentenciadosPNC['depto_ocu'].map(departamento_map)
sentenciadosPNC = sentenciadosPNC.drop(columns=['depto_ocu'])
sentenciadosPNC['mupio_ocu'] = sentenciadosPNC['mupio_ocu'].apply(normalizar_texto)
sentenciadosPNC['sexo_per'] = sentenciadosPNC['sexo_per'].apply(normalizar_sexo)
sentenciadosPNC['edad_per'] = sentenciadosPNC['edad_per'].apply(normalizar_edad).astype('Int64')
sentenciadosPNC.head()

,núm_corre,hora_ocu,área_geo_ocu,mupio_ocu,zona_ocu,sexo_per,edad_per,delito_com,g_delitos,fecha,id_departamento
0,1,7,Urbano,Guatemala,9,Hombre,35,Orden de captura,Otras causas,2023-01-01,1
1,2,3,Urbano,Guatemala,1,Hombre,21,Otras causas,Otras causas,2023-01-01,1
2,3,17,Urbano,Guatemala,18,Hombre,52,Otras causas,Otras causas,2023-01-01,1
3,4,15,Rural,Palencia,Ignorada,Hombre,38,Otras causas,Otras causas,2023-01-01,1
4,5,16,Rural,Chinautla,Ignorada,Hombre,24,Riña,Otras causas,2023-01-01,1


In [99]:
sentenciadosPNC = pd.merge(sentenciadosPNC, diccionarioMunicipios, left_on='mupio_ocu', right_on='municipio', how='left')
sentenciadosPNC.head()

,núm_corre,hora_ocu,área_geo_ocu,mupio_ocu,zona_ocu,sexo_per,edad_per,delito_com,g_delitos,fecha,id_departamento,id,municipio
0,1,7,Urbano,Guatemala,9,Hombre,35,Orden de captura,Otras causas,2023-01-01,1,1.0,Guatemala
1,2,3,Urbano,Guatemala,1,Hombre,21,Otras causas,Otras causas,2023-01-01,1,1.0,Guatemala
2,3,17,Urbano,Guatemala,18,Hombre,52,Otras causas,Otras causas,2023-01-01,1,1.0,Guatemala
3,4,15,Rural,Palencia,Ignorada,Hombre,38,Otras causas,Otras causas,2023-01-01,1,105.0,Palencia
4,5,16,Rural,Chinautla,Ignorada,Hombre,24,Riña,Otras causas,2023-01-01,1,106.0,Chinautla


In [100]:
sentenciadosPNC = sentenciadosPNC.drop(columns=['mupio_ocu','municipio'])
sentenciadosPNC = sentenciadosPNC.rename(columns={'id': 'id_municipio'})
sentenciadosPNC['área_geo_ocu'] = sentenciadosPNC['área_geo_ocu'].replace({'Urbano': 'Urbana', 'Ignorado': 'Ignorada'})
sentenciadosPNC.head()

,núm_corre,hora_ocu,área_geo_ocu,zona_ocu,sexo_per,edad_per,delito_com,g_delitos,fecha,id_departamento,id_municipio
0,1,7,Urbana,9,Hombre,35,Orden de captura,Otras causas,2023-01-01,1,1.0
1,2,3,Urbana,1,Hombre,21,Otras causas,Otras causas,2023-01-01,1,1.0
2,3,17,Urbana,18,Hombre,52,Otras causas,Otras causas,2023-01-01,1,1.0
3,4,15,Rural,Ignorada,Hombre,38,Otras causas,Otras causas,2023-01-01,1,105.0
4,5,16,Rural,Ignorada,Hombre,24,Riña,Otras causas,2023-01-01,1,106.0


In [101]:
sentenciadosPNC['área_geo_ocu'] = sentenciadosPNC['área_geo_ocu'].replace({'Urbano': 'Urbana', 'Ignorado': 'Ignorada'})
sentenciadosPNC.head()

,núm_corre,hora_ocu,área_geo_ocu,zona_ocu,sexo_per,edad_per,delito_com,g_delitos,fecha,id_departamento,id_municipio
0,1,7,Urbana,9,Hombre,35,Orden de captura,Otras causas,2023-01-01,1,1.0
1,2,3,Urbana,1,Hombre,21,Otras causas,Otras causas,2023-01-01,1,1.0
2,3,17,Urbana,18,Hombre,52,Otras causas,Otras causas,2023-01-01,1,1.0
3,4,15,Rural,Ignorada,Hombre,38,Otras causas,Otras causas,2023-01-01,1,105.0
4,5,16,Rural,Ignorada,Hombre,24,Riña,Otras causas,2023-01-01,1,106.0


In [102]:
sentenciadosPNC ['área_geo_ocu'] = sentenciadosPNC['área_geo_ocu'].apply(normalizar_texto)
sentenciadosPNC ['id_area'] = sentenciadosPNC['área_geo_ocu'].map({v: k for k, v in area_geografica_map.items()})
sentenciadosPNC = sentenciadosPNC.drop(columns=['área_geo_ocu'])
sentenciadosPNC.head()

,núm_corre,hora_ocu,zona_ocu,sexo_per,edad_per,delito_com,g_delitos,fecha,id_departamento,id_municipio,id_area
0,1,7,9,Hombre,35,Orden de captura,Otras causas,2023-01-01,1,1.0,1
1,2,3,1,Hombre,21,Otras causas,Otras causas,2023-01-01,1,1.0,1
2,3,17,18,Hombre,52,Otras causas,Otras causas,2023-01-01,1,1.0,1
3,4,15,Ignorada,Hombre,38,Otras causas,Otras causas,2023-01-01,1,105.0,2
4,5,16,Ignorada,Hombre,24,Riña,Otras causas,2023-01-01,1,106.0,2


In [103]:
sentenciadosPNC['id_sexo'] = sentenciadosPNC['sexo_per'].map({v: k for k, v in sexo_map.items()})
sentenciadosPNC = sentenciadosPNC.drop(columns=['sexo_per'])
sentenciadosPNC.head()

,núm_corre,hora_ocu,zona_ocu,edad_per,delito_com,g_delitos,fecha,id_departamento,id_municipio,id_area,id_sexo
0,1,7,9,35,Orden de captura,Otras causas,2023-01-01,1,1.0,1,1
1,2,3,1,21,Otras causas,Otras causas,2023-01-01,1,1.0,1,1
2,3,17,18,52,Otras causas,Otras causas,2023-01-01,1,1.0,1,1
3,4,15,Ignorada,38,Otras causas,Otras causas,2023-01-01,1,105.0,2,1
4,5,16,Ignorada,24,Riña,Otras causas,2023-01-01,1,106.0,2,1


In [104]:
sentenciadosPNC['delito_com'] = sentenciadosPNC['delito_com'].apply(normalizar_texto)
delitosComPNC = pd.read_excel(urlSentenciadosPNC,sheet_name='causa', header=0)
delitosComPNC = pd.DataFrame(delitosComPNC)
delitosComPNC['nombre'] = delitosComPNC['nombre'].apply(normalizar_texto)
delitosComPNC.head()

,id,nombre
0,1,Homicidio Por Arma De Fuego
1,2,Homicidio Por Arma Blanca
2,3,Homicidio Por Arma Contundente
3,4,Homicidio Por Artefacto Explosivo
4,5,Homicidio Por Estrangulamiento


In [105]:
sentenciadosPNC = pd.merge(sentenciadosPNC, delitosComPNC, left_on='delito_com', right_on='nombre', how='left')
sentenciadosPNC = sentenciadosPNC.drop(columns=['delito_com','nombre'])
sentenciadosPNC = sentenciadosPNC.rename(columns={'id': 'id_delito'})
sentenciadosPNC.head()

,núm_corre,hora_ocu,zona_ocu,edad_per,g_delitos,fecha,id_departamento,id_municipio,id_area,id_sexo,id_delito
0,1,7,9,35,Otras causas,2023-01-01,1,1.0,1,1,47.0
1,2,3,1,21,Otras causas,2023-01-01,1,1.0,1,1,NaN
2,3,17,18,52,Otras causas,2023-01-01,1,1.0,1,1,NaN
3,4,15,Ignorada,38,Otras causas,2023-01-01,1,105.0,2,1,NaN
4,5,16,Ignorada,24,Otras causas,2023-01-01,1,106.0,2,1,48.0


In [106]:
sentenciadosPNC['id_delito'] = sentenciadosPNC['id_delito'].fillna(68)
sentenciadosPNC['id_delito'] = sentenciadosPNC['id_delito'].astype('Int64')
sentenciadosPNC.head()

,núm_corre,hora_ocu,zona_ocu,edad_per,g_delitos,fecha,id_departamento,id_municipio,id_area,id_sexo,id_delito
0,1,7,9,35,Otras causas,2023-01-01,1,1.0,1,1,47
1,2,3,1,21,Otras causas,2023-01-01,1,1.0,1,1,68
2,3,17,18,52,Otras causas,2023-01-01,1,1.0,1,1,68
3,4,15,Ignorada,38,Otras causas,2023-01-01,1,105.0,2,1,68
4,5,16,Ignorada,24,Otras causas,2023-01-01,1,106.0,2,1,48


In [107]:
delito_map = {
    1: "Homicidios",
    2: "Lesiones",
    3: "Contra El Patrimonio",
    5: "Contra La Libertad",
    6: "Extorsion Y Chantaje",
    7: "Otras Causas"
}

sentenciadosPNC['g_delitos'] = sentenciadosPNC['g_delitos'].apply(normalizar_texto)
sentenciadosPNC['id_g_delito_principal'] = sentenciadosPNC['g_delitos'].map({v: k for k, v in delito_map.items()})
sentenciadosPNC

,núm_corre,hora_ocu,zona_ocu,edad_per,g_delitos,fecha,id_departamento,id_municipio,id_area,id_sexo,id_delito,id_g_delito_principal
0,1,7,9,35,Otras Causas,2023-01-01,1,1.0,1,1,47,7.0
1,2,3,1,21,Otras Causas,2023-01-01,1,1.0,1,1,68,7.0
2,3,17,18,52,Otras Causas,2023-01-01,1,1.0,1,1,68,7.0
3,4,15,Ignorada,38,Otras Causas,2023-01-01,1,105.0,2,1,68,7.0
4,5,16,Ignorada,24,Otras Causas,2023-01-01,1,106.0,2,1,48,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...
45273,45274,1,Ignorada,25,Contra El Patrimonio,2023-12-31,14,305.0,1,1,36,3.0
45274,45275,0,Ignorada,25,Otras Causas,2023-12-31,14,301.0,2,1,68,7.0
45275,45276,0,Ignorada,24,Otras Causas,2023-12-31,14,301.0,2,2,68,7.0
45276,45277,22,Ignorada,52,Otras Causas,2023-12-31,14,301.0,2,1,68,7.0


In [108]:
sentenciadosPNC = sentenciadosPNC.drop(columns=['g_delitos'])
sentenciadosPNC = sentenciadosPNC.rename(columns={'núm_corre':'id'})
sentenciadosPNC['zona_ocu'] = sentenciadosPNC['zona_ocu'].replace('Ignorada',np.nan)

delito_df = pd.DataFrame(list(delito_map.items()), columns=['id', 'delito'])
delito_df.to_csv('delito_general_df.csv', index=False)

sentenciadosPNC.to_csv('sentenciadosPNC.csv', index=False)

delitosComPNC.to_csv('delitosComPNC.csv', index=False)


<ipython-input-108-350728b6c159>:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  sentenciadosPNC['zona_ocu'] = sentenciadosPNC['zona_ocu'].replace('Ignorada',np.nan)


In [109]:
urlAtencionVCM = '/content/Atenciones brindades por el Instituto de la Víctima 2020-2023(1).xlsx'
cargaExcelAtencionVCM = pd.read_excel(urlAtencionVCM, sheet_name='2020-2023', header=0)
atencionVCM = pd.DataFrame(cargaExcelAtencionVCM)
atencionVCM.head()
#

,Fecha de atención,Día,Mes,Año,Sede de la atención,Depatamento de la sede,Municipio de la sede,Tipo del delito atendido,Atención brindada,Departamento de procedencia de la víctima,Municipio de la procedencia de la víctima,Edad,Tango de edad,Grupo étnico,Orientación sexual,Valor
0,2021-08-13,viernes,agosto,2021,SEDE CENTRAL,GUATEMALA,GUATEMALA,VIOLENCIA CONTRA LA MUJER,LEGAL/VICTIMOLÓGICO,GUATEMALA,GUATEMALA,22,21 a 30 años,LADINO Y/O MESTIZO,HETEROSEXUAL,1
1,2020-12-08,martes,diciembre,2020,SEDE JUTIAPA,JUTIAPA,JUTIAPA,VIOLENCIA PSICOLÓGICA,LEGAL/VICTIMOLÓGICO,ALTA VERAPAZ,SAN PEDRO CARCHÁ,33,31 a 40 años,SD,HETEROSEXUAL,1
2,2022-03-16,miércoles,marzo,2022,SEDE CENTRAL,GUATEMALA,GUATEMALA,VIOLENCIA CONTRA LA MUJER,LEGAL/VICTIMOLÓGICO,GUATEMALA,VILLA NUEVA,36,31 a 40 años,LADINO Y/O MESTIZO,HETEROSEXUAL,1
3,2020-12-10,jueves,diciembre,2020,SEDE ALTA VERAPAZ,ALTA VERAPAZ,COBÁN,VIOLENCIA CONTRA LA MUJER,LEGAL/VICTIMOLÓGICO,SUCHITEPÉQUEZ,CHICACAO,51,51 a 60 años,SD,SD,1
4,2020-10-18,domingo,octubre,2020,SEDE CENTRAL,GUATEMALA,GUATEMALA,VIOLENCIA CONTRA LA MUJER,LEGAL/VICTIMOLÓGICO,TOTONICAPÁN,TOTONICAPÁN,31,31 a 40 años,SD,SD,1


In [110]:
atencionVCM = atencionVCM.drop(columns=['Día','Mes','Año','Tango de edad'])
atencionVCM['Sede de la atención'] = atencionVCM['Sede de la atención'].apply(normalizar_texto)
atencionVCM['Depatamento de la sede'] = atencionVCM['Depatamento de la sede'].apply(normalizar_texto)
atencionVCM['Municipio de la sede'] = atencionVCM['Municipio de la sede'].apply(normalizar_texto)
atencionVCM['Tipo del delito atendido'] = atencionVCM['Tipo del delito atendido'].apply(normalizar_texto)
atencionVCM['Atención brindada'] = atencionVCM['Atención brindada'].apply(normalizar_texto)
atencionVCM['Departamento de procedencia de la víctima'] = atencionVCM['Departamento de procedencia de la víctima'].apply(normalizar_texto)
atencionVCM['Municipio de la procedencia de la víctima'] = atencionVCM['Municipio de la procedencia de la víctima'].apply(normalizar_texto)
atencionVCM['Grupo étnico'] = atencionVCM['Grupo étnico'].apply(normalizar_texto)
atencionVCM['Orientación sexual'] = atencionVCM['Orientación sexual'].apply(normalizar_texto)
atencionVCM['Edad'] = atencionVCM['Edad'].apply(normalizar_edad).astype('Int64')
atencionVCM.head()

,Fecha de atención,Sede de la atención,Depatamento de la sede,Municipio de la sede,Tipo del delito atendido,Atención brindada,Departamento de procedencia de la víctima,Municipio de la procedencia de la víctima,Edad,Grupo étnico,Orientación sexual,Valor
0,2021-08-13,Sede Central,Guatemala,Guatemala,Violencia Contra La Mujer,Legal/Victimologico,Guatemala,Guatemala,22,Ladino Y/O Mestizo,Heterosexual,1
1,2020-12-08,Sede Jutiapa,Jutiapa,Jutiapa,Violencia Psicologica,Legal/Victimologico,Alta Verapaz,San Pedro Carcha,33,Sd,Heterosexual,1
2,2022-03-16,Sede Central,Guatemala,Guatemala,Violencia Contra La Mujer,Legal/Victimologico,Guatemala,Villa Nueva,36,Ladino Y/O Mestizo,Heterosexual,1
3,2020-12-10,Sede Alta Verapaz,Alta Verapaz,Coban,Violencia Contra La Mujer,Legal/Victimologico,Suchitepequez,Chicacao,51,Sd,Sd,1
4,2020-10-18,Sede Central,Guatemala,Guatemala,Violencia Contra La Mujer,Legal/Victimologico,Totonicapan,Totonicapan,31,Sd,Sd,1


In [111]:
atencionVCM['id_depa_sede'] = atencionVCM['Depatamento de la sede'].map(departamento_map)
atencionVCM = atencionVCM.drop(columns=['Depatamento de la sede'])
atencionVCM['id_depa_procedencia'] = atencionVCM['Departamento de procedencia de la víctima'].map(departamento_map)
atencionVCM = atencionVCM.drop(columns=['Departamento de procedencia de la víctima'])
atencionVCM = pd.merge(atencionVCM, diccionarioMunicipios, left_on='Municipio de la sede', right_on='municipio', how='left')
atencionVCM = atencionVCM.drop(columns=['municipio','Municipio de la sede'])
atencionVCM = atencionVCM.rename(columns={'id': 'id_municipio_sede'})
atencionVCM = pd.merge(atencionVCM, diccionarioMunicipios, left_on='Municipio de la procedencia de la víctima', right_on='municipio', how='left')
atencionVCM = atencionVCM.drop(columns=['municipio','Municipio de la procedencia de la víctima'])
atencionVCM = atencionVCM.rename(columns={'id': 'id_municipio_procedencia'})
atencionVCM.head()

,Fecha de atención,Sede de la atención,Tipo del delito atendido,Atención brindada,Edad,Grupo étnico,Orientación sexual,Valor,id_depa_sede,id_depa_procedencia,id_municipio_sede,id_municipio_procedencia
0,2021-08-13,Sede Central,Violencia Contra La Mujer,Legal/Victimologico,22,Ladino Y/O Mestizo,Heterosexual,1,1,1.0,1,1.0
1,2020-12-08,Sede Jutiapa,Violencia Psicologica,Legal/Victimologico,33,Sd,Heterosexual,1,4,3.0,22,1609.0
2,2022-03-16,Sede Central,Violencia Contra La Mujer,Legal/Victimologico,36,Ladino Y/O Mestizo,Heterosexual,1,1,1.0,1,115.0
3,2020-12-10,Sede Alta Verapaz,Violencia Contra La Mujer,Legal/Victimologico,51,Sd,Sd,1,3,21.0,1601,1013.0
4,2020-10-18,Sede Central,Violencia Contra La Mujer,Legal/Victimologico,31,Sd,Sd,1,1,2.0,1,8.0


In [112]:
atencionVCM['id_depa_procedencia'] = atencionVCM['id_depa_procedencia'].astype('Int64')
atencionVCM['id_municipio_procedencia'] = atencionVCM['id_municipio_procedencia'].astype('Int64')
atencionVCM.head()

,Fecha de atención,Sede de la atención,Tipo del delito atendido,Atención brindada,Edad,Grupo étnico,Orientación sexual,Valor,id_depa_sede,id_depa_procedencia,id_municipio_sede,id_municipio_procedencia
0,2021-08-13,Sede Central,Violencia Contra La Mujer,Legal/Victimologico,22,Ladino Y/O Mestizo,Heterosexual,1,1,1,1,1
1,2020-12-08,Sede Jutiapa,Violencia Psicologica,Legal/Victimologico,33,Sd,Heterosexual,1,4,3,22,1609
2,2022-03-16,Sede Central,Violencia Contra La Mujer,Legal/Victimologico,36,Ladino Y/O Mestizo,Heterosexual,1,1,1,1,115
3,2020-12-10,Sede Alta Verapaz,Violencia Contra La Mujer,Legal/Victimologico,51,Sd,Sd,1,3,21,1601,1013
4,2020-10-18,Sede Central,Violencia Contra La Mujer,Legal/Victimologico,31,Sd,Sd,1,1,2,1,8


In [113]:
# Obtener valores únicos
sede_vals = atencionVCM['Sede de la atención'].dropna().unique()
delito_vals = atencionVCM['Tipo del delito atendido'].dropna().unique()
atencion_vals = atencionVCM['Atención brindada'].dropna().unique()

# Crear diccionarios numerados
sede_dict = {i+1: v for i, v in enumerate(sorted(sede_vals))}
tipo_delito_dict = {i+1: v for i, v in enumerate(sorted(delito_vals))}
atencion_dict = {i+1: v for i, v in enumerate(sorted(atencion_vals))}


In [114]:
# Invertir los diccionarios para mapear valor → id
sede_id_map = {v: k for k, v in sede_dict.items()}
delito_id_map = {v: k for k, v in tipo_delito_dict.items()}
atencion_id_map = {v: k for k, v in atencion_dict.items()}

atencionVCM['id_sede'] = atencionVCM['Sede de la atención'].map(sede_id_map)
atencionVCM['id_tipo_delito'] = atencionVCM['Tipo del delito atendido'].map(delito_id_map)
atencionVCM['id_atencion'] = atencionVCM['Atención brindada'].map(atencion_id_map)


In [115]:
valores_grupo_etnico = sorted(atencionVCM['Grupo étnico'].dropna().unique())
valores_orientacion = sorted(atencionVCM['Orientación sexual'].dropna().unique())

# Crear diccionarios con ID automático
grupo_etnico_dict = {i+1: v for i, v in enumerate(valores_grupo_etnico)}
orientacion_sexual_dict = {i+1: v for i, v in enumerate(valores_orientacion)}

# Invertir los diccionarios
grupo_etnico_map = {v: k for k, v in grupo_etnico_dict.items()}
orientacion_sexual_map = {v: k for k, v in orientacion_sexual_dict.items()}

# Asignar columnas de ID
atencionVCM['id_grupo_etnico'] = atencionVCM['Grupo étnico'].map(grupo_etnico_map)
atencionVCM['id_orientacion_sexual'] = atencionVCM['Orientación sexual'].map(orientacion_sexual_map)
atencionVCM.head()

,Fecha de atención,Sede de la atención,Tipo del delito atendido,Atención brindada,Edad,Grupo étnico,Orientación sexual,Valor,id_depa_sede,id_depa_procedencia,id_municipio_sede,id_municipio_procedencia,id_sede,id_tipo_delito,id_atencion,id_grupo_etnico,id_orientacion_sexual
0,2021-08-13,Sede Central,Violencia Contra La Mujer,Legal/Victimologico,22,Ladino Y/O Mestizo,Heterosexual,1,1,1,1,1,2,1,1,1,3
1,2020-12-08,Sede Jutiapa,Violencia Psicologica,Legal/Victimologico,33,Sd,Heterosexual,1,4,3,22,1609,6,4,1,13,3
2,2022-03-16,Sede Central,Violencia Contra La Mujer,Legal/Victimologico,36,Ladino Y/O Mestizo,Heterosexual,1,1,1,1,115,2,1,1,1,3
3,2020-12-10,Sede Alta Verapaz,Violencia Contra La Mujer,Legal/Victimologico,51,Sd,Sd,1,3,21,1601,1013,1,1,1,13,5
4,2020-10-18,Sede Central,Violencia Contra La Mujer,Legal/Victimologico,31,Sd,Sd,1,1,2,1,8,2,1,1,13,5


In [116]:
atencionVCM = atencionVCM.drop(columns=['Sede de la atención','Tipo del delito atendido','Atención brindada','Grupo étnico','Orientación sexual'])
atencionVCM.head()

,Fecha de atención,Edad,Valor,id_depa_sede,id_depa_procedencia,id_municipio_sede,id_municipio_procedencia,id_sede,id_tipo_delito,id_atencion,id_grupo_etnico,id_orientacion_sexual
0,2021-08-13,22,1,1,1,1,1,2,1,1,1,3
1,2020-12-08,33,1,4,3,22,1609,6,4,1,13,3
2,2022-03-16,36,1,1,1,1,115,2,1,1,1,3
3,2020-12-10,51,1,3,21,1601,1013,1,1,1,13,5
4,2020-10-18,31,1,1,2,1,8,2,1,1,13,5


In [117]:


# Convertir a DataFrame
df_grupo_etnico = pd.DataFrame(list(grupo_etnico_map.items()), columns=['nombre', 'id'])
df_orientacion_sexual = pd.DataFrame(list(orientacion_sexual_map.items()), columns=['nombre', 'id'])
df_sede = pd.DataFrame(list(sede_id_map.items()), columns=['nombre', 'id'])
df_delito = pd.DataFrame(list(delito_id_map.items()), columns=['nombre', 'id'])
df_atencion = pd.DataFrame(list(atencion_id_map.items()), columns=['nombre', 'id'])

# Reordenar columnas para tener primero el id
df_grupo_etnico = df_grupo_etnico[['id', 'nombre']]
df_orientacion_sexual = df_orientacion_sexual[['id', 'nombre']]
df_sede = df_sede[['id', 'nombre']]
df_delito = df_delito[['id', 'nombre']]
df_atencion = df_atencion[['id', 'nombre']]

# Exportar a CSV
df_grupo_etnico.to_csv('grupo_etnico.csv', index=False)
df_orientacion_sexual.to_csv('orientacion_sexual.csv', index=False)
df_sede.to_csv('sede_atencion.csv', index=False)
df_delito.to_csv('tipo_delito.csv', index=False)
df_atencion.to_csv('atencion_brindada.csv', index=False)


In [118]:
atencionVCM['id'] = range(1, len(atencionVCM) + 1)
atencionVCM = atencionVCM.rename(columns={'id': 'id_atencion_vcm','Fecha de atención':'fecha'})
atencionVCM.head()

,fecha,Edad,Valor,id_depa_sede,id_depa_procedencia,id_municipio_sede,id_municipio_procedencia,id_sede,id_tipo_delito,id_atencion,id_grupo_etnico,id_orientacion_sexual,id_atencion_vcm
0,2021-08-13,22,1,1,1,1,1,2,1,1,1,3,1
1,2020-12-08,33,1,4,3,22,1609,6,4,1,13,3,2
2,2022-03-16,36,1,1,1,1,115,2,1,1,1,3,3
3,2020-12-10,51,1,3,21,1601,1013,1,1,1,13,5,4
4,2020-10-18,31,1,1,2,1,8,2,1,1,13,5,5


In [119]:
atencionVCM.to_csv('atencionVCM.csv', index=False)

In [120]:
urlfemicidiosMP_VCM = '/content/Delitos contra la vida de las mujeres y femicidios del MP.xlsx'
cargarFemicidiosMP_VCM = pd.read_excel(urlfemicidiosMP_VCM, sheet_name='2008-2024', header=0)
femicidiosMP_VCM = pd.DataFrame(cargarFemicidiosMP_VCM)
femicidiosMP_VCM.head()


,Departamento,Municipio,Año,Delito,Estado del Caso,Valor
0,Alta Verapaz,Chahal,2008,Ejecución extrajudicial,Desestimado,3
1,Alta Verapaz,Chahal,2008,Homicidio,Desestimado,2
2,Alta Verapaz,Chahal,2008,Homicidio,Conexado,2
3,Alta Verapaz,Chahal,2008,Asesinato,Desestimado,2
4,Alta Verapaz,Chahal,2010,Homicidio,Archivado,1


In [121]:
femicidiosMP_VCM['Departamento'] = femicidiosMP_VCM['Departamento'].apply(normalizar_texto)
femicidiosMP_VCM['id_departamento'] = femicidiosMP_VCM['Departamento'].map(departamento_map)
femicidiosMP_VCM = femicidiosMP_VCM.drop(columns=['Departamento'])
femicidiosMP_VCM['Municipio'] = femicidiosMP_VCM['Municipio'].apply(normalizar_texto)
femicidiosMP_VCM = pd.merge(femicidiosMP_VCM, diccionarioMunicipios, left_on='Municipio', right_on='municipio', how='left')
femicidiosMP_VCM = femicidiosMP_VCM.drop(columns=['municipio','Municipio'])
femicidiosMP_VCM = femicidiosMP_VCM.rename(columns={'id': 'id_municipio'})
femicidiosMP_VCM['Delito'] = femicidiosMP_VCM['Delito'].apply(normalizar_texto)
femicidiosMP_VCM['Estado del Caso'] = femicidiosMP_VCM['Estado del Caso'].apply(normalizar_texto)
femicidiosMP_VCM['id'] = range(1, len(femicidiosMP_VCM) + 1)
femicidiosMP_VCM

,Año,Delito,Estado del Caso,Valor,id_departamento,id_municipio,id
0,2008,Ejecucion Extrajudicial,Desestimado,3,3,1614.0,1
1,2008,Homicidio,Desestimado,2,3,1614.0,2
2,2008,Homicidio,Conexado,2,3,1614.0,3
3,2008,Asesinato,Desestimado,2,3,1614.0,4
4,2010,Homicidio,Archivado,1,3,1614.0,5
...,...,...,...,...,...,...,...
11241,2022,Homicidio,En Investigacion,1,22,1911.0,11242
11242,2016,Homicidio,Archivado,1,22,1911.0,11243
11243,2016,Femicidio,Archivado,1,22,1911.0,11244
11244,2014,Homicidio,Archivado,1,22,1911.0,11245


In [122]:
femicidiosMP_VCM['id_municipio'] = femicidiosMP_VCM['id_municipio'].astype('Int64')
# Crear diccionario para "Delito"
delito_dict = {v: i + 1 for i, v in enumerate(sorted(femicidiosMP_VCM['Delito'].dropna().unique()))}

# Crear diccionario para "Estado del Caso"
estado_caso_dict = {v: i + 1 for i, v in enumerate(sorted(femicidiosMP_VCM['Estado del Caso'].dropna().unique()))}


In [123]:
# 2. Crear diccionarios basados en los valores normalizados
delito_dict = {v: i + 1 for i, v in enumerate(sorted(femicidiosMP_VCM['Delito'].dropna().unique()))}
estado_caso_dict = {v: i + 1 for i, v in enumerate(sorted(femicidiosMP_VCM['Estado del Caso'].dropna().unique()))}

# 3. Mapear usando los diccionarios directamente
femicidiosMP_VCM['id_delito'] = femicidiosMP_VCM['Delito'].map(delito_dict)
femicidiosMP_VCM['id_estado_caso'] = femicidiosMP_VCM['Estado del Caso'].map(estado_caso_dict)

# 4. Exportar diccionarios a DataFrame
df_delito_MP_VCM = pd.DataFrame(list(delito_dict.items()), columns=['Delito', 'id_delito'])
df_estado_caso = pd.DataFrame(list(estado_caso_dict.items()), columns=['Estado_del_Caso', 'id_estado_caso'])
# Exportar a CSV
df_delito_MP_VCM.to_csv('diccionario_delito_MP_VCM.csv', index=False, encoding='utf-8')
df_estado_caso.to_csv('diccionario_estado_caso.csv', index=False, encoding='utf-8')
femicidiosMP_VCM

,Año,Delito,Estado del Caso,Valor,id_departamento,id_municipio,id,id_delito,id_estado_caso
0,2008,Ejecucion Extrajudicial,Desestimado,3,3,1614,1,2,9
1,2008,Homicidio,Desestimado,2,3,1614,2,4,9
2,2008,Homicidio,Conexado,2,3,1614,3,4,7
3,2008,Asesinato,Desestimado,2,3,1614,4,1,9
4,2010,Homicidio,Archivado,1,3,1614,5,4,2
...,...,...,...,...,...,...,...,...,...
11241,2022,Homicidio,En Investigacion,1,22,1911,11242,4,10
11242,2016,Homicidio,Archivado,1,22,1911,11243,4,2
11243,2016,Femicidio,Archivado,1,22,1911,11244,3,2
11244,2014,Homicidio,Archivado,1,22,1911,11245,4,2


In [124]:
femicidiosMP_VCM = femicidiosMP_VCM.drop(columns=['Delito','Estado del Caso'])
femicidiosMP_VCM.to_csv('femicidiosMP_VCM.csv', index=False)
#

In [125]:
urlDEnuncias_MP_VCM = '/content/Denuncias del MP por el delito de VCM.xlsx'
cargarDenuncias_MP_VCM = pd.read_excel(urlDEnuncias_MP_VCM, sheet_name='Denuncias 2008-2024', header=0)
denuncias_MP_VCM = pd.DataFrame(cargarDenuncias_MP_VCM)
denuncias_MP_VCM.head()
#

,Fecha,Dia,Mes,Año,Departamento,Municipio,Edad,Rangos de edad,Escolaridad,Pueblo de Pertenencia,Orientacion,Estado Caso,Valor
0,2008-06-01,domingo,junio,2008,Alta Verapaz,Cobán,73,de 61 años a más,No registrado,Sin selección,Sin selección,Sobreseido,1
1,2008-06-02,lunes,junio,2008,Alta Verapaz,Cobán,20,de 11 a 20 años,No registrado,Sin selección,Sin selección,Sobreseido,1
2,2008-06-02,lunes,junio,2008,Alta Verapaz,Cobán,28,de 21 a 30 años,No registrado,Sin selección,Sin selección,Desestimado,1
3,2008-06-02,lunes,junio,2008,Alta Verapaz,Cobán,37,de 31 a 40 años,No registrado,Sin selección,Sin selección,Archivado,1
4,2008-06-03,martes,junio,2008,Alta Verapaz,Cobán,SD,SD,No registrado,Sin selección,Sin selección,Desestimado,1


In [126]:
denuncias_MP_VCM['Departamento'] = denuncias_MP_VCM['Departamento'].apply(normalizar_texto)
denuncias_MP_VCM['id_departamento'] = denuncias_MP_VCM['Departamento'].map(departamento_map)
denuncias_MP_VCM = denuncias_MP_VCM.drop(columns=['Departamento'])
denuncias_MP_VCM['Municipio'] = denuncias_MP_VCM['Municipio'].apply(normalizar_texto)
denuncias_MP_VCM = pd.merge(denuncias_MP_VCM, diccionarioMunicipios, left_on='Municipio', right_on='municipio', how='left')
denuncias_MP_VCM = denuncias_MP_VCM.drop(columns=['municipio','Municipio','Mes','Año'])
denuncias_MP_VCM = denuncias_MP_VCM.rename(columns={'id': 'id_municipio'})
denuncias_MP_VCM['Edad'] = denuncias_MP_VCM['Edad'].apply(normalizar_edad).astype('Int64')
denuncias_MP_VCM

,Fecha,Dia,Edad,Rangos de edad,Escolaridad,Pueblo de Pertenencia,Orientacion,Estado Caso,Valor,id_departamento,id_municipio
0,2008-06-01,domingo,73,de 61 años a más,No registrado,Sin selección,Sin selección,Sobreseido,1,3,1601.0
1,2008-06-02,lunes,20,de 11 a 20 años,No registrado,Sin selección,Sin selección,Sobreseido,1,3,1601.0
2,2008-06-02,lunes,28,de 21 a 30 años,No registrado,Sin selección,Sin selección,Desestimado,1,3,1601.0
3,2008-06-02,lunes,37,de 31 a 40 años,No registrado,Sin selección,Sin selección,Archivado,1,3,1601.0
4,2008-06-03,martes,<NA>,SD,No registrado,Sin selección,Sin selección,Desestimado,1,3,1601.0
...,...,...,...,...,...,...,...,...,...,...,...
672667,2024-01-25,jueves,48,de 41 a 50 años,No registrado,Ladino,Sin selección,Desestimado,1,22,19.0
672668,2024-01-25,jueves,22,de 21 a 30 años,No registrado,Ladino,Sin selección,Desestimado,1,22,19.0
672669,2024-01-25,jueves,40,de 31 a 40 años,No registrado,Ladino,Sin selección,Desestimado,1,22,19.0
672670,2024-01-25,jueves,<NA>,SD,No registrado,Ladino,Sin selección,Desestimado,1,22,19.0


In [127]:
denuncias_MP_VCM['id_municipio'] = denuncias_MP_VCM['id_municipio'].astype('Int64')
denuncias_MP_VCM = denuncias_MP_VCM.drop(columns=['Rangos de edad'])
denuncias_MP_VCM['id_denuncias'] = range(1, len(denuncias_MP_VCM) + 1)
denuncias_MP_VCM['Dia'] = denuncias_MP_VCM['Dia'].apply(normalizar_texto)
denuncias_MP_VCM

,Fecha,Dia,Edad,Escolaridad,Pueblo de Pertenencia,Orientacion,Estado Caso,Valor,id_departamento,id_municipio,id_denuncias
0,2008-06-01,Domingo,73,No registrado,Sin selección,Sin selección,Sobreseido,1,3,1601,1
1,2008-06-02,Lunes,20,No registrado,Sin selección,Sin selección,Sobreseido,1,3,1601,2
2,2008-06-02,Lunes,28,No registrado,Sin selección,Sin selección,Desestimado,1,3,1601,3
3,2008-06-02,Lunes,37,No registrado,Sin selección,Sin selección,Archivado,1,3,1601,4
4,2008-06-03,Martes,<NA>,No registrado,Sin selección,Sin selección,Desestimado,1,3,1601,5
...,...,...,...,...,...,...,...,...,...,...,...
672667,2024-01-25,Jueves,48,No registrado,Ladino,Sin selección,Desestimado,1,22,19,672668
672668,2024-01-25,Jueves,22,No registrado,Ladino,Sin selección,Desestimado,1,22,19,672669
672669,2024-01-25,Jueves,40,No registrado,Ladino,Sin selección,Desestimado,1,22,19,672670
672670,2024-01-25,Jueves,<NA>,No registrado,Ladino,Sin selección,Desestimado,1,22,19,672671


In [128]:
denuncias_MP_VCM['Escolaridad'] = denuncias_MP_VCM['Escolaridad'].apply(normalizar_texto)
denuncias_MP_VCM['Pueblo de Pertenencia'] = denuncias_MP_VCM['Pueblo de Pertenencia'].apply(normalizar_texto)
denuncias_MP_VCM['Orientacion'] = denuncias_MP_VCM['Orientacion'].apply(normalizar_texto)
escolaridad_dict = {v: i + 1 for i, v in enumerate(sorted(denuncias_MP_VCM['Escolaridad'].dropna().unique()))}
pueblo_dict = {v: i + 1 for i, v in enumerate(sorted(denuncias_MP_VCM['Pueblo de Pertenencia'].dropna().unique()))}
orientacion_dict = {v: i + 1 for i, v in enumerate(sorted(denuncias_MP_VCM['Orientacion'].dropna().unique()))}

denuncias_MP_VCM['id_escolaridad'] = denuncias_MP_VCM['Escolaridad'].map(escolaridad_dict)
denuncias_MP_VCM['id_pueblo'] = denuncias_MP_VCM['Pueblo de Pertenencia'].map(pueblo_dict)
denuncias_MP_VCM['id_orientacion'] = denuncias_MP_VCM['Orientacion'].map(orientacion_dict)

denuncias_MP_VCM = denuncias_MP_VCM.drop(columns=['Escolaridad','Pueblo de Pertenencia','Orientacion'])
denuncias_MP_VCM

,Fecha,Dia,Edad,Estado Caso,Valor,id_departamento,id_municipio,id_denuncias,id_escolaridad,id_pueblo,id_orientacion
0,2008-06-01,Domingo,73,Sobreseido,1,3,1601,1,27,9,4
1,2008-06-02,Lunes,20,Sobreseido,1,3,1601,2,27,9,4
2,2008-06-02,Lunes,28,Desestimado,1,3,1601,3,27,9,4
3,2008-06-02,Lunes,37,Archivado,1,3,1601,4,27,9,4
4,2008-06-03,Martes,<NA>,Desestimado,1,3,1601,5,27,9,4
...,...,...,...,...,...,...,...,...,...,...,...
672667,2024-01-25,Jueves,48,Desestimado,1,22,19,672668,27,3,4
672668,2024-01-25,Jueves,22,Desestimado,1,22,19,672669,27,3,4
672669,2024-01-25,Jueves,40,Desestimado,1,22,19,672670,27,3,4
672670,2024-01-25,Jueves,<NA>,Desestimado,1,22,19,672671,27,3,4


In [129]:
denuncias_MP_VCM['Estado Caso'] = denuncias_MP_VCM['Estado Caso'].apply(normalizar_texto)
escolaridad_dict = {v: i + 1 for i, v in enumerate(sorted(denuncias_MP_VCM['Estado Caso'].dropna().unique()))}
denuncias_MP_VCM['id_estado_caso'] = denuncias_MP_VCM['Estado Caso'].map(escolaridad_dict)
denuncias_MP_VCM = denuncias_MP_VCM.drop(columns=['Estado Caso'])


In [130]:
df_escolaridad = pd.DataFrame(list(escolaridad_dict.items()), columns=['Escolaridad', 'id_escolaridad'])
df_pueblo = pd.DataFrame(list(pueblo_dict.items()), columns=['Pueblo_de_Pertenencia', 'id_pueblo'])
df_orientacion = pd.DataFrame(list(orientacion_dict.items()), columns=['Orientacion', 'id_orientacion'])
df_estado_caso = pd.DataFrame(list(estado_caso_dict.items()), columns=['Estado_del_Caso', 'id_estado_caso'])

df_escolaridad.to_csv('diccionario_escolaridad_MP_VCM.csv', index=False)
df_pueblo.to_csv('diccionario_pueblo_MP_VCM.csv', index=False)
df_orientacion.to_csv('diccionario_orientacion_MP_VCM.csv', index=False)
df_estado_caso.to_csv('diccionario_estado_caso_MP_VCM.csv', index=False)
denuncias_MP_VCM


,Fecha,Dia,Edad,Valor,id_departamento,id_municipio,id_denuncias,id_escolaridad,id_pueblo,id_orientacion,id_estado_caso
0,2008-06-01,Domingo,73,1,3,1601,1,27,9,4,31
1,2008-06-02,Lunes,20,1,3,1601,2,27,9,4,31
2,2008-06-02,Lunes,28,1,3,1601,3,27,9,4,15
3,2008-06-02,Lunes,37,1,3,1601,4,27,9,4,4
4,2008-06-03,Martes,<NA>,1,3,1601,5,27,9,4,15
...,...,...,...,...,...,...,...,...,...,...,...
672667,2024-01-25,Jueves,48,1,22,19,672668,27,3,4,15
672668,2024-01-25,Jueves,22,1,22,19,672669,27,3,4,15
672669,2024-01-25,Jueves,40,1,22,19,672670,27,3,4,15
672670,2024-01-25,Jueves,<NA>,1,22,19,672671,27,3,4,15


In [131]:
denundias_MP_VCM = denuncias_MP_VCM.drop(columns=['Dia'])
denundias_MP_VCM.to_csv('denundias_MP_VCM.csv', index=False)

In [132]:
denundias_MP_VCM.to_csv('denundias_MP_VCM.csv', index=False)

In [133]:
urlEvaluacionINACIF_VCM = '/content/Evaluciones realizadas por el INACIF 2008 al 2024.xlsx'
cargarEvaluacionINACIF_VCM = pd.read_excel(urlEvaluacionINACIF_VCM, sheet_name='Evaluciones 2008-2024', header=0)
evaluacionINACIF_VCM = pd.DataFrame(cargarEvaluacionINACIF_VCM)
evaluacionINACIF_VCM.head()
#

,FECHA,DÍA,AÑO,SEDE,MUNICIPIO,DEPARTAMENTO,Edad,TIPOS DE EVALUACIÓN,ORIENTACIÓN SEXUAL,VALOR
0,2008-05-03 00:00:00,sábado,2008,COBAN,COBÁN,ALTA VERAPAZ,19,ABUSO SEXUAL,SD,1
1,2008-05-04 00:00:00,domingo,2008,COBAN,COBÁN,ALTA VERAPAZ,16,ABUSO SEXUAL,SD,1
2,2008-05-12 00:00:00,lunes,2008,COBAN,COBÁN,ALTA VERAPAZ,15,ABUSO SEXUAL,SD,1
3,2008-05-12 00:00:00,lunes,2008,COBAN,COBÁN,ALTA VERAPAZ,23,ABUSO SEXUAL,SD,1
4,2008-05-16 00:00:00,viernes,2008,COBAN,COBÁN,ALTA VERAPAZ,25,ABUSO SEXUAL,SD,1


In [134]:
evaluacionINACIF_VCM['DEPARTAMENTO'] = evaluacionINACIF_VCM['DEPARTAMENTO'].apply(normalizar_texto)
evaluacionINACIF_VCM['id_departamento'] = evaluacionINACIF_VCM['DEPARTAMENTO'].map(departamento_map)
evaluacionINACIF_VCM = evaluacionINACIF_VCM.drop(columns=['DEPARTAMENTO','DÍA','AÑO'])

evaluacionINACIF_VCM['MUNICIPIO'] = evaluacionINACIF_VCM['MUNICIPIO'].apply(normalizar_texto)
evaluacionINACIF_VCM['TIPOS DE EVALUACIÓN'] = evaluacionINACIF_VCM['TIPOS DE EVALUACIÓN'].apply(normalizar_texto)
evaluacionINACIF_VCM['ORIENTACIÓN SEXUAL'] = evaluacionINACIF_VCM['ORIENTACIÓN SEXUAL'].apply(normalizar_texto)
evaluacionINACIF_VCM['SEDE'] = evaluacionINACIF_VCM['SEDE'].apply(normalizar_texto)

evaluacionINACIF_VCM.head()

,FECHA,SEDE,MUNICIPIO,Edad,TIPOS DE EVALUACIÓN,ORIENTACIÓN SEXUAL,VALOR,id_departamento
0,2008-05-03 00:00:00,Coban,Coban,19,Abuso Sexual,Sd,1,3.0
1,2008-05-04 00:00:00,Coban,Coban,16,Abuso Sexual,Sd,1,3.0
2,2008-05-12 00:00:00,Coban,Coban,15,Abuso Sexual,Sd,1,3.0
3,2008-05-12 00:00:00,Coban,Coban,23,Abuso Sexual,Sd,1,3.0
4,2008-05-16 00:00:00,Coban,Coban,25,Abuso Sexual,Sd,1,3.0


In [135]:
evaluacionINACIF_VCM = pd.merge(evaluacionINACIF_VCM, diccionarioMunicipios, left_on='MUNICIPIO', right_on='municipio', how='left')
evaluacionINACIF_VCM = evaluacionINACIF_VCM.drop(columns=['municipio','MUNICIPIO'])
evaluacionINACIF_VCM = evaluacionINACIF_VCM.rename(columns={'id': 'id_municipio'})
evaluacionINACIF_VCM.head()

,FECHA,SEDE,Edad,TIPOS DE EVALUACIÓN,ORIENTACIÓN SEXUAL,VALOR,id_departamento,id_municipio
0,2008-05-03 00:00:00,Coban,19,Abuso Sexual,Sd,1,3.0,1601.0
1,2008-05-04 00:00:00,Coban,16,Abuso Sexual,Sd,1,3.0,1601.0
2,2008-05-12 00:00:00,Coban,15,Abuso Sexual,Sd,1,3.0,1601.0
3,2008-05-12 00:00:00,Coban,23,Abuso Sexual,Sd,1,3.0,1601.0
4,2008-05-16 00:00:00,Coban,25,Abuso Sexual,Sd,1,3.0,1601.0


In [136]:
evaluacionINACIF_VCM['id_departamento'] = evaluacionINACIF_VCM['id_departamento'].astype('Int64')
evaluacionINACIF_VCM['id_municipio'] = evaluacionINACIF_VCM['id_municipio'].astype('Int64')
evaluacionINACIF_VCM['Edad '] = evaluacionINACIF_VCM['Edad '].apply(normalizar_edad).astype('Int64')
evaluacionINACIF_VCM = evaluacionINACIF_VCM.rename(columns={'ORIENTACIÓN SEXUAL':'orientacion_sexual','Edad ':'edad','VALOR':'valor'})
evaluacionINACIF_VCM.head()

,FECHA,SEDE,edad,TIPOS DE EVALUACIÓN,orientacion_sexual,valor,id_departamento,id_municipio
0,2008-05-03 00:00:00,Coban,19,Abuso Sexual,Sd,1,3,1601
1,2008-05-04 00:00:00,Coban,16,Abuso Sexual,Sd,1,3,1601
2,2008-05-12 00:00:00,Coban,15,Abuso Sexual,Sd,1,3,1601
3,2008-05-12 00:00:00,Coban,23,Abuso Sexual,Sd,1,3,1601
4,2008-05-16 00:00:00,Coban,25,Abuso Sexual,Sd,1,3,1601


In [137]:
sede_dict = {v: i + 1 for i, v in enumerate(sorted(evaluacionINACIF_VCM['SEDE'].dropna().unique()))}
tipo_eva_dict = {v: i + 1 for i, v in enumerate(sorted(evaluacionINACIF_VCM['TIPOS DE EVALUACIÓN'].dropna().unique()))}
tipo_or_sex = {v: i + 1 for i, v in enumerate(sorted(evaluacionINACIF_VCM['orientacion_sexual'].dropna().unique()))}
tipo_or_sex

{'Bisexual': 1,
 'Heterosexual': 2,
 'Intersexual': 3,
 'Lesbiana': 4,
 'Sd': 5,
 'Transexual': 6}

In [138]:
evaluacionINACIF_VCM['id_sede'] = evaluacionINACIF_VCM['SEDE'].map(sede_dict)
evaluacionINACIF_VCM['id_tipo_evaluacion'] = evaluacionINACIF_VCM['TIPOS DE EVALUACIÓN'].map(tipo_eva_dict)
evaluacionINACIF_VCM = evaluacionINACIF_VCM.drop(columns=['SEDE','TIPOS DE EVALUACIÓN'])
evaluacionINACIF_VCM['id_orientacion_sexual'] = evaluacionINACIF_VCM['orientacion_sexual'].map(tipo_or_sex)
evaluacionINACIF_VCM = evaluacionINACIF_VCM.drop(columns=['orientacion_sexual'])
evaluacionINACIF_VCM.head()

,FECHA,edad,valor,id_departamento,id_municipio,id_sede,id_tipo_evaluacion,id_orientacion_sexual
0,2008-05-03 00:00:00,19,1,3,1601,8,1,5
1,2008-05-04 00:00:00,16,1,3,1601,8,1,5
2,2008-05-12 00:00:00,15,1,3,1601,8,1,5
3,2008-05-12 00:00:00,23,1,3,1601,8,1,5
4,2008-05-16 00:00:00,25,1,3,1601,8,1,5


In [139]:
df_sede_INACIF = pd.DataFrame(list(sede_dict.items()), columns=['sede', 'id_sede'])
df_tipo_eva_INACIF = pd.DataFrame(list(tipo_eva_dict.items()), columns=['tipo_evaluacion', 'id_tipo_evaluacion'])
df_orientacion_sexual_INACIF = pd.DataFrame(list(tipo_or_sex.items()), columns=['orientacion_sexual', 'id_orientacion_sexual'])
evaluacionINACIF_VCM['id'] = range(1, len(evaluacionINACIF_VCM) + 1)

df_sede_INACIF.to_csv('diccionario_sede_INACIF.csv', index=False)
df_tipo_eva_INACIF.to_csv('diccionario_tipo_evaluacion_INACIF.csv', index=False)
df_orientacion_sexual_INACIF.to_csv('diccionario_orientacion_sexual_INACIF.csv', index=False)
evaluacionINACIF_VCM.to_csv('evaluacionINACIF_VCM.csv', index=False)
#

In [140]:
urlDic = '/content/CASOS DISCRIMINACIÓN 2016-2023.xls'
cargarDic = pd.read_excel(urlDic, sheet_name='Hoja9', header=0)
disc1 = pd.DataFrame(cargarDic)
disc1.head()
#

,H,M,edad,dep,tipo,pueblo,com_linguistica,aneo
0,X,NaN,41,Guatemala,Étnica,Maya,K'iche',2016
1,X,NaN,59,Alta Verapaz,Étnica,Maya,Q'eqchi',2016
2,NaN,X,30,Guatemala,Étnica,Maya,K'iche',2016
3,NaN,NaN,No Indica,Alta Verapaz,Étnica,Maya,Q'eqchi',2016
4,NaN,X,53,Guatemala,Étnica,Maya,Q'eqchi',2016


In [141]:
disc1['edad'] = disc1['edad'].apply(normalizar_edad).astype('Int64')
disc1['dep'] = disc1['dep'].apply(normalizar_texto)
disc1['tipo'] = disc1['tipo'].apply(normalizar_texto)
disc1['pueblo'] = disc1['pueblo'].apply(normalizar_texto)
disc1['com_linguistica'] = disc1['com_linguistica'].apply(normalizar_texto)
disc1.head()


,H,M,edad,dep,tipo,pueblo,com_linguistica,aneo
0,X,NaN,41,Guatemala,Etnica,Maya,K'Iche',2016
1,X,NaN,59,Alta Verapaz,Etnica,Maya,Q'Eqchi',2016
2,NaN,X,30,Guatemala,Etnica,Maya,K'Iche',2016
3,NaN,NaN,<NA>,Alta Verapaz,Etnica,Maya,Q'Eqchi',2016
4,NaN,X,53,Guatemala,Etnica,Maya,Q'Eqchi',2016


In [142]:
def obtener_sexo(fila):
    if str(fila['H']).strip().upper() == 'X':
        return 1  # Hombre
    elif str(fila['M']).strip().upper() == 'X':
        return 2  # Mujer
    else:
        return None  # No definido

disc1['sexo'] = disc1.apply(obtener_sexo, axis=1)
disc1.head()

,H,M,edad,dep,tipo,pueblo,com_linguistica,aneo,sexo
0,X,NaN,41,Guatemala,Etnica,Maya,K'Iche',2016,1.0
1,X,NaN,59,Alta Verapaz,Etnica,Maya,Q'Eqchi',2016,1.0
2,NaN,X,30,Guatemala,Etnica,Maya,K'Iche',2016,2.0
3,NaN,NaN,<NA>,Alta Verapaz,Etnica,Maya,Q'Eqchi',2016,NaN
4,NaN,X,53,Guatemala,Etnica,Maya,Q'Eqchi',2016,2.0


In [143]:
disc1 = disc1.drop(columns=['H','M'])
disc1['sexo'] = disc1['sexo'].astype('Int64')
disc1.head()


,edad,dep,tipo,pueblo,com_linguistica,aneo,sexo
0,41,Guatemala,Etnica,Maya,K'Iche',2016,1
1,59,Alta Verapaz,Etnica,Maya,Q'Eqchi',2016,1
2,30,Guatemala,Etnica,Maya,K'Iche',2016,2
3,<NA>,Alta Verapaz,Etnica,Maya,Q'Eqchi',2016,<NA>
4,53,Guatemala,Etnica,Maya,Q'Eqchi',2016,2


In [144]:
cargadic2 = pd.read_excel(urlDic, sheet_name='Hoja10', header=0)
disc2 = pd.DataFrame(cargadic2)
disc2.head()
#

,m,h,edad,maya,garifuna,xinka,Comunidad Lingüística,departamento,Tipo de discriminación,aneo
0,NaN,X,30.0,X,NaN,NaN,K'iche',Guatemala,Étnica,2020
1,X,NaN,42.0,x,NaN,NaN,K'iche',Totonicapán,Étnica,2020
2,X,NaN,20.0,NaN,X,NaN,K'iche',Guatemala,Racial y Étnica,2020
3,NaN,X,28.0,NaN,X,NaN,K'iche',Guatemala,Racial y Étnica,2020
4,NaN,X,52.0,x,NaN,NaN,Q'anjob'al,Huehuetenango,Étnica,2020


In [145]:
def determinar_sexo(row):
    if str(row['h']).strip().lower() == 'x':
        return 1  # Hombre
    elif str(row['m']).strip().lower() == 'x':
        return 2  # Mujer
    else:
        return None

disc2['sexo'] = disc2.apply(determinar_sexo, axis=1)
def determinar_pueblo(row):
    for pueblo in ['maya', 'garifuna', 'xinka']:
        if str(row[pueblo]).strip().lower() == 'x':
            return pueblo.capitalize()
    return None

disc2['pueblo'] = disc2.apply(determinar_pueblo, axis=1)
disc2 = disc2.drop(columns=['h', 'm', 'maya', 'garifuna', 'xinka'])
disc2['edad'] = disc2['edad'].apply(normalizar_edad).astype('Int64')
disc2

,edad,Comunidad Lingüística,departamento,Tipo de discriminación,aneo,sexo,pueblo
0,30,K'iche',Guatemala,Étnica,2020,1,Maya
1,42,K'iche',Totonicapán,Étnica,2020,2,Maya
2,20,K'iche',Guatemala,Racial y Étnica,2020,2,Garifuna
3,28,K'iche',Guatemala,Racial y Étnica,2020,1,Garifuna
4,52,Q'anjob'al,Huehuetenango,Étnica,2020,1,Maya
5,40,K'iche',Quetzaltenango,Étnica,2020,1,Maya
6,22,Mam,San Marcos,Étnica,2020,2,Maya
7,46,Kaqchikel,Sololá,Discriminación,2021,2,Maya
8,30,Kaqchikel,Guatemala,Discriminación,2021,2,Maya
9,15,Q'eqchi',Alta Verapaz,Discriminación,2021,1,Maya


In [146]:
disc2['Comunidad Lingüística'] = disc2['Comunidad Lingüística'].apply(normalizar_texto)
disc2['departamento'] = disc2['departamento'].apply(normalizar_texto)
disc2['Tipo de discriminación '] = disc2['Tipo de discriminación '].apply(normalizar_texto)
disc2 = disc2.rename(columns={'departamento': 'dep', 'Tipo de discriminación ': 'tipo','Comunidad Lingüística':'com_linguistica'})
disc2.head()

,edad,com_linguistica,dep,tipo,aneo,sexo,pueblo
0,30,K'Iche',Guatemala,Etnica,2020,1,Maya
1,42,K'Iche',Totonicapan,Etnica,2020,2,Maya
2,20,K'Iche',Guatemala,Racial Y Etnica,2020,2,Garifuna
3,28,K'Iche',Guatemala,Racial Y Etnica,2020,1,Garifuna
4,52,Q'Anjob'Al,Huehuetenango,Etnica,2020,1,Maya


In [147]:
orden_columnas = ['edad', 'dep', 'tipo', 'pueblo', 'com_linguistica', 'aneo', 'sexo']
disc2 = disc2[orden_columnas]
disc2.head()

,edad,dep,tipo,pueblo,com_linguistica,aneo,sexo
0,30,Guatemala,Etnica,Maya,K'Iche',2020,1
1,42,Totonicapan,Etnica,Maya,K'Iche',2020,2
2,20,Guatemala,Racial Y Etnica,Garifuna,K'Iche',2020,2
3,28,Guatemala,Racial Y Etnica,Garifuna,K'Iche',2020,1
4,52,Huehuetenango,Etnica,Maya,Q'Anjob'Al,2020,1


In [148]:
df_final = pd.concat([disc1, disc2], ignore_index=True)
df_final.head()

,edad,dep,tipo,pueblo,com_linguistica,aneo,sexo
0,41,Guatemala,Etnica,Maya,K'Iche',2016,1
1,59,Alta Verapaz,Etnica,Maya,Q'Eqchi',2016,1
2,30,Guatemala,Etnica,Maya,K'Iche',2016,2
3,<NA>,Alta Verapaz,Etnica,Maya,Q'Eqchi',2016,<NA>
4,53,Guatemala,Etnica,Maya,Q'Eqchi',2016,2


In [149]:
df_final

,edad,dep,tipo,pueblo,com_linguistica,aneo,sexo
0,41,Guatemala,Etnica,Maya,K'Iche',2016,1
1,59,Alta Verapaz,Etnica,Maya,Q'Eqchi',2016,1
2,30,Guatemala,Etnica,Maya,K'Iche',2016,2
3,<NA>,Alta Verapaz,Etnica,Maya,Q'Eqchi',2016,<NA>
4,53,Guatemala,Etnica,Maya,Q'Eqchi',2016,2
...,...,...,...,...,...,...,...
108,34,Huehuetenango,Discriminacion Etnica,Maya,Jakalteka,2022,2
109,23,Huehuetenango,Discriminacion,Maya,Q'Anjob'Al,2022,2
110,49,Quetzaltenango,Discriminacion Etnica,Xinka,Nahuatl,2022,1
111,40,Guatemala,Etnica Laboral,Maya,K'Iche',2023,2


In [150]:
df_final['dep'] = df_final['dep'].apply(normalizar_texto)
df_final['tipo'] = df_final['tipo'].apply(normalizar_texto)
df_final['pueblo'] = df_final['pueblo'].apply(normalizar_texto)
df_final['com_linguistica'] = df_final['com_linguistica'].apply(normalizar_texto)
df_final['id_departamento'] = df_final['dep'].map(departamento_map)
df_final = df_final.drop(columns=['dep'])
df_final


,edad,tipo,pueblo,com_linguistica,aneo,sexo,id_departamento
0,41,Etnica,Maya,K'Iche',2016,1,1.0
1,59,Etnica,Maya,Q'Eqchi',2016,1,3.0
2,30,Etnica,Maya,K'Iche',2016,2,1.0
3,<NA>,Etnica,Maya,Q'Eqchi',2016,<NA>,3.0
4,53,Etnica,Maya,Q'Eqchi',2016,2,1.0
...,...,...,...,...,...,...,...
108,34,Discriminacion Etnica,Maya,Jakalteka,2022,2,16.0
109,23,Discriminacion,Maya,Q'Anjob'Al,2022,2,16.0
110,49,Discriminacion Etnica,Xinka,Nahuatl,2022,1,20.0
111,40,Etnica Laboral,Maya,K'Iche',2023,2,1.0


In [151]:
dic_pueblo_disc = {v: i + 1 for i, v in enumerate(sorted(df_final['pueblo'].dropna().unique()))}
dic_tipo_disc = {v: i + 1 for i, v in enumerate(sorted(df_final['tipo'].dropna().unique()))}
dic_com_lin_disc = {v: i + 1 for i, v in enumerate(sorted(df_final['com_linguistica'].dropna().unique()))}

print(dic_pueblo_disc)
print(dic_tipo_disc)
print(dic_com_lin_disc)
#

{'Garifuna': 1, "K'Iche'": 2, 'Maya': 3, 'No Indica': 4, "Q'Qechi'": 5, 'Xinca': 6, 'Xinka': 7}
{'Discriminacion': 1, 'Discriminacion Etnica': 2, 'Etnica': 3, 'Etnica Laboral': 4, 'Etnica Racial': 5, 'Etnica/ Genero': 6, 'Racial': 7, 'Racial Y Etnica': 8}
{'*  Idioma Garifuna': 1, 'Chorti': 2, 'Garifuna': 3, 'Ixil': 4, 'Jakalteka': 5, "K'Iche": 6, "K'Iche'": 7, 'Kaqchikel': 8, "Ki'Che'": 9, 'Mam': 10, 'Nahuatl': 11, 'No Indica': 12, "Poqomch'": 13, "Q'Anjob'Al": 14, "Q'Eqchi'": 15, "Q'Qechi'": 16, 'Q`Eqchi`': 17, "Tz'Utujil": 18, 'Xinca': 19}


In [152]:
df_final['id_pueblo'] = df_final['pueblo'].map(dic_pueblo_disc).astype('Int64')
df_final['id_tipo_discriminacion'] = df_final['tipo'].map(dic_tipo_disc).astype('Int64')
df_final['id_com_linguistica'] = df_final['com_linguistica'].map(dic_com_lin_disc).astype('Int64')
df_final = df_final.drop(columns=['pueblo','tipo','com_linguistica'])
df_final['id_departamento'] = df_final['id_departamento'].astype('Int64')
df_final['id'] = range(1, len(df_final) + 1)
df_final.head()

,edad,aneo,sexo,id_departamento,id_pueblo,id_tipo_discriminacion,id_com_linguistica,id
0,41,2016,1,1,3,3,7,1
1,59,2016,1,3,3,3,15,2
2,30,2016,2,1,3,3,7,3
3,<NA>,2016,<NA>,3,3,3,15,4
4,53,2016,2,1,3,3,15,5


In [153]:
df_dic_pueblo = pd.DataFrame(list(dic_pueblo_disc.items()), columns=['pueblo', 'id_pueblo'])
df_dic_tipo = pd.DataFrame(list(dic_tipo_disc.items()), columns=['tipo', 'id_tipo_discriminacion'])
df_dic_com_lin = pd.DataFrame(list(dic_com_lin_disc.items()), columns=['com_linguistica', 'id_com_linguistica'])

df_dic_pueblo.to_csv('diccionario_pueblo_disc.csv', index=False)
df_dic_tipo.to_csv('diccionario_tipo_disc.csv', index=False)
df_dic_com_lin.to_csv('diccionario_com_linguistica.csv', index=False)
df_final.to_csv('discriminacion.csv', index=False)


In [220]:
urlViolenciaIntraFamiliar = '/content/violenciaIntrafamiliar.xlsx'
urlVIF = pd.read_excel(urlViolenciaIntraFamiliar, sheet_name='Base VIF 2023', header=0)
violenciaIntrafamiliar = pd.DataFrame(urlVIF)
violenciaIntrafamiliar.head()

,HEC_DIA,HEC_MES,HEC_ANO,HEC_DEPTO,departamento_hecho,HEC_DEPTOMCPIO,municipio_hecho,HEC_TIPAGRE,hecho_tipoAgre,NUMERO_BOLETA,...,AGR_OTRAS_N_AS,INST_DENUN_HECHO,institucion_denuncia.1,ORGANISMO_JURISDICCIONAL,organismo_juri,CONDUCENTE,LEY_APLICABLE,ley_aplicable,ORGANISMO_REMITE,organismo_remite
0,22,7,2023,16,Alta Verapaz,1601,Cobán,1122,Física - psicológica,1072,...,0.0,3,Policía Nacional Civil,0,Ignoarado,NaN,9,Ignorado,99,Ignorado
1,19,9,2023,1,Guatemala,115,Villa Nueva,2212,Sexual,558,...,0.0,4,Organismo Judicial,15,Juzgados de Primera Instancia de Familia,True,1,Ley VIF ó Decreto 97-96,99,Ignorado
2,7,9,2023,9,Quetzaltenango,905,Sibilia,1222,Física,74,...,NaN,4,Organismo Judicial,4,Juzgado de Paz Penal de Conocimiento a Prevenc...,True,1,Ley VIF ó Decreto 97-96,99,Ignorado
3,25,9,2023,4,Chimaltenango,413,San Andrés Itzapa,2121,Psicológica - patrimonial,164,...,0.0,3,Policía Nacional Civil,0,Ignoarado,NaN,9,Ignorado,99,Ignorado
4,20,1,2023,22,Jutiapa,2201,Jutiapa,1222,Física,21,...,NaN,3,Policía Nacional Civil,0,Ignoarado,NaN,9,Ignorado,99,Ignorado


In [221]:
def convertir_fecha_dia_num(mes_num, anio, dia):
    try:
        # 1) Verificar valores nulos
        if pd.isna(mes_num) or pd.isna(anio) or pd.isna(dia):
            return None

        # 2) Convertir a enteros
        mes_num = int(mes_num)
        anio = int(anio)
        dia = int(dia)

        # 3) Validar rangos
        if mes_num < 1 or mes_num > 12:
            return None
        if anio < 1900 or anio > 2100:
            return None
        if dia < 1 or dia > 31:
            return None

        # 4) Construir y retornar la fecha
        return datetime(anio, mes_num, dia).strftime('%Y-%m-%d')

    except Exception:
        return None

In [222]:
#fJudiciales['fecha'] = fJudiciales.apply(
#    lambda r: convertir_fecha(r['Mes_boleta'], r['Año_boleta']),
#    axis=1
#)
violenciaIntrafamiliar['fecha_hecho'] = violenciaIntrafamiliar.apply(
    lambda r: convertir_fecha_dia_num(r['HEC_MES'], r['HEC_ANO'],r['HEC_DIA']),
    axis=1
)
violenciaIntrafamiliar = violenciaIntrafamiliar.drop(columns=['HEC_MES','HEC_ANO','HEC_DIA','HEC_DEPTO','HEC_DEPTOMCPIO','HEC_TIPAGRE'])
violenciaIntrafamiliar.head()

,departamento_hecho,municipio_hecho,hecho_tipoAgre,NUMERO_BOLETA,DIA_EMISION,MES_EMISION,ANO_EMISION,DEPTO,depto_emision,DEPTO_MCPIO,...,INST_DENUN_HECHO,institucion_denuncia.1,ORGANISMO_JURISDICCIONAL,organismo_juri,CONDUCENTE,LEY_APLICABLE,ley_aplicable,ORGANISMO_REMITE,organismo_remite,fecha_hecho
0,Alta Verapaz,Cobán,Física - psicológica,1072,24,7,2023,16,Alta Verapaz,1601,...,3,Policía Nacional Civil,0,Ignoarado,NaN,9,Ignorado,99,Ignorado,2023-07-22
1,Guatemala,Villa Nueva,Sexual,558,29,9,2023,1,Guatemala,108,...,4,Organismo Judicial,15,Juzgados de Primera Instancia de Familia,True,1,Ley VIF ó Decreto 97-96,99,Ignorado,2023-09-19
2,Quetzaltenango,Sibilia,Física,74,11,9,2023,9,Quetzaltenango,905,...,4,Organismo Judicial,4,Juzgado de Paz Penal de Conocimiento a Prevenc...,True,1,Ley VIF ó Decreto 97-96,99,Ignorado,2023-09-07
3,Chimaltenango,San Andrés Itzapa,Psicológica - patrimonial,164,25,9,2023,4,Chimaltenango,413,...,3,Policía Nacional Civil,0,Ignoarado,NaN,9,Ignorado,99,Ignorado,2023-09-25
4,Jutiapa,Jutiapa,Física,21,20,1,2023,22,Jutiapa,2201,...,3,Policía Nacional Civil,0,Ignoarado,NaN,9,Ignorado,99,Ignorado,2023-01-20


In [223]:
violenciaIntrafamiliar['fecha_emision'] = violenciaIntrafamiliar.apply(
    lambda r: convertir_fecha_dia_num(r['MES_EMISION'], r['ANO_EMISION'],r['DIA_EMISION']),
    axis=1
)
violenciaIntrafamiliar = violenciaIntrafamiliar.drop(columns=['MES_EMISION','ANO_EMISION','DIA_EMISION','DEPTO','DEPTO_MCPIO'])
violenciaIntrafamiliar.head()

,departamento_hecho,municipio_hecho,hecho_tipoAgre,NUMERO_BOLETA,depto_emision,muni_emision,QUIEN_REPORTA,quien_reporta,VIC_SEXO,sexo_vic,...,institucion_denuncia.1,ORGANISMO_JURISDICCIONAL,organismo_juri,CONDUCENTE,LEY_APLICABLE,ley_aplicable,ORGANISMO_REMITE,organismo_remite,fecha_hecho,fecha_emision
0,Alta Verapaz,Cobán,Física - psicológica,1072,Alta Verapaz,Cobán,2,Familiar de la víctima,2,Mujer,...,Policía Nacional Civil,0,Ignoarado,NaN,9,Ignorado,99,Ignorado,2023-07-22,2023-07-24
1,Guatemala,Villa Nueva,Sexual,558,Guatemala,Mixco,2,Familiar de la víctima,2,Mujer,...,Organismo Judicial,15,Juzgados de Primera Instancia de Familia,True,1,Ley VIF ó Decreto 97-96,99,Ignorado,2023-09-19,2023-09-29
2,Quetzaltenango,Sibilia,Física,74,Quetzaltenango,Sibilia,1,Víctima,1,Hombre,...,Organismo Judicial,4,Juzgado de Paz Penal de Conocimiento a Prevenc...,True,1,Ley VIF ó Decreto 97-96,99,Ignorado,2023-09-07,2023-09-11
3,Chimaltenango,San Andrés Itzapa,Psicológica - patrimonial,164,Chimaltenango,San Andrés Itzapa,1,Víctima,2,Mujer,...,Policía Nacional Civil,0,Ignoarado,NaN,9,Ignorado,99,Ignorado,2023-09-25,2023-09-25
4,Jutiapa,Jutiapa,Física,21,Jutiapa,Jutiapa,1,Víctima,2,Mujer,...,Policía Nacional Civil,0,Ignoarado,NaN,9,Ignorado,99,Ignorado,2023-01-20,2023-01-20


In [224]:
violenciaIntrafamiliar['id_VIF'] = range(1, len(violenciaIntrafamiliar) + 1)
violenciaIntrafamiliar.head()

,departamento_hecho,municipio_hecho,hecho_tipoAgre,NUMERO_BOLETA,depto_emision,muni_emision,QUIEN_REPORTA,quien_reporta,VIC_SEXO,sexo_vic,...,ORGANISMO_JURISDICCIONAL,organismo_juri,CONDUCENTE,LEY_APLICABLE,ley_aplicable,ORGANISMO_REMITE,organismo_remite,fecha_hecho,fecha_emision,id_VIF
0,Alta Verapaz,Cobán,Física - psicológica,1072,Alta Verapaz,Cobán,2,Familiar de la víctima,2,Mujer,...,0,Ignoarado,NaN,9,Ignorado,99,Ignorado,2023-07-22,2023-07-24,1
1,Guatemala,Villa Nueva,Sexual,558,Guatemala,Mixco,2,Familiar de la víctima,2,Mujer,...,15,Juzgados de Primera Instancia de Familia,True,1,Ley VIF ó Decreto 97-96,99,Ignorado,2023-09-19,2023-09-29,2
2,Quetzaltenango,Sibilia,Física,74,Quetzaltenango,Sibilia,1,Víctima,1,Hombre,...,4,Juzgado de Paz Penal de Conocimiento a Prevenc...,True,1,Ley VIF ó Decreto 97-96,99,Ignorado,2023-09-07,2023-09-11,3
3,Chimaltenango,San Andrés Itzapa,Psicológica - patrimonial,164,Chimaltenango,San Andrés Itzapa,1,Víctima,2,Mujer,...,0,Ignoarado,NaN,9,Ignorado,99,Ignorado,2023-09-25,2023-09-25,4
4,Jutiapa,Jutiapa,Física,21,Jutiapa,Jutiapa,1,Víctima,2,Mujer,...,0,Ignoarado,NaN,9,Ignorado,99,Ignorado,2023-01-20,2023-01-20,5


In [225]:
violenciaIntrafamiliar = violenciaIntrafamiliar.drop(columns=['QUIEN_REPORTA','VIC_SEXO','ORGANISMO_JURISDICCIONAL','LEY_APLICABLE','ORGANISMO_REMITE'])
violenciaIntrafamiliar['departamento_hecho'] = violenciaIntrafamiliar['departamento_hecho'].apply(normalizar_texto)
violenciaIntrafamiliar['id_departamento_hecho'] = violenciaIntrafamiliar['departamento_hecho'].map(departamento_map).astype('Int64')
violenciaIntrafamiliar = violenciaIntrafamiliar.drop(columns=['departamento_hecho'])
violenciaIntrafamiliar['municipio_hecho'] = violenciaIntrafamiliar['municipio_hecho'].apply(normalizar_texto)
violenciaIntrafamiliar = pd.merge(violenciaIntrafamiliar, diccionarioMunicipios, left_on='municipio_hecho', right_on='municipio', how='left')
violenciaIntrafamiliar = violenciaIntrafamiliar.drop(columns=['municipio_hecho','municipio'])
violenciaIntrafamiliar = violenciaIntrafamiliar.rename(columns={'id': 'id_municipio_hecho'})
violenciaIntrafamiliar['hecho_tipoAgre'] = violenciaIntrafamiliar['hecho_tipoAgre'].apply(normalizar_texto)
violenciaIntrafamiliar['depto_emision'] = violenciaIntrafamiliar['depto_emision'].apply(normalizar_texto)
violenciaIntrafamiliar['id_departamento_emision'] = violenciaIntrafamiliar['depto_emision'].map(departamento_map).astype('Int64')
violenciaIntrafamiliar['muni_emision'] = violenciaIntrafamiliar['muni_emision'].apply(normalizar_texto)
violenciaIntrafamiliar = pd.merge(violenciaIntrafamiliar,diccionarioMunicipios, left_on='muni_emision',right_on='municipio', how='left')
violenciaIntrafamiliar = violenciaIntrafamiliar.drop(columns=['municipio','muni_emision'])
violenciaIntrafamiliar = violenciaIntrafamiliar.rename(columns={'id':'id_municipio_emision'})
violenciaIntrafamiliar.head()

,hecho_tipoAgre,NUMERO_BOLETA,depto_emision,quien_reporta,sexo_vic,VIC_EDAD,TOTAL_HIJOS,NUM_HIJ_HOM,NUM_HIJ_MUJ,VIC_ALFAB,...,CONDUCENTE,ley_aplicable,organismo_remite,fecha_hecho,fecha_emision,id_VIF,id_departamento_hecho,id_municipio_hecho,id_departamento_emision,id_municipio_emision
0,Fisica - Psicologica,1072,Alta Verapaz,Familiar de la víctima,Mujer,14,0.0,0.0,0.0,1.0,...,NaN,Ignorado,Ignorado,2023-07-22,2023-07-24,1,3,1601.0,3,1601.0
1,Sexual,558,Guatemala,Familiar de la víctima,Mujer,23,1.0,0.0,1.0,1.0,...,True,Ley VIF ó Decreto 97-96,Ignorado,2023-09-19,2023-09-29,2,1,115.0,1,108.0
2,Fisica,74,Quetzaltenango,Víctima,Hombre,30,3.0,2.0,1.0,1.0,...,True,Ley VIF ó Decreto 97-96,Ignorado,2023-09-07,2023-09-11,3,20,905.0,20,905.0
3,Psicologica - Patrimonial,164,Chimaltenango,Víctima,Mujer,44,1.0,0.0,1.0,1.0,...,NaN,Ignorado,Ignorado,2023-09-25,2023-09-25,4,15,413.0,15,413.0
4,Fisica,21,Jutiapa,Víctima,Mujer,36,99.0,99.0,99.0,1.0,...,NaN,Ignorado,Ignorado,2023-01-20,2023-01-20,5,4,22.0,4,22.0


In [226]:
violenciaIntrafamiliar = violenciaIntrafamiliar.drop(columns=['depto_emision','VIC_ALFAB','VIC_ESCOLARIDAD','VIC_EST_CIV','VIC_GRUPET','VIC_NACIONAL'])
violenciaIntrafamiliar.head()

,hecho_tipoAgre,NUMERO_BOLETA,quien_reporta,sexo_vic,VIC_EDAD,TOTAL_HIJOS,NUM_HIJ_HOM,NUM_HIJ_MUJ,alfabetismoVictima,escolaridad_vict,...,CONDUCENTE,ley_aplicable,organismo_remite,fecha_hecho,fecha_emision,id_VIF,id_departamento_hecho,id_municipio_hecho,id_departamento_emision,id_municipio_emision
0,Fisica - Psicologica,1072,Familiar de la víctima,Mujer,14,0.0,0.0,0.0,Alfabeta,Sexto Primaria,...,NaN,Ignorado,Ignorado,2023-07-22,2023-07-24,1,3,1601.0,3,1601.0
1,Sexual,558,Familiar de la víctima,Mujer,23,1.0,0.0,1.0,Alfabeta,Diversificado(Grado Ignorado),...,True,Ley VIF ó Decreto 97-96,Ignorado,2023-09-19,2023-09-29,2,1,115.0,1,108.0
2,Fisica,74,Víctima,Hombre,30,3.0,2.0,1.0,Alfabeta,Primaria (grado ignorado),...,True,Ley VIF ó Decreto 97-96,Ignorado,2023-09-07,2023-09-11,3,20,905.0,20,905.0
3,Psicologica - Patrimonial,164,Víctima,Mujer,44,1.0,0.0,1.0,Alfabeta,Primaria (grado ignorado),...,NaN,Ignorado,Ignorado,2023-09-25,2023-09-25,4,15,413.0,15,413.0
4,Fisica,21,Víctima,Mujer,36,99.0,99.0,99.0,Alfabeta,Sexto Primaria,...,NaN,Ignorado,Ignorado,2023-01-20,2023-01-20,5,4,22.0,4,22.0


In [227]:
violenciaIntrafamiliar = violenciaIntrafamiliar.drop(columns=['VIC_OCUP','VIC_TRABAJA','VIC_DEDICA','VIC_DISC'])
violenciaIntrafamiliar.head()

,hecho_tipoAgre,NUMERO_BOLETA,quien_reporta,sexo_vic,VIC_EDAD,TOTAL_HIJOS,NUM_HIJ_HOM,NUM_HIJ_MUJ,alfabetismoVictima,escolaridad_vict,...,CONDUCENTE,ley_aplicable,organismo_remite,fecha_hecho,fecha_emision,id_VIF,id_departamento_hecho,id_municipio_hecho,id_departamento_emision,id_municipio_emision
0,Fisica - Psicologica,1072,Familiar de la víctima,Mujer,14,0.0,0.0,0.0,Alfabeta,Sexto Primaria,...,NaN,Ignorado,Ignorado,2023-07-22,2023-07-24,1,3,1601.0,3,1601.0
1,Sexual,558,Familiar de la víctima,Mujer,23,1.0,0.0,1.0,Alfabeta,Diversificado(Grado Ignorado),...,True,Ley VIF ó Decreto 97-96,Ignorado,2023-09-19,2023-09-29,2,1,115.0,1,108.0
2,Fisica,74,Víctima,Hombre,30,3.0,2.0,1.0,Alfabeta,Primaria (grado ignorado),...,True,Ley VIF ó Decreto 97-96,Ignorado,2023-09-07,2023-09-11,3,20,905.0,20,905.0
3,Psicologica - Patrimonial,164,Víctima,Mujer,44,1.0,0.0,1.0,Alfabeta,Primaria (grado ignorado),...,NaN,Ignorado,Ignorado,2023-09-25,2023-09-25,4,15,413.0,15,413.0
4,Fisica,21,Víctima,Mujer,36,99.0,99.0,99.0,Alfabeta,Sexto Primaria,...,NaN,Ignorado,Ignorado,2023-01-20,2023-01-20,5,4,22.0,4,22.0


In [228]:
violenciaIntrafamiliar = violenciaIntrafamiliar.drop(columns=['TIPO_DISCAQ','discapacidad_vic','VIC_REL_AGR','HEC_AREA','INST_DONDE_DENUNCIO','AGR_ALFAB','AGR_ESCOLARIDAD','AGR_EST_CIV','AGR_NACIONAL','AGR_TRABAJA','AGR_OCUP','INST_DENUN_HECHO'])
violenciaIntrafamiliar.head()

,hecho_tipoAgre,NUMERO_BOLETA,quien_reporta,sexo_vic,VIC_EDAD,TOTAL_HIJOS,NUM_HIJ_HOM,NUM_HIJ_MUJ,alfabetismoVictima,escolaridad_vict,...,CONDUCENTE,ley_aplicable,organismo_remite,fecha_hecho,fecha_emision,id_VIF,id_departamento_hecho,id_municipio_hecho,id_departamento_emision,id_municipio_emision
0,Fisica - Psicologica,1072,Familiar de la víctima,Mujer,14,0.0,0.0,0.0,Alfabeta,Sexto Primaria,...,NaN,Ignorado,Ignorado,2023-07-22,2023-07-24,1,3,1601.0,3,1601.0
1,Sexual,558,Familiar de la víctima,Mujer,23,1.0,0.0,1.0,Alfabeta,Diversificado(Grado Ignorado),...,True,Ley VIF ó Decreto 97-96,Ignorado,2023-09-19,2023-09-29,2,1,115.0,1,108.0
2,Fisica,74,Víctima,Hombre,30,3.0,2.0,1.0,Alfabeta,Primaria (grado ignorado),...,True,Ley VIF ó Decreto 97-96,Ignorado,2023-09-07,2023-09-11,3,20,905.0,20,905.0
3,Psicologica - Patrimonial,164,Víctima,Mujer,44,1.0,0.0,1.0,Alfabeta,Primaria (grado ignorado),...,NaN,Ignorado,Ignorado,2023-09-25,2023-09-25,4,15,413.0,15,413.0
4,Fisica,21,Víctima,Mujer,36,99.0,99.0,99.0,Alfabeta,Sexto Primaria,...,NaN,Ignorado,Ignorado,2023-01-20,2023-01-20,5,4,22.0,4,22.0


In [229]:
violenciaVictima = violenciaIntrafamiliar
violenciaVictima.head()

,hecho_tipoAgre,NUMERO_BOLETA,quien_reporta,sexo_vic,VIC_EDAD,TOTAL_HIJOS,NUM_HIJ_HOM,NUM_HIJ_MUJ,alfabetismoVictima,escolaridad_vict,...,CONDUCENTE,ley_aplicable,organismo_remite,fecha_hecho,fecha_emision,id_VIF,id_departamento_hecho,id_municipio_hecho,id_departamento_emision,id_municipio_emision
0,Fisica - Psicologica,1072,Familiar de la víctima,Mujer,14,0.0,0.0,0.0,Alfabeta,Sexto Primaria,...,NaN,Ignorado,Ignorado,2023-07-22,2023-07-24,1,3,1601.0,3,1601.0
1,Sexual,558,Familiar de la víctima,Mujer,23,1.0,0.0,1.0,Alfabeta,Diversificado(Grado Ignorado),...,True,Ley VIF ó Decreto 97-96,Ignorado,2023-09-19,2023-09-29,2,1,115.0,1,108.0
2,Fisica,74,Víctima,Hombre,30,3.0,2.0,1.0,Alfabeta,Primaria (grado ignorado),...,True,Ley VIF ó Decreto 97-96,Ignorado,2023-09-07,2023-09-11,3,20,905.0,20,905.0
3,Psicologica - Patrimonial,164,Víctima,Mujer,44,1.0,0.0,1.0,Alfabeta,Primaria (grado ignorado),...,NaN,Ignorado,Ignorado,2023-09-25,2023-09-25,4,15,413.0,15,413.0
4,Fisica,21,Víctima,Mujer,36,99.0,99.0,99.0,Alfabeta,Sexto Primaria,...,NaN,Ignorado,Ignorado,2023-01-20,2023-01-20,5,4,22.0,4,22.0


In [230]:
violenciaVictima = violenciaVictima.drop(columns=['hecho_tipoAgre','NUMERO_BOLETA','quien_reporta','CONDUCENTE','ley_aplicable','organismo_remite','fecha_hecho','fecha_emision'])

In [231]:
violenciaVictima.head()

,sexo_vic,VIC_EDAD,TOTAL_HIJOS,NUM_HIJ_HOM,NUM_HIJ_MUJ,alfabetismoVictima,escolaridad_vict,estado_civil_vic,pueblo_pertenencia_vic,nacionalidad_vic,...,AGR_OTRAS_MUJ,AGR_OTROS_N_OS,AGR_OTRAS_N_AS,institucion_denuncia.1,organismo_juri,id_VIF,id_departamento_hecho,id_municipio_hecho,id_departamento_emision,id_municipio_emision
0,Mujer,14,0.0,0.0,0.0,Alfabeta,Sexto Primaria,Soltero(a),Maya,Guatemalteca,...,0.0,0.0,0.0,Policía Nacional Civil,Ignoarado,1,3,1601.0,3,1601.0
1,Mujer,23,1.0,0.0,1.0,Alfabeta,Diversificado(Grado Ignorado),Casado(a),Ladino(a),Guatemalteca,...,0.0,1.0,0.0,Organismo Judicial,Juzgados de Primera Instancia de Familia,2,1,115.0,1,108.0
2,Hombre,30,3.0,2.0,1.0,Alfabeta,Primaria (grado ignorado),Ignorado,No indica,Guatemalteca,...,NaN,NaN,NaN,Organismo Judicial,Juzgado de Paz Penal de Conocimiento a Prevenc...,3,20,905.0,20,905.0
3,Mujer,44,1.0,0.0,1.0,Alfabeta,Primaria (grado ignorado),Ignorado,Ladino(a),Guatemalteca,...,0.0,0.0,0.0,Policía Nacional Civil,Ignoarado,4,15,413.0,15,413.0
4,Mujer,36,99.0,99.0,99.0,Alfabeta,Sexto Primaria,Casado(a),Ladino(a),Guatemalteca,...,NaN,NaN,NaN,Policía Nacional Civil,Ignoarado,5,4,22.0,4,22.0


In [232]:
violenciaVictima = violenciaVictima.drop(columns=['id_municipio_emision','id_departamento_emision','id_municipio_hecho','id_departamento_hecho','organismo_juri','institucion_denuncia','AGR_OTRAS_N_AS','AGR_OTROS_N_OS'])
violenciaVictima.head()

,sexo_vic,VIC_EDAD,TOTAL_HIJOS,NUM_HIJ_HOM,NUM_HIJ_MUJ,alfabetismoVictima,escolaridad_vict,estado_civil_vic,pueblo_pertenencia_vic,nacionalidad_vic,...,nacionalidad_agresor,agresor_trabaja,ocupacion_agresor,AGR_DEDICA,dedicacion_agresor,AGRESORES_OTROS_TOTAL,AGR_OTROS_HOM,AGR_OTRAS_MUJ,institucion_denuncia.1,id_VIF
0,Mujer,14,0.0,0.0,0.0,Alfabeta,Sexto Primaria,Soltero(a),Maya,Guatemalteca,...,Guatemalteca,No trabaja por un salario o ingreso,Ocupación ignorada,3,Estudiar,1.0,1.0,0.0,Policía Nacional Civil,1
1,Mujer,23,1.0,0.0,1.0,Alfabeta,Diversificado(Grado Ignorado),Casado(a),Ladino(a),Guatemalteca,...,Guatemalteca,No trabaja por un salario o ingreso,Ocupación ignorada,3,Estudiar,1.0,0.0,0.0,Organismo Judicial,2
2,Hombre,30,3.0,2.0,1.0,Alfabeta,Primaria (grado ignorado),Ignorado,No indica,Guatemalteca,...,Guatemalteca,No trabaja por un salario o ingreso,Ocupación ignorada,1,Quehaceres del hogar,NaN,NaN,NaN,Organismo Judicial,3
3,Mujer,44,1.0,0.0,1.0,Alfabeta,Primaria (grado ignorado),Ignorado,Ladino(a),Guatemalteca,...,Guatemalteca,Si trabaja por un salario o ingreso,Ocupación ignorada,9,Ignorado,0.0,0.0,0.0,Policía Nacional Civil,4
4,Mujer,36,99.0,99.0,99.0,Alfabeta,Sexto Primaria,Casado(a),Ladino(a),Guatemalteca,...,Guatemalteca,No trabaja por un salario o ingreso,Ocupación ignorada,3,Estudiar,NaN,NaN,NaN,Policía Nacional Civil,5


In [233]:
violenciaVictima = violenciaVictima.drop(columns=['institucion_denuncia.1','AGR_OTRAS_MUJ','AGR_OTROS_HOM','AGRESORES_OTROS_TOTAL','AGR_DEDICA','ocupacion_agresor','agresor_trabaja','nacionalidad_agresor'])
violenciaVictima.head()

,sexo_vic,VIC_EDAD,TOTAL_HIJOS,NUM_HIJ_HOM,NUM_HIJ_MUJ,alfabetismoVictima,escolaridad_vict,estado_civil_vic,pueblo_pertenencia_vic,nacionalidad_vic,...,AGR_SEXO,sexo_agresor,AGR_EDAD,alfabetismo_agresor,escolaridad_agresor,agresor_estad_civil,AGR_GURPET,pueglo_agresor,dedicacion_agresor,id_VIF
0,Mujer,14,0.0,0.0,0.0,Alfabeta,Sexto Primaria,Soltero(a),Maya,Guatemalteca,...,1,Hombre,11.0,Alfabeta,Quinto Primaria,Ignorado,2,Maya,Estudiar,1
1,Mujer,23,1.0,0.0,1.0,Alfabeta,Diversificado(Grado Ignorado),Casado(a),Ladino(a),Guatemalteca,...,1,Hombre,12.0,Alfabeta,Primaria (grado ignorado),Soltero(a),1,Ladino(a),Estudiar,2
2,Hombre,30,3.0,2.0,1.0,Alfabeta,Primaria (grado ignorado),Ignorado,No indica,Guatemalteca,...,2,Mujer,12.0,Ignorado,Ignorado (tanto nivel como grado),Ignorado,5,Otro,Quehaceres del hogar,3
3,Mujer,44,1.0,0.0,1.0,Alfabeta,Primaria (grado ignorado),Ignorado,Ladino(a),Guatemalteca,...,1,Hombre,12.0,Ignorado,Ignorado (tanto nivel como grado),Ignorado,2,Maya,Ignorado,4
4,Mujer,36,99.0,99.0,99.0,Alfabeta,Sexto Primaria,Casado(a),Ladino(a),Guatemalteca,...,2,Mujer,13.0,Alfabeta,Tercero Primaria,Soltero(a),1,Ladino(a),Estudiar,5


In [234]:
columnas_a_eliminar = [
    'AGR_SEXO',
    'sexo_agresor',
    'AGR_EDAD',
    'alfabetismo_agresor',
    'escolaridad_agresor',
    'agresor_estad_civil',
    'AGR_GURPET',
    'pueglo_agresor',
    'dedicacion_agresor'
]

violenciaVictima = violenciaVictima.drop(columns=columnas_a_eliminar, errors='ignore')

print(violenciaVictima.columns)

Index(['sexo_vic', 'VIC_EDAD', 'TOTAL_HIJOS', 'NUM_HIJ_HOM', 'NUM_HIJ_MUJ',
       'alfabetismoVictima', 'escolaridad_vict', 'estado_civil_vic',
       'pueblo_pertenencia_vic', 'nacionalidad_vic', 'trabajo_victima',
       'ocupaacion_vic', 'dedica_vic', 'tipo_discapacidad_vic',
       'relafion_agresor_victima', 'OTRAS_VICTIMAS', 'VIC_OTRAS_HOM',
       'VIC_OTRAS_MUJ', 'VIC_OTRAS_N_OS', 'VIC_OTRAS_N_AS', 'area_heccho',
       'HEC_RECUR_DENUN', 'id_VIF'],
      dtype='object')


In [235]:
columnas_a_eliminar = ['area_heccho',
       'HEC_RECUR_DENUN']
violenciaVictima = violenciaVictima.drop(columns=columnas_a_eliminar, errors='ignore')

violenciaVictima

,sexo_vic,VIC_EDAD,TOTAL_HIJOS,NUM_HIJ_HOM,NUM_HIJ_MUJ,alfabetismoVictima,escolaridad_vict,estado_civil_vic,pueblo_pertenencia_vic,nacionalidad_vic,...,ocupaacion_vic,dedica_vic,tipo_discapacidad_vic,relafion_agresor_victima,OTRAS_VICTIMAS,VIC_OTRAS_HOM,VIC_OTRAS_MUJ,VIC_OTRAS_N_OS,VIC_OTRAS_N_AS,id_VIF
0,Mujer,14,0.0,0.0,0.0,Alfabeta,Sexto Primaria,Soltero(a),Maya,Guatemalteca,...,Ocupación ignorada,Quehaceres del hogar,sin discapacidad,Otro pariente,0.0,0.0,0.0,0.0,0.0,1
1,Mujer,23,1.0,0.0,1.0,Alfabeta,Diversificado(Grado Ignorado),Casado(a),Ladino(a),Guatemalteca,...,Ocupación ignorada,Quehaceres del hogar,sin discapacidad,Otro pariente,3.0,1.0,1.0,0.0,1.0,2
2,Hombre,30,3.0,2.0,1.0,Alfabeta,Primaria (grado ignorado),Ignorado,No indica,Guatemalteca,...,Agricultores y trabajadores calificados de cul...,Ignorado,sin discapacidad,Conviviente,NaN,NaN,NaN,NaN,NaN,3
3,Mujer,44,1.0,0.0,1.0,Alfabeta,Primaria (grado ignorado),Ignorado,Ladino(a),Guatemalteca,...,Ocupación ignorada,Quehaceres del hogar,sin discapacidad,Exconviviente,0.0,0.0,0.0,0.0,0.0,4
4,Mujer,36,99.0,99.0,99.0,Alfabeta,Sexto Primaria,Casado(a),Ladino(a),Guatemalteca,...,Ocupación ignorada,Quehaceres del hogar,sin discapacidad,Padre/Madre,NaN,NaN,NaN,NaN,NaN,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37343,Mujer,40,99.0,99.0,99.0,NaN,NaN,Ignorado,No indica,Guatemalteca,...,Ocupaciones no especificadas en las anteriores,Ignorado,sin discapacidad,Exconviviente,NaN,NaN,NaN,NaN,NaN,37344
37344,Hombre,49,99.0,99.0,99.0,NaN,NaN,Casado(a),No indica,Extranjera,...,Representantes comerciales,Ignorado,sin discapacidad,Exconviviente,1.0,0.0,1.0,0.0,0.0,37345
37345,Mujer,29,2.0,0.0,2.0,NaN,NaN,Ignorado,Ladino(a),Guatemalteca,...,Otro personal de limpieza,Ignorado,sin discapacidad,Conviviente,NaN,NaN,NaN,NaN,NaN,37346
37346,Mujer,20,99.0,99.0,99.0,NaN,NaN,Ignorado,No indica,Guatemalteca,...,Ocupación ignorada,Quehaceres del hogar,Ignorado,Exconviviente,NaN,NaN,NaN,NaN,NaN,37347


In [236]:
violenciaVictima['id_sexo'] = violenciaVictima['sexo_vic'].map({v: k for k, v in sexo_map.items()})
violenciaVictima = violenciaVictima.drop(columns=['sexo_vic'])

violenciaVictima['VIC_EDAD'] = violenciaVictima['VIC_EDAD'].apply(normalizar_edad).astype('Int64')
violenciaVictima['TOTAL_HIJOS'] = violenciaVictima['TOTAL_HIJOS'].astype('Int64')
violenciaVictima['NUM_HIJ_HOM'] = violenciaVictima['NUM_HIJ_HOM'].apply(normalizar_edad).astype('Int64')
violenciaVictima['NUM_HIJ_MUJ'] = violenciaVictima['NUM_HIJ_MUJ'].astype('Int64')

violenciaVictima['OTRAS_VICTIMAS'] = violenciaVictima['OTRAS_VICTIMAS'].apply(normalizar_edad).astype('Int64')
violenciaVictima['VIC_OTRAS_HOM'] = violenciaVictima['VIC_OTRAS_HOM'].astype('Int64')
violenciaVictima['VIC_OTRAS_MUJ'] = violenciaVictima['VIC_OTRAS_MUJ'].apply(normalizar_edad).astype('Int64')
violenciaVictima['VIC_OTRAS_N_OS'] = violenciaVictima['VIC_OTRAS_N_OS'].astype('Int64')
violenciaVictima['VIC_OTRAS_N_AS'] = violenciaVictima['VIC_OTRAS_N_AS'].astype('Int64')

violenciaVictima['relafion_agresor_victima']=violenciaVictima['relafion_agresor_victima'].apply(normalizar_texto)
violenciaVictima['tipo_discapacidad_vic']=violenciaVictima['tipo_discapacidad_vic'].apply(normalizar_texto)
violenciaVictima['dedica_vic']=violenciaVictima['dedica_vic'].apply(normalizar_texto)
violenciaVictima['ocupaacion_vic']=violenciaVictima['ocupaacion_vic'].apply(normalizar_texto)

violenciaVictima['nacionalidad_vic']=violenciaVictima['nacionalidad_vic'].apply(normalizar_texto)
violenciaVictima['pueblo_pertenencia_vic']=violenciaVictima['pueblo_pertenencia_vic'].apply(normalizar_texto)
violenciaVictima['escolaridad_vict']=violenciaVictima['escolaridad_vict'].apply(normalizar_texto)
violenciaVictima['alfabetismoVictima']=violenciaVictima['alfabetismoVictima'].apply(normalizar_texto)

violenciaVictima.head()

,VIC_EDAD,TOTAL_HIJOS,NUM_HIJ_HOM,NUM_HIJ_MUJ,alfabetismoVictima,escolaridad_vict,estado_civil_vic,pueblo_pertenencia_vic,nacionalidad_vic,trabajo_victima,...,dedica_vic,tipo_discapacidad_vic,relafion_agresor_victima,OTRAS_VICTIMAS,VIC_OTRAS_HOM,VIC_OTRAS_MUJ,VIC_OTRAS_N_OS,VIC_OTRAS_N_AS,id_VIF,id_sexo
0,14,0,0,0,Alfabeta,Sexto Primaria,Soltero(a),Maya,Guatemalteca,No trabaja por un salario o ingreso,...,Quehaceres Del Hogar,Sin Discapacidad,Otro Pariente,0,0,0,0,0,1,2
1,23,1,0,1,Alfabeta,Diversificado(Grado Ignorado),Casado(a),Ladino(A),Guatemalteca,No trabaja por un salario o ingreso,...,Quehaceres Del Hogar,Sin Discapacidad,Otro Pariente,3,1,1,0,1,2,2
2,30,3,2,1,Alfabeta,Primaria (Grado Ignorado),Ignorado,No Indica,Guatemalteca,Si trabaja por un salario o ingreso,...,Ignorado,Sin Discapacidad,Conviviente,<NA>,<NA>,<NA>,<NA>,<NA>,3,1
3,44,1,0,1,Alfabeta,Primaria (Grado Ignorado),Ignorado,Ladino(A),Guatemalteca,No trabaja por un salario o ingreso,...,Quehaceres Del Hogar,Sin Discapacidad,Exconviviente,0,0,0,0,0,4,2
4,36,99,99,99,Alfabeta,Sexto Primaria,Casado(a),Ladino(A),Guatemalteca,No trabaja por un salario o ingreso,...,Quehaceres Del Hogar,Sin Discapacidad,Padre/Madre,<NA>,<NA>,<NA>,<NA>,<NA>,5,2


In [237]:
violenciaVictima['id_alfabetismo'] = violenciaVictima['alfabetismoVictima'].map({v: k for k, v in alfabetismo_map.items()})
violenciaVictima = violenciaVictima.drop(columns=['alfabetismoVictima','TOTAL_HIJOS'])
violenciaVictima['id_estado_civil'] = violenciaVictima['estado_civil_vic'].map({v: k for k, v in estado_conyugal_map.items()})
violenciaVictima = violenciaVictima.drop(columns=['estado_civil_vic'])
violenciaVictima.head()

,VIC_EDAD,NUM_HIJ_HOM,NUM_HIJ_MUJ,escolaridad_vict,pueblo_pertenencia_vic,nacionalidad_vic,trabajo_victima,ocupaacion_vic,dedica_vic,tipo_discapacidad_vic,relafion_agresor_victima,OTRAS_VICTIMAS,VIC_OTRAS_HOM,VIC_OTRAS_MUJ,VIC_OTRAS_N_OS,VIC_OTRAS_N_AS,id_VIF,id_sexo,id_alfabetismo,id_estado_civil
0,14,0,0,Sexto Primaria,Maya,Guatemalteca,No trabaja por un salario o ingreso,Ocupacion Ignorada,Quehaceres Del Hogar,Sin Discapacidad,Otro Pariente,0,0,0,0,0,1,2,1.0,NaN
1,23,0,1,Diversificado(Grado Ignorado),Ladino(A),Guatemalteca,No trabaja por un salario o ingreso,Ocupacion Ignorada,Quehaceres Del Hogar,Sin Discapacidad,Otro Pariente,3,1,1,0,1,2,2,1.0,NaN
2,30,2,1,Primaria (Grado Ignorado),No Indica,Guatemalteca,Si trabaja por un salario o ingreso,Agricultores Y Trabajadores Calificados De Cul...,Ignorado,Sin Discapacidad,Conviviente,<NA>,<NA>,<NA>,<NA>,<NA>,3,1,1.0,6.0
3,44,0,1,Primaria (Grado Ignorado),Ladino(A),Guatemalteca,No trabaja por un salario o ingreso,Ocupacion Ignorada,Quehaceres Del Hogar,Sin Discapacidad,Exconviviente,0,0,0,0,0,4,2,1.0,6.0
4,36,99,99,Sexto Primaria,Ladino(A),Guatemalteca,No trabaja por un salario o ingreso,Ocupacion Ignorada,Quehaceres Del Hogar,Sin Discapacidad,Padre/Madre,<NA>,<NA>,<NA>,<NA>,<NA>,5,2,1.0,NaN


In [239]:
violenciaIntrafamiliar = violenciaIntrafamiliar.drop(columns=['sexo_vic',
       'VIC_EDAD', 'TOTAL_HIJOS', 'NUM_HIJ_HOM', 'NUM_HIJ_MUJ',
       'alfabetismoVictima', 'escolaridad_vict', 'estado_civil_vic',
       'pueblo_pertenencia_vic', 'nacionalidad_vic', 'trabajo_victima',
       'ocupaacion_vic', 'dedica_vic', 'tipo_discapacidad_vic',
       'relafion_agresor_victima', 'OTRAS_VICTIMAS', 'VIC_OTRAS_HOM',
       'VIC_OTRAS_MUJ', 'VIC_OTRAS_N_OS', 'VIC_OTRAS_N_AS'])
violenciaIntrafamiliar.columns

Index(['hecho_tipoAgre', 'NUMERO_BOLETA', 'quien_reporta', 'area_heccho',
       'HEC_RECUR_DENUN', 'institucion_denuncia', 'AGR_SEXO', 'sexo_agresor',
       'AGR_EDAD', 'alfabetismo_agresor', 'escolaridad_agresor',
       'agresor_estad_civil', 'AGR_GURPET', 'pueglo_agresor',
       'nacionalidad_agresor', 'agresor_trabaja', 'ocupacion_agresor',
       'AGR_DEDICA', 'dedicacion_agresor', 'AGRESORES_OTROS_TOTAL',
       'AGR_OTROS_HOM', 'AGR_OTRAS_MUJ', 'AGR_OTROS_N_OS', 'AGR_OTRAS_N_AS',
       'institucion_denuncia.1', 'organismo_juri', 'CONDUCENTE',
       'ley_aplicable', 'organismo_remite', 'fecha_hecho', 'fecha_emision',
       'id_VIF', 'id_departamento_hecho', 'id_municipio_hecho',
       'id_departamento_emision', 'id_municipio_emision'],
      dtype='object')

In [240]:
violenciaAgresor = violenciaIntrafamiliar
violenciaAgresor = violenciaAgresor.drop(columns=['hecho_tipoAgre', 'NUMERO_BOLETA', 'quien_reporta', 'area_heccho',
       'HEC_RECUR_DENUN', 'institucion_denuncia','AGR_DEDICA','institucion_denuncia.1', 'organismo_juri', 'CONDUCENTE',
       'ley_aplicable', 'organismo_remite', 'fecha_hecho', 'fecha_emision','id_departamento_hecho', 'id_municipio_hecho',
       'id_departamento_emision', 'id_municipio_emision'])
violenciaAgresor.head()

,AGR_SEXO,sexo_agresor,AGR_EDAD,alfabetismo_agresor,escolaridad_agresor,agresor_estad_civil,AGR_GURPET,pueglo_agresor,nacionalidad_agresor,agresor_trabaja,ocupacion_agresor,dedicacion_agresor,AGRESORES_OTROS_TOTAL,AGR_OTROS_HOM,AGR_OTRAS_MUJ,AGR_OTROS_N_OS,AGR_OTRAS_N_AS,id_VIF
0,1,Hombre,11.0,Alfabeta,Quinto Primaria,Ignorado,2,Maya,Guatemalteca,No trabaja por un salario o ingreso,Ocupación ignorada,Estudiar,1.0,1.0,0.0,0.0,0.0,1
1,1,Hombre,12.0,Alfabeta,Primaria (grado ignorado),Soltero(a),1,Ladino(a),Guatemalteca,No trabaja por un salario o ingreso,Ocupación ignorada,Estudiar,1.0,0.0,0.0,1.0,0.0,2
2,2,Mujer,12.0,Ignorado,Ignorado (tanto nivel como grado),Ignorado,5,Otro,Guatemalteca,No trabaja por un salario o ingreso,Ocupación ignorada,Quehaceres del hogar,NaN,NaN,NaN,NaN,NaN,3
3,1,Hombre,12.0,Ignorado,Ignorado (tanto nivel como grado),Ignorado,2,Maya,Guatemalteca,Si trabaja por un salario o ingreso,Ocupación ignorada,Ignorado,0.0,0.0,0.0,0.0,0.0,4
4,2,Mujer,13.0,Alfabeta,Tercero Primaria,Soltero(a),1,Ladino(a),Guatemalteca,No trabaja por un salario o ingreso,Ocupación ignorada,Estudiar,NaN,NaN,NaN,NaN,NaN,5


In [241]:
violenciaAgresor = violenciaAgresor.drop(columns=['AGR_SEXO','AGR_GURPET'])

In [242]:
violenciaIntrafamiliar.columns

Index(['hecho_tipoAgre', 'NUMERO_BOLETA', 'quien_reporta', 'area_heccho',
       'HEC_RECUR_DENUN', 'institucion_denuncia', 'AGR_SEXO', 'sexo_agresor',
       'AGR_EDAD', 'alfabetismo_agresor', 'escolaridad_agresor',
       'agresor_estad_civil', 'AGR_GURPET', 'pueglo_agresor',
       'nacionalidad_agresor', 'agresor_trabaja', 'ocupacion_agresor',
       'AGR_DEDICA', 'dedicacion_agresor', 'AGRESORES_OTROS_TOTAL',
       'AGR_OTROS_HOM', 'AGR_OTRAS_MUJ', 'AGR_OTROS_N_OS', 'AGR_OTRAS_N_AS',
       'institucion_denuncia.1', 'organismo_juri', 'CONDUCENTE',
       'ley_aplicable', 'organismo_remite', 'fecha_hecho', 'fecha_emision',
       'id_VIF', 'id_departamento_hecho', 'id_municipio_hecho',
       'id_departamento_emision', 'id_municipio_emision'],
      dtype='object')

In [243]:
violenciaIntrafamiliar = violenciaIntrafamiliar.drop(columns=['AGR_SEXO', 'sexo_agresor',
       'AGR_EDAD', 'alfabetismo_agresor', 'escolaridad_agresor',
       'agresor_estad_civil', 'AGR_GURPET', 'pueglo_agresor',
       'nacionalidad_agresor', 'agresor_trabaja', 'ocupacion_agresor',
       'AGR_DEDICA', 'dedicacion_agresor', 'AGRESORES_OTROS_TOTAL',
       'AGR_OTROS_HOM', 'AGR_OTRAS_MUJ', 'AGR_OTROS_N_OS', 'AGR_OTRAS_N_AS'])

violenciaIntrafamiliar

,hecho_tipoAgre,NUMERO_BOLETA,quien_reporta,area_heccho,HEC_RECUR_DENUN,institucion_denuncia,institucion_denuncia.1,organismo_juri,CONDUCENTE,ley_aplicable,organismo_remite,fecha_hecho,fecha_emision,id_VIF,id_departamento_hecho,id_municipio_hecho,id_departamento_emision,id_municipio_emision
0,Fisica - Psicologica,1072,Familiar de la víctima,Rural,False,NaN,Policía Nacional Civil,Ignoarado,NaN,Ignorado,Ignorado,2023-07-22,2023-07-24,1,3,1601.0,3,1601.0
1,Sexual,558,Familiar de la víctima,Urbana,True,Organismo Judicial,Organismo Judicial,Juzgados de Primera Instancia de Familia,True,Ley VIF ó Decreto 97-96,Ignorado,2023-09-19,2023-09-29,2,1,115.0,1,108.0
2,Fisica,74,Víctima,Rural,False,NaN,Organismo Judicial,Juzgado de Paz Penal de Conocimiento a Prevenc...,True,Ley VIF ó Decreto 97-96,Ignorado,2023-09-07,2023-09-11,3,20,905.0,20,905.0
3,Psicologica - Patrimonial,164,Víctima,Urbana,False,NaN,Policía Nacional Civil,Ignoarado,NaN,Ignorado,Ignorado,2023-09-25,2023-09-25,4,15,413.0,15,413.0
4,Fisica,21,Víctima,Rural,False,NaN,Policía Nacional Civil,Ignoarado,NaN,Ignorado,Ignorado,2023-01-20,2023-01-20,5,4,22.0,4,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37343,Fisica - Psicologica,509,Víctima,Urbana,NaN,Ignorado,Organismo Judicial,Juzgados de Paz,True,Ambas leyes (VIF y VCM),Ministerio Público (como órgano estatal),2023-12-10,2023-12-11,37344,1,113.0,1,113.0
37344,Psicologica,514,Víctima,Urbana,False,NaN,Organismo Judicial,Juzgados de Paz,False,Ley VIF ó Decreto 97-96,Ignorado,2023-12-12,2023-12-13,37345,1,117.0,1,113.0
37345,Psicologica,559,Víctima,Urbana,False,NaN,Organismo Judicial,Juzgados de Primera Instancia de Familia,NaN,Ley VIF ó Decreto 97-96,Ignorado,2023-12-17,2023-12-21,37346,1,114.0,1,114.0
37346,Psicologica,599,Víctima,Urbana,True,Ignorado,Organismo Judicial,Juzgados de Primera Instancia de Familia,NaN,Ignorado,Ignorado,2023-05-26,2023-12-28,37347,1,115.0,1,115.0


In [245]:
violenciaIntrafamiliar.to_csv('violenciaIntrafamiliar.csv', index=False)
violenciaVictima.to_csv('violenciaVictima.csv', index=False)
violenciaAgresor.to_csv('violenciaAgresor.csv', index=False)